# Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from nlb_tools.nwb_interface import NWBDataset
from scipy.stats import pearsonr
import copy
from scipy.stats import gaussian_kde
from scipy.interpolate import splprep, splev
from scipy.signal import convolve2d
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from functools import reduce
import operator
import time
import numbers
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
import math
from scipy._lib._util import _contains_nan
import scipy.special
import random
import warnings
from scipy.spatial.distance import pdist, squareform
from scipy.stats import norm
from IPython.display import Audio
import os
from matplotlib.ticker import AutoMinorLocator
from scipy import stats

In [ ]:
'''
Adds to the trial info dataset the time of the reach (also adds the target position)
'''
def add_reachTime(dataset):
    actual_dataset = dataset.make_trial_data(start_field='move_onset_time')
    
    cursor_pos = np.asarray(actual_dataset['cursor_pos'])
    trials_id = np.asarray(actual_dataset['trial_id'])
    mask = np.concatenate(([True], trials_id[1:] != trials_id[:-1]))
    targets = np.asarray(dataset.trial_info['target_pos'])
    active_id = np.asarray(dataset.trial_info['active_target'])
    target_pos = [target[active_id[i]] for i, target in enumerate(targets)]
    target_pos = np.array(target_pos, dtype='int64')
    started_time = np.array(actual_dataset['align_time'])
    move_time = np.array(dataset.trial_info['move_onset_time'])
    
    diffs = []
    for i in range(len(cursor_pos)):
        diffs += [np.sqrt(np.power(target_pos[trials_id[i]][0] - cursor_pos[i][0], 2)+
                          np.power(target_pos[trials_id[i]][1] - cursor_pos[i][1], 2))]
    
    diffs = np.split(diffs, np.where(mask)[0])[1:]
    started_time = np.split(started_time, np.where(mask)[0])[1:]
    reach_time = []
    
    for i in range(len(target_pos)):
        reach_time += [started_time[i][np.argmin(diffs[i])]+move_time[i]]
        
    dataset.trial_info['reach_time'] = reach_time
    dataset.trial_info['active_pos_x'] = target_pos[:,0]
    dataset.trial_info['active_pos_y'] = target_pos[:,1]

Standard dataset path: /Users/guilhermec.f/000128/sub-Jenkins
<br>
It takes 93-133 sec to run

Large dataset path: /Users/guilhermec.f/000138/sub-Jenkins
<br>
It takes 6 sec to run

Medium dataset path: /Users/guilhermec.f/000139/sub-Jenkins
<br>
It takes 3 sec to run

Small dataset path: /Users/guilhermec.f/000140/sub-Jenkins
<br>
It takes 1 sec to run

In [ ]:
standardPath = 'your path'
standardDS = NWBDataset(standardPath, "*train", split_heldout=False)
standardDS.resample(5) #resizes bins to 5ms (only once needed)
standard_conds = standardDS.trial_info.set_index(['trial_type', 'trial_version']).index.unique().tolist()
standard_neurons = standardDS.data['spikes'].columns.tolist() #Gets the name of all neurons
add_reachTime(standardDS)

In [ ]:
largePath = 'your path'
largeDS = NWBDataset(largePath, "*train", split_heldout=False)
largeDS.resample(5) #resizes bins to 5ms (only once needed)
large_conds = largeDS.trial_info.set_index(['trial_type', 'trial_version']).index.unique().tolist()
large_neurons = largeDS.data['spikes'].columns.tolist() #Gets the name of all neurons
add_reachTime(largeDS)

In [ ]:
mediumPath = 'your path'
mediumDS = NWBDataset(mediumPath, "*train", split_heldout=False)
mediumDS.resample(5)
medium_conds = mediumDS.trial_info.set_index(['trial_type', 'trial_version']).index.unique().tolist()
medium_neurons = mediumDS.data['spikes'].columns.tolist() #Gets the name of all neurons
add_reachTime(mediumDS)

In [ ]:
smallPath = 'your path'
smallDS = NWBDataset(smallPath, "*train", split_heldout=False)
smallDS.resample(5) #resizes bins to 5ms (only once needed)
conds = smallDS.trial_info.set_index(['trial_type', 'trial_version']).index.unique().tolist()
small_neurons = smallDS.data['spikes'].columns.tolist() #Gets the name of all neurons
add_reachTime(smallDS)

# Functions from the Demo Notebook

In [ ]:
# Plots the PSTH with STDG for a given condition for a given neuron
def plot_givenPSTHs(dataset, condition, neuron = -1, seed=2468):
    
    # Seed generator for consistent plots
    np.random.seed(seed)
    
    # Check if 'spikes_smth_50' column already exists
    if 'spikes_smth_50' not in dataset.data.columns:
        # Smooth spikes with 50 ms std Gaussian
        dataset.smooth_spk(50, name='smth_50')
    
    if(neuron == -1):
        neur_num = np.random.choice(dataset.data.spikes.columns)
    else:
        neur_num = neuron

    print(neur_num)
    
    # Find unique conditions
    conds = dataset.trial_info.set_index(['trial_type', 'trial_version']).index.unique().tolist()
    print("Number of conditions", len(conds))
    
    if(type(condition) == int):
        n_conds = condition
        test = np.random.choice(len(conds), size=n_conds, replace=False)

    # Plot random subset of conditions
        for i in test:
            if(type(condition) == int): cond = conds[i]
            else: cond = i
            # Find trials in condition
            mask = np.all(dataset.trial_info[['trial_type', 'trial_version']] == cond, axis=1)
            # Extract trial data
            trial_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-50, 450), ignored_trials=(~mask))
            # Average hand position across trials
            psth = trial_data.groupby('align_time')[[('spikes_smth_50', neur_num)]].mean().to_numpy() / dataset.bin_width * 1000
            # Color PSTHs by reach angle
            active_target = dataset.trial_info[mask].target_pos.iloc[0][dataset.trial_info[mask].active_target.iloc[0]]
            reach_angle = np.arctan2(*active_target[::-1])
            # Plot reach
            plt.plot(np.arange(-50, 450, dataset.bin_width), psth, label=cond, color=plt.cm.hsv(reach_angle / (2*np.pi) + 0.5))
    else:
        # Plot random subset of conditions
        for i in condition:
            cond = i
            # Find trials in condition
            mask = np.all(dataset.trial_info[['trial_type', 'trial_version']] == cond, axis=1)
            # Extract trial data
            trial_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-50, 450), ignored_trials=(~mask))
            # Average hand position across trials
            psth = trial_data.groupby('align_time')[[('spikes_smth_50', neur_num)]].mean().to_numpy() / dataset.bin_width * 1000
            # Color PSTHs by reach angle
            active_target = dataset.trial_info[mask].target_pos.iloc[0][dataset.trial_info[mask].active_target.iloc[0]]
            reach_angle = np.arctan2(*active_target[::-1])
            # Plot reach
            plt.plot(np.arange(-50, 450, dataset.bin_width), psth, label=cond, color=plt.cm.hsv(reach_angle / (2*np.pi) + 0.5))
    
    # Add labels
    plt.ylim(bottom=0)
    plt.xlabel('Time after movement onset (ms)')
    plt.ylabel('Firing rate (spk/s)')
    plt.title(f'Neur {neur_num} PSTH')
    plt.legend(title='condition', loc='upper right')
    plt.show()

In [ ]:
# Plots path for all conditions
def plot_path(dataset, conditions=-1):
    label_check = False
    if conditions == -1:
        conds = dataset.trial_info.set_index(['trial_type', 'trial_version']).index.unique().tolist()
    else:
        conds = conditions
        label_check = True
    
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
    
    for i, cond in enumerate(conds):
        mask = np.all(dataset.trial_info[['trial_type', 'trial_version']] == cond, axis=1)
        trial_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-50, 450), ignored_trials=(~mask))
        traj = trial_data.groupby('align_time')[[('hand_pos', 'x'), ('hand_pos', 'y')]].mean().to_numpy()
        active_target = dataset.trial_info[mask].target_pos.iloc[0][dataset.trial_info[mask].active_target.iloc[0]]
        reach_angle = np.arctan2(*active_target[::-1])
        
        if(label_check): ax.plot(traj[:, 0], traj[:, 1], linewidth=0.7, color=plt.cm.hsv(reach_angle / (2*np.pi) + 0.5), label=f'{conds[i]}')
        else: ax.plot(traj[:, 0], traj[:, 1], linewidth=0.7, color=plt.cm.hsv(reach_angle / (2*np.pi) + 0.5)) 
    
    if(label_check): plt.legend(title='Condition', loc='upper right')
    plt.show()

In [ ]:
# Fits the data to a ridge regression, compares the results with R2 and spits out a DS for analysing the predictions
def kinematic_decoding_eval (dataset):
    # Extract neural data and lagged hand velocity
    trial_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-130, 370))
    lagged_trial_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-50, 450))
    rates = trial_data.spikes_smth_50.to_numpy()
    vel = lagged_trial_data.hand_vel.to_numpy()
    
    # Fit and evaluate decoder
    gscv = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 0, 5)})
    gscv.fit(rates, vel)
    pred_vel = gscv.predict(rates)
    print(f"Decoding R2: {gscv.best_score_}")
    print("Best alpha value:", gscv.best_params_['alpha'])

    #Saves the extracted predictions in a new DS
    predictedDS = copy.deepcopy(dataset)
    
    # Merge predictions back to continuous data
    pred_vel_df = pd.DataFrame(pred_vel, index=lagged_trial_data.clock_time, columns=pd.MultiIndex.from_tuples([('pred_vel', 'x'), ('pred_vel', 'y')]))
    predictedDS.data = pd.concat([predictedDS.data, pred_vel_df], axis=1)

    return predictedDS

In [ ]:
# Plot predicted vs true kinematics
def plot_comparison(dataset, condition):
    conds = dataset.trial_info.set_index(['trial_type', 'trial_version']).index.unique().tolist()
    cond = conds[condition]
    
    # Find trials in condition and extract data
    mask = np.all(dataset.trial_info[['trial_type', 'trial_version']] == cond, axis=1)
    trial_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-50, 450), ignored_trials=(~mask))
    
    # Initialize figure
    fig, axs = plt.subplots(2, 3, figsize=(10, 4))
    t = np.arange(-50, 450, dataset.bin_width)
    
    # Loop through trials in condition
    for _, trial in trial_data.groupby('trial_id'):
        # True and predicted x velocity
        axs[0][0].plot(t, trial.hand_vel.x, linewidth=0.7, color='black')
        axs[1][0].plot(t, trial.pred_vel.x, linewidth=0.7, color='blue')
        # True and predicted y velocity
        axs[0][1].plot(t, trial.hand_vel.y, linewidth=0.7, color='black')
        axs[1][1].plot(t, trial.pred_vel.y, linewidth=0.7, color='blue')
        # True and predicted trajectories
        true_traj = np.cumsum(trial.hand_vel.to_numpy(), axis=0) * dataset.bin_width / 1000
        pred_traj = np.cumsum(trial.pred_vel.to_numpy(), axis=0) * dataset.bin_width / 1000
        axs[0][2].plot(true_traj[:, 0], true_traj[:, 1], linewidth=0.7, color='black')
        axs[1][2].plot(pred_traj[:, 0], pred_traj[:, 1], linewidth=0.7, color='blue')
    
    # Set up shared axes
    for i in range(2):
        axs[i][0].set_xlim(-50, 450)
        axs[i][1].set_xlim(-50, 450)
        axs[i][2].set_xlim(-180, 180)
        axs[i][2].set_ylim(-130, 130)
    
    # Add labels
    axs[0][0].set_title('X velocity (mm/s)')
    axs[0][1].set_title('Y velocity (mm/s)')
    axs[0][2].set_title('Reach trajectory')
    plt.show()

In [ ]:
# Plot neural trajectories for subset of conditions
def plot_neuralTrajectories(dataset, seed=2021):
    # Seed generator for consistent plots
    np.random.seed(seed)
    n_conds = 27 # number of conditions to plot
    
    # Get unique conditions
    conds = dataset.trial_info.set_index(['trial_type', 'trial_version']).index.unique().tolist()
    
    # Loop through conditions
    rates = []
    colors = []
    for i in np.random.choice(len(conds), n_conds):
        cond = conds[i]
        # Find trials in condition
        mask = np.all(dataset.trial_info[['trial_type', 'trial_version']] == cond, axis=1)
        # Extract trial data
        trial_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-50, 450), ignored_trials=(~mask))
        # Append averaged smoothed spikes for condition
        rates.append(trial_data.groupby('align_time')[trial_data[['spikes_smth_50']].columns].mean().to_numpy())
        # Append reach angle-based color for condition
        active_target = dataset.trial_info[mask].target_pos.iloc[0][dataset.trial_info[mask].active_target.iloc[0]]
        reach_angle = np.arctan2(*active_target[::-1])
        colors.append(plt.cm.hsv(reach_angle / (2*np.pi) + 0.5))
    
    # Stack data and apply PCA
    rate_stack = np.vstack(rates)
    rate_scaled = StandardScaler().fit_transform(rate_stack)
    pca = PCA(n_components=3)
    traj_stack = pca.fit_transform(rate_scaled)
    traj_arr = traj_stack.reshape((n_conds, len(rates[0]), -1))
    
    # Loop through trajectories and plot
    fig = plt.figure(figsize=(9, 6))
    ax = fig.add_subplot(111, projection='3d')
    for traj, col in zip(traj_arr, colors):
        ax.plot(traj[:, 0], traj[:, 1], traj[:, 2], color=col)
        ax.scatter(traj[0, 0], traj[0, 1], traj[0, 2], color=col) 
    
    # Add labels
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    ax.set_zlabel('PC3')
    plt.show()

# Final Functions

### Makes Datasets per Trial

In [ ]:
'''
Returns the dataset for only one trial
'''
def trial_datasetMaker(dataset, trials):
    if(len(trials) == 1):
        trials_possible = trials_present(dataset)
        check_mask = np.isin(trials, trials_possible)[0]
    
        if(check_mask):
            columns = dataset.columns
            matrix = np.asarray(dataset)
            
            trial_ids = np.array(dataset['trial_id'])
            mask = trial_ids == trials[0]
            matrix = matrix[mask]
            trial_dataset = pd.DataFrame(matrix, columns=columns)
            return trial_dataset
        else:
            print('Trial ', trials[0], ' not in the dataset')

### Generate Subsets

In [ ]:
'''
Divides the original training set into train/val/test sets with a fixed seed for reproducibility.
'''
def generate_sets(dataset, ratio, folds, unique=True, start_time='move_onset_time', end_time='reach_time', seed=2003):
    
    def generate_subset(dataset, subset_name, start_time='move_onset_time', end_time='reach_time'):
        mask = np.all(dataset.trial_info[['split']] == subset_name, axis=1)
        return dataset.make_trial_data(start_field=start_time, end_field=end_time, ignored_trials=~mask)
    
    # Set the random seed if provided
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    # Data Unpacking
    ref_dataset = generate_subset(standardDS, 'train')
    
    trials = trials_present(dataset.trial_info)
    selectable_trials = trials_present(ref_dataset)
    full_train_trials = selectable_trials
    val_length = int(len(trials_present(ref_dataset)) * 0.1)
    
    # Train and Val set making
    train_sets = []
    val_sets = []
    val_selectable_trials = selectable_trials
    for k in range(folds):
        val_trials = np.sort(np.random.choice(selectable_trials, val_length, replace=False))
        train_trials = np.setdiff1d(full_train_trials, val_trials)
        
        val_mask = np.isin(trials, val_trials)
        train_mask = np.isin(trials, train_trials)
        val_dataset = dataset.make_trial_data(start_field=start_time, end_field=end_time, ignored_trials=(~val_mask))
        train_dataset = dataset.make_trial_data(start_field=start_time, end_field=end_time, ignored_trials=(~train_mask))
        if unique:
            selectable_trials = train_trials
        val_sets.append(val_dataset)
        train_sets.append(train_dataset)
    
    # Test set making
    test_dataset = generate_subset(dataset, 'val')

    return train_sets, val_sets, test_dataset

### Gets the Firing Rate

In [ ]:
'''
Returns the firing rate of a dataset's spikes with a specific window size ------ OTIMIZATION NEEDED
'''
def firing_rate(dataset, window_size, bin_time=5, panda=True):
    size = len(dataset)
    neurons_names = dataset['spikes'].columns
    nr_neurons = len(neurons_names)
    spikes_data = np.asarray(dataset['spikes'], dtype='float64')
    trial_ids = np.asarray(dataset['trial_id'], dtype='int64')
    firing_matrix = []
    trials = np.unique(trial_ids)
    kernel = (np.ones((window_size, 1)) / window_size)
    
    for trial in trials:
        mask = trial_ids == trial
        matrix = spikes_data[mask]
        convolved = (convolve2d(matrix, kernel, mode='same')/5)*1000
        firing_matrix.append(convolved)
        
    firing_matrix = np.concatenate(firing_matrix, axis=0)
    if(panda): 
        #dataset['spikes'] = pd.DataFrame(firing_matrix, columns=np.asarray(neurons_names, dtype='int'))
        dataset['spikes'] = firing_matrix
    else: return firing_matrix, trial_ids

### Gets the Trials Present

In [ ]:
'''
Returns the trials present in one dataset
'''
def trials_present(dataset):
    trial_dataset = np.asarray(dataset['trial_id'], dtype='int64')
    return np.unique(trial_dataset)

### Plots all Trajectories

In [ ]:
'''
Plots all trajectories separate (with trial duration in ms)
'''
def plot_allTrajectoriesS(dataset, original_dataset, nr_cols=4, save=False, save_name='plot.png'):
    #Gets the hand positions (trua and pred)
    cursor_pos = np.asarray(dataset['cursor_pos'], dtype='float64')
    true_pos = np.asarray(dataset['hand_pos'], dtype='float64')
    predicted_x = np.asarray(dataset['pred_X'], dtype='float64')
    predicted_y = np.asarray(dataset['pred_Y'], dtype='float64')
    true_x = true_pos[:,0]
    true_y = true_pos[:,1]
    cursor_x = cursor_pos[:,0]
    cursor_y = cursor_pos[:,1]
    
    #Gets the trials and time
    trial_ids = np.asarray(dataset['trial_id'], dtype='int64')
    mask = np.concatenate(([True], trial_ids[1:] != trial_ids[:-1]))
    split_indices = np.where(mask)[0]
    align_time = np.asarray(dataset['align_time'])
    
    #Gets the target positions per trial
    trials = np.unique(trial_ids)
    target_pos = np.asarray(original_dataset.trial_info[['active_pos_x', 'active_pos_y']], dtype='int64')[trials]
    barrier_pos = np.asarray(original_dataset.trial_info['barrier_pos'])[trials]
    barrier_lengths = np.array([len(inner_array) for inner_array in barrier_pos])
    
    
    #Splits all arrays acording to the trials
    predicted_x = np.split(predicted_x, split_indices)[1:]
    predicted_y = np.split(predicted_y, split_indices)[1:]
    align_time = np.split(align_time, split_indices)[1:]
    true_x = np.split(true_x, split_indices)[1:]
    true_y = np.split(true_y, split_indices)[1:]
    cursor_x = np.split(cursor_x, split_indices)[1:]
    cursor_y = np.split(cursor_y, split_indices)[1:]
    
    nr_trials = len(trials)
    #The ploting starts
    if(nr_trials%nr_cols == 0): nr_rows = nr_trials//nr_cols
    else: nr_rows = nr_trials//nr_cols + 1
    
    fig, axs = plt.subplots(nrows=nr_rows, ncols=nr_cols, figsize=(nr_cols*4, nr_rows*4))
    axs = axs.flatten()
    
    #Plotting
    for i in range(nr_trials):
        ax = axs[i]
        x_coords = true_x[i]
        y_coords = true_y[i]
        x_pred = predicted_x[i]
        y_pred = predicted_y[i]
        x_cursor = cursor_x[i]
        y_cursor = cursor_y[i]
        target_x = target_pos[i,0]
        target_y = target_pos[i,1]
        duration = int(align_time[i][-1])/1000000000
        distance = round(np.sqrt(np.power(x_pred[-1]-x_coords[-1],2) + np.power(y_pred[-1]-y_coords[-1],2)),3)
        barriers = barrier_pos[i]
    
        #Gets the limit
        x_lim = np.max([np.max(x_coords), np.max(x_pred), np.max(x_cursor), np.abs(np.min(x_coords)), np.abs(np.min(x_pred)), 
                        np.abs(np.min(x_cursor))])
        y_lim = np.max([np.max(y_coords), np.max(y_pred), np.max(y_cursor), np.abs(np.min(y_coords)), np.abs(np.min(y_pred)), 
                        np.abs(np.min(y_cursor))])
        lim_value = np.max([x_lim,y_lim])+15
        
        #Plots the barriers
        for j in range(barrier_lengths[i]):
            x, y, half_width, half_height = barrier_pos[i][j]
            left = x - half_width
            bottom = y - half_height
            width = 2 * half_width
            height = 2 * half_height
            ax.add_patch(plt.Rectangle((left, bottom), width, height, facecolor='gray'))
    
        ax.plot(x_cursor, y_cursor, label='Cursor Path')
        ax.plot(x_pred, y_pred, label='Predicted Path')
        ax.plot(x_coords, y_coords, label='True Path')
        ax.scatter(target_x, target_y, label='Target Position', color='red')
        ax.scatter(x_coords[0], y_coords[0], label='Start Point', color='black')
        ax.set_title(f'Trial {trials[i]}\nLasted {duration} seconds\nDistance Final: {distance}')
        ax.set_xlim(-lim_value, lim_value)
        ax.set_ylim(-lim_value, lim_value)
        ax.legend()
        ax.grid(True)
    
    plt.suptitle(f'Trial Trajectories', y=1, fontsize=20)
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    plt.show()

### Plots the Cross Correlation

In [ ]:
'''
Plots the correlation for each trial in a dataset in the same plot.

Parameteres
-----------
dataset - pandas.Dataframe
    The required dataset with at least two columns named and with the specified metrics
trials - array_like
    The list of trials to be plotted (must be present in the dataset)
metric - array_like of strings
    The list with the column name of actual metric and the predicted metric
window_size - integer, optional
    The size of the visualization window in the x axis (-window_size+1, window_size)
correlation - string, optional
    Define the correlation metric applied
    - 'cross' (default)
        Uses the normalized cross correlation
    - 'pearson'
        Uses the pearson correlation
bin_size - float, optional
    Specifies what is the size of each bin in ms
focus_trial - {None, int, string}, optional
    Defines a plotted line that will recieve emphasis
    - ''None'' (default)
        No trial recieves emphasis
    - ''int''
        The specified trial recieves emphasis (needs to be in the trials array)
    - 'max'
        The trial with higher correlation recieves emphasis
    - 'min'
        The trial with lower correlation recieves emphasis
    - 'best'
        The trial with the best correlation recieves emphasis (positive or negative)
    - 'worst'
        The trial with the worst correlation recieves emphasis (closer to 0)
    - 'best_lag'
        The trial with the lowest lag for the highest correlation recieves emphasis (closer to 0)
    - 'worst_lag'
        The trial with the highest lag for the highest correlation recieves emphasis (positive or negative)
    - 'mean'
        A new plot line with the mean and the standard error shaded will recieve emphasis with is maximum correlation 
        for a given lag pointed out
compare_axis - bool, optional
    If true the y axis will have the full correlation scope (-1 - margin, 1 + margin) 
color_template - string, optional
    Defines the plt.colormaps used for the plot lines color (uses the full spectrum)
save - bool, optional
    If true the plot will be saved in the same folder
save_name - string, optional
    If save true will define the file name and type
    
'''
def plot_ccTogether(dataset, trials, metric, window_size=150, correlation='pearson', bin_size=5, focus_trial=None, compare_axis=True, 
                    color_template='Greys', save=False, save_name='all_corrs.png'):
    #Data Extraction
    nr_trials = len(trials)
    color_array = np.round(np.linspace(0, 255, nr_trials))
    bin_window = window_size//bin_size
    dataset_matrix = np.asarray(dataset)
    columns = np.array(dataset.columns)
    pred_index = next(i for i, col in enumerate(columns) if col[0] == metric[1])
    obs_index = next(i for i, col in enumerate(columns) if col[0] == metric[0])
    trial_ids = np.array(dataset['trial_id'])
    trials_possible = trials_present(dataset)
    lags = np.arange(-window_size + 1, window_size, bin_size)
    
    #Check if any trial is not there
    check_mask = np.isin(trials, trials_possible)
    if(not check_mask.all()):
        print('Some trial not in the dataset')
    
    #Initializes the arrays
    corr_matrix = []
    max_matrix = []
    max_correlations = []
    max_correlationsAbs = []
    max_lags = []
    max_lagsAbs = []
    
    #Get the trial_datasets
    trial_datasets = np.asarray([dataset_matrix[trial_ids == trial] for trial in trials], dtype='object')
    sep_preds = [trial[:, pred_index] for trial in trial_datasets]
    sep_obs = [trial[:, obs_index] for trial in trial_datasets]
    
    #Computes the crosss and lags
    for i in range(nr_trials): 
        obs = np.array(sep_obs[i], dtype='float64')
        pred = np.array(sep_preds[i], dtype='float64')
        if(correlation == 'pearson'):
            cross_corr = cross_pearson(obs, pred)
        elif(correlation == 'cross'):
            k = np.sqrt((np.sum(np.power(obs,2)))*(np.sum(np.power(pred,2))))
            cross_corr = np.correlate(obs, pred, mode='full')/k
        cross_corr = cross_corr[len(cross_corr)//2-bin_window:len(cross_corr)//2+bin_window]
    
        #Stores the information
        max_corr, max_lag = maximaze_corrParameters(cross_corr, lags)
        max_correlations += [max_corr]
        max_correlationsAbs += [np.abs(max_corr)]
        max_lags += [max_corr]
        max_lagsAbs += [np.abs(max_lag)]
        corr_matrix += [[cross_corr, lags]]
        max_matrix += [[max_corr, max_lag]]
    corr_matrix = np.asarray(corr_matrix, dtype='float64')
    
    #Defines the focus_trial
    if(focus_trial == 'max'):
        focus_trial = trials[np.argmax(max_correlations)]
    elif(focus_trial == 'min'):
        focus_trial = trials[np.argmin(max_correlations)]
    elif(focus_trial == 'best'):
        focus_trial = trials[np.argmax(max_correlationsAbs)]
    elif(focus_trial == 'worst'):
        focus_trial = trials[np.argmin(max_correlationsAbs)]
    elif(focus_trial == 'best_lag'):
        focus_trial = trials[np.argmin(max_lagsAbs)]
    elif(focus_trial == 'worst_lag'):
        focus_trial = trials[np.argmax(max_lagsAbs)]
    
    #Plot function
    plt.figure(figsize=(15,10))
    if(nr_trials > 30): no_label = True
    else: no_label = False
    
    for i in range(nr_trials):
        cross_corr = corr_matrix[i][0]
        lags = corr_matrix[i][1]
        max_corr = max_matrix[i][0]
        max_lag = max_matrix[i][1]
        if(focus_trial == None):
            if(not no_label): plt.plot(lags, cross_corr, color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), 
                                       label=f'Trial {trials[i]}', alpha=1)
            else: plt.plot(lags, cross_corr, color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), alpha=1)
        elif(focus_trial == trials[i]):
            plt.plot(lags, cross_corr, color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), label=f'Trial {trials[i]}', 
                     alpha=1)
            plt.scatter(max_lag, max_corr, color='red', label=f'Corr: {round(max_corr,2)}\nLag: {max_lag} ms')
        else:
            if(not no_label): plt.plot(lags, cross_corr, color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), 
                                       label=f'Trial {trials[i]}', alpha=0.25)
            else: plt.plot(lags, cross_corr, color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), alpha=0.1)

    #In case the focus trial is the mean, its the last to be applied
    if(focus_trial == 'mean'):
        mean_vector = np.mean(corr_matrix, axis=0)
        ste_vector = np.std(corr_matrix, axis=0)
        ste_values = ste_vector[0] / np.sqrt(nr_trials)
        cross_corr = mean_vector[0]
        lags = mean_vector[1]
        max_corr, max_lag = maximaze_corrParameters(cross_corr, lags)
        plt.plot(lags, cross_corr, color='black', label='Mean Correlation', alpha=1)
        plt.fill_between(lags, cross_corr-ste_values, cross_corr+ste_values, color='black', label='Standard Error', alpha=0.25)
        plt.scatter(max_lag, max_corr, color='red', label=f'Corr: {round(max_corr,2)}\nLag: {max_lag} ms')
        plt.scatter(0, max_corr, color='red', label=f'Corr: {round(max_corr,2)}\nLag: {max_lag} ms')
    #Labeling and Formating
    plt.xlabel('Lag (ms)')
    if(correlation == 'cross'):
        plt.ylabel('Cross-Correlation')
        plt.title(f'Cross-Correlation of All Trials ({metric[0]})')
    if(correlation == 'pearson'):
        plt.ylabel('Pearson\'s Correlation')
        plt.title(f'Pearson\'s Correlation of All Trials ({metric[0]})')
    if(compare_axis): plt.ylim(-1.1,1.1)
    plt.grid(True)
    if(no_label): plt.legend(loc='lower left')
    if(save): plt.savefig(save_name)
    plt.show()
    
    return corr_matrix

### Gets the Cross Pearson Correlation

In [ ]:
'''
Returns the cross-correlation with pearson correlation values
'''
def cross_pearson(array1, array2):
    array1 = np.asarray(array1, dtype='float64')
    array2 = np.asarray(array2, dtype='float64')
    
    corr_p = np.zeros(len(array2) + len(array1) - 1)
    size = len(array2) - 1
    #left pad
    for index, i in enumerate(range(size, 0, -1)):
        x = np.pad(array2,(i), mode='constant')
        x = x[i*2:]
        corr_p[index] = np.corrcoef(array1,x)[0,1]
    #righ pad
    for index, i in enumerate(range(0, size+1)):
        index = index+size
        x = np.pad(array2,(i), mode='constant')
        x = x[:len(x)-i*2]
        if(i == 0):
            if(np.all(x == x[0])):
                noise = np.random.uniform(low=-0.0001, high=0.0001, size=x.shape)
                x = x + noise
        corr_p[index] = np.corrcoef(array1,x)[0,1]
    return corr_p

### Maximizes Correlation Parameters

In [ ]:
def maximaze_corrParameters(cross_corr, lags):
    max_corr_index = np.argmax(cross_corr)
    min_corr_index = np.argmin(cross_corr)
    max_corr_lag = lags[max_corr_index]
    min_corr_lag = lags[min_corr_index]
    max_corr = round(cross_corr[max_corr_index],2)
    min_corr = round(cross_corr[min_corr_index],2)
    if(abs(max_corr)<abs(min_corr)): 
        max_corr = min_corr
        max_corr_lag = min_corr_lag
    return max_corr, max_corr_lag

### Gets the Mean Pearson Correlation of a Dataset

In [ ]:
'''
Computes the mean pearson correlation of all trials (and standard deviation) at 0
'''
def get_pearson(dataset, metric, extra_metrics=False):
    #Data Extraction
    trial_ids = np.array(dataset['trial_id'])
    trials = np.unique(trial_ids)
    nr_trials = len(trials)
    color_array = np.round(np.linspace(0, 255, nr_trials))
    
    dataset_matrix = np.asarray(dataset)
    columns = np.array(dataset.columns)
    pred_index = next(i for i, col in enumerate(columns) if col[0] == metric[1])
    obs_index = next(i for i, col in enumerate(columns) if col[0] == metric[0])
    
    #Get the trial_datasets
    trial_datasets = np.asarray([dataset_matrix[trial_ids == trial] for trial in trials], dtype='object')
    sep_preds = [trial[:, pred_index] for trial in trial_datasets]
    sep_obs = [trial[:, obs_index] for trial in trial_datasets]
    
    #Initializes the arrays
    pearson_matrix = []
    
    #Computes the crosss and lags
    for i in range(nr_trials): 
        obs = np.array(sep_obs[i], dtype='float64')
        pred = np.array(sep_preds[i], dtype='float64')
        if(np.all(pred == pred[0])):
            noise = np.random.uniform(low=-0.0001, high=0.0001, size=pred.shape)
            pred = pred + noise
            print(f'-> Noise added to trial {trials[i]}')
        pearson = np.corrcoef(obs, pred)[0,1]
        if(np.isnan(pearson).any()): 
            print(f'Trial {trials[i]} as NaN Pearson')
            print(f'Observed has Nan? {np.isnan(obs).any()}')
            print(obs)
            print(f'Predicted has Nan? {np.isnan(pred).any()}')
            print(pred)
        pearson_matrix.append(pearson)
    pearson_matrix = np.asarray(pearson_matrix, dtype='float64')
    
    #Gets the mean and the standard deviation
    r2 = np.power(pearson_matrix, 2)
    mean_pearson = np.mean(pearson_matrix)
    std_pearson = np.std(pearson_matrix)
    mean_r2 = np.mean(r2)
    std_r2 = np.std(r2)

    if(extra_metrics): return mean_pearson, std_pearson, mean_r2, std_r2, pearson_matrix
    else: return mean_pearson, std_pearson, mean_r2, std_r2

### Removes Neurons for Population Vector

In [ ]:
'''
Removes the neurons from the decoder and prepares the val_dataset for predictions
'''
def remove_neurons(predicted_matrix, observed_matrix, dataset, neurons, threshold, metric='depth', remove_fromDataset=True):
    #Gets the values from the metric used (depth)
    if(metric == 'depth'):
        max_vector = np.max(predicted_matrix, axis=1)
        min_vector = np.min(predicted_matrix, axis=1)
        depth_vector = max_vector-min_vector
        acepted_neurons = np.where(depth_vector>=threshold, neurons, None)
        rejected_neurons = np.where(depth_vector<threshold, neurons, None)
    elif(metric == 'difference'):
        difference_vector = np.sqrt(np.sum(np.power(predicted_matrix - observed_matrix.T, 2), axis=1))
        acepted_neurons = np.where(difference_vector<=threshold, neurons, None)
        rejected_neurons = np.where(difference_vector>threshold, neurons, None)
    
    #Builds arrays with the rejected and acepted neurons
    acepted_neurons = np.asarray(np.ma.masked_equal(acepted_neurons, None).compressed(), dtype='int64')
    
    #Removes from the decoder
    new_decoder = decoder[rejected_neurons == None]
    
    #Removes those columns from the dataset
    rejected_neurons = np.asarray(np.ma.masked_equal(rejected_neurons, None).compressed(), dtype='int64')
    if(remove_fromDataset):
        col_toRemove = np.empty((len(rejected_neurons)), dtype=tuple)
        for i, neuron in enumerate(rejected_neurons):
            col_toRemove[i] = ('spikes', neuron)
        dataset.drop(columns=col_toRemove, inplace=True)
    return new_decoder, acepted_neurons

### Remove Neurons for Naïve Bayes

In [ ]:
'''
Removes the neurons according to a certain threshold
'''
def nb_remove_neurons(datasets, neurons, predicted, observed, grids, method, threshold=None, remove_fromDataset=True):
    #Finding the neurons
    if(method == 'depth'):
        amplitude = predicted[:,0]
        mask = np.where(amplitude >= threshold, True, False)
    elif(method == 'std'):
        x_std = np.array(predicted[:,3])
        y_std = np.array(predicted[:,4])
        mean_std = (x_std + y_std)/2
        mask = np.where(mean_std <= threshold, True, False)
    elif(method == 'diff'):
        # Convert grids to the appropriate format
        grids = np.array(grids)
        x_grid = grids[0]
        y_grid = grids[1]
        
        # Initialize an array to store differences
        difference_matrix = np.zeros((len(observed), x_grid.shape[0], y_grid.shape[1]))
        mask = ~np.isnan(observed[0])
        
        # Loop over all observed instances
        for i in range(len(observed)):
            # Calculate the predicted Gaussian surface for the entire grid
            pred = ex_twoD_Gaussian2((x_grid, y_grid), *predicted[i])
        
            # Calculate the difference where observed is not NaN
            difference_matrix[i][mask] = pred[mask] - observed[i][mask]
        
        # Flatten the differences for each instance
        difference_matrix = difference_matrix.reshape(len(observed), -1)
        distance_matrix = np.sqrt(np.sum(np.power(difference_matrix, 2), axis=1))

        mask = np.where(distance_matrix <= threshold, True, False)

    #Removing the neurons
    neurons = np.array(neurons)
    removed_neurons = neurons[~mask]
    acepted_neurons = neurons[mask]
    acepted_predicted = predicted[mask]
    
    if(remove_fromDataset):
        for j, dataset in enumerate(datasets):
            col_toRemove = np.empty((len(removed_neurons)), dtype=tuple)
            for i, neuron in enumerate(removed_neurons):
                col_toRemove[i] = ('spikes', neuron)
            dataset.drop(columns=col_toRemove, inplace=True)
    return acepted_neurons, acepted_predicted

### Get the distance of Tuning Surfaces

In [ ]:
# Convert grids to the appropriate format
def get_tune_distance(grids, observed, predicted):
    grids = np.array(grids)
    x_grid = grids[0]
    y_grid = grids[1]
    
    # Initialize an array to store differences
    difference_matrix = np.zeros((len(observed), x_grid.shape[0], y_grid.shape[1]))
    mask = ~np.isnan(observed[0])
    
    # Loop over all observed instances
    for i in range(len(observed)):
        # Calculate the predicted Gaussian surface for the entire grid
        pred = ex_twoD_Gaussian2((x_grid, y_grid), *predicted[i])
    
        # Calculate the difference where observed is not NaN
        difference_matrix[i][mask] = pred[mask] - observed[i][mask]
    
    # Flatten the differences for each instance
    difference_matrix = difference_matrix.reshape(len(observed), -1)
    distance_matrix = np.sqrt(np.sum(np.power(difference_matrix, 2), axis=1))

    return distance_matrix

### Compares Models

In [ ]:
'''
Visualizes the different models pearson preformance
'''
def compare_models(parameters, all_entries, model_names, eval='Pearson', title_name='title', label_name='label', 
                   save=False, save_name='plot.png'):
    # Data extraction
    nr_trials = len(all_entries[0, 0])
    if(eval == 'Pearson'):
        x_std = parameters[:, 0, 1]
        y_std = parameters[:, 1, 1]
        x_mean = parameters[:, 0, 0]
        y_mean = parameters[:, 1, 0]
        label_x = 'Mean Pearson for Mx'
        label_y = 'Mean Pearson for My'
    elif(eval == 'R2'):
        x_std = parameters[:, 0, 3]
        y_std = parameters[:, 1, 3]
        x_mean = parameters[:, 0, 2]
        y_mean = parameters[:, 1, 2]
        label_x = 'Mean R2 for Mx'
        label_y = 'Mean R2 for My'
    
    results = {
        label_x: x_mean,
        label_y: y_mean
    }
    cov_results = {
        'Coefficient of Variation for X': x_std / x_mean,
        'Coefficient of Variation for Y': y_std / y_mean,
        'Mean Coefficient of Variation': np.mean([x_std / x_mean, y_std / y_mean], axis=0)
    }
    
    # Extra Variables
    max_mean_x = np.max(x_mean)
    max_mean_y = np.max(y_mean)
    max_x_model = np.argmax(x_mean)
    max_y_model = np.argmax(y_mean)
    deviation = np.array([x_std, y_std])
    error = deviation / np.sqrt(nr_trials)
    
    #Begin Plotting
    fig, ax1 = plt.subplots(figsize=(15, 5), layout='constrained')
    ax2 = ax1.twinx()
    model_x = np.arange(len(model_names))  # the label locations
    width = 0.25  # the width of the bars
    colors = ['tomato', 'dodgerblue', 'tomato', 'dodgerblue', 'black']
    
    # Plot CoV on the right y-axis
    for i, (attribute, measurement) in enumerate(cov_results.items()):
        offset = width * (i + 1)
        alpha_value = 0.15 if i != 2 else 0.3
        ax2.plot(model_x + offset, measurement, color=colors[i + 2], alpha=alpha_value, label=attribute)
    
    # Plot Pearson values on the left y-axis
    for i, (attribute, measurement) in enumerate(results.items()):
        offset = width * (i + 1)
        ax1.errorbar(model_x + offset, measurement, error[i], linewidth=width * 5, ls='none', capsize=5, label=attribute, color=colors[i])
    
    # Highlight preferred model
    mean_cov = np.mean([x_std / x_mean, y_std / y_mean], axis=0)
    min_cov_index = np.argmin(mean_cov)
    t1_x = model_x[min_cov_index]
    t2_x = t1_x + offset + width
    ax1.vlines(t1_x, 0, 1, color='grey', linestyle='--', linewidth=0.5)
    ax1.vlines(t2_x, 0, 1, color='grey', linestyle='--', linewidth=0.5)
    ax1.fill_betweenx([0, 1], t1_x, t2_x, color='yellow', alpha=0.3, 
                      label=f'Best Parameter: {model_names[min_cov_index]} (x: {round(x_mean[min_cov_index],2)} y: {round(y_mean[min_cov_index],2)})')
    
    # Add labels, title, and custom x-axis tick labels for the left y-axis
    ax1.set_ylabel(eval)
    ax1.set_xlabel(label_name)
    ax1.set_title(title_name)
    ax1.set_xticks(model_x + width)
    ax1.set_xticklabels(np.array(model_names, dtype=int), fontsize=8)
    ax1.set_ylim(0, 1)
    ax1.legend(loc='upper left', ncols=1)
    ax1.grid(True)
    
    # Add label for the right y-axis
    ax2.set_ylabel('Coefficient of Variation')
    ax2.legend(loc='upper right', ncols=1)
    
    # Save figure if required
    if save: plt.savefig(save_name)
    
    # Show plot
    plt.show()

# Programming Journey (Danger Zone)

## 1º Meeting - Play with the Dataset

In [ ]:
#Linear regression with given loss function and given regularization
def linear_regression(dataset, loss='sse', lambaRidge=2):
    # Data preparation
    trial_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-130, 370))
    lagged_trial_data = dataset.make_trial_data(align_field='move_onset_time', align_range=(-50, 450))
    rates = trial_data.spikes.to_numpy()
    vel = lagged_trial_data.hand_vel.to_numpy()
        
    #Loss Function Selection
    if(loss == 'sse'):
        rates = np.insert(rates, 0, 1, axis=1)
        penrose = np.linalg.pinv(rates.astype(np.float32))
        w = np.matmul(penrose,vel)
        predicted = np.matmul(rates,w)
    elif(loss == 'ridge'):
        rates = np.insert(rates, 0, 1, axis=1)
        ratesT = np.transpose(rates)
        prod = np.matmul(ratesT,rates)
        identity = np.identity(len(prod))
        helper = prod + lambaRidge * identity
        inverse = np.linalg.inv(helper)
        penrose = np.matmul(inverse,ratesT)
        w = np.matmul(penrose,vel)
        predicted = np.matmul(rates,w)

    print(f"Pearson for the X direction: {pearsonr(predicted[:,0], trial_data['hand_vel']['x'])[0]}")
    print(f"Pearson for the Y direction: {pearsonr(predicted[:,1], trial_data['hand_vel']['y'])[0]}")

    predictedDS = copy.deepcopy(dataset)
    pred_vel_df = pd.DataFrame(predicted, index=lagged_trial_data.clock_time, columns=pd.MultiIndex.from_tuples([('pred_vel', 'x'), ('pred_vel', 'y')]))
    predictedDS.data = pd.concat([predictedDS.data, pred_vel_df], axis=1)

    return predictedDS

In [ ]:
testDS = linear_regression(smallDS,'ridge',np.exp(-18))
plot_comparison(testDS, 11)

In [ ]:
testDS = linear_regression(smallDS,'ridge',1)
plot_comparison(testDS, 11)

In [ ]:
testDS = linear_regression(smallDS)
plot_comparison(testDS, 11)

In [ ]:
trial_data = smallDS.make_trial_data(align_field='move_onset_time', align_range=(-130, 370))
lagged_trial_data = smallDS.make_trial_data(align_field='move_onset_time', align_range=(-50, 450))
vel = lagged_trial_data.hand_vel.to_numpy()

rates = trial_data.spikes.to_numpy()
rates = np.insert(rates, 0, 1, axis=1)
ratesT = np.transpose(rates)
prod = np.matmul(ratesT,rates)
identity = np.identity(len(prod))
helper = prod + 2 * identity
inverse = np.linalg.inv(helper)
penrose = np.matmul(inverse,ratesT)
w = np.matmul(penrose,vel)
predicted = np.matmul(rates,w)

len(predicted)

## 2º Meeting - Rate Code and Tuning Curves

### Plot Functions

In [ ]:
#Plots the data from the tunning curve into a graph
def plot_tuning(tuning_data, prefered_angle):
    
    x_coords = [item[0] for item in tuning_data]
    values = [item[1] for item in tuning_data]
    deviation = [item[2] for item in tuning_data]

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot the first subplot - tunning curve with the deviations
    axes[0,0].errorbar(x_coords, values, deviation, fmt='o', linewidth=1, capsize=6)
    axes[0,0].set_xlabel('Angles [-pi,pi]')
    axes[0,0].set_ylabel('Firing Rates')
    axes[0,0].set_title(f'Tuning Data from Neuron {neuron}')
    axes[0,0].set_xlim(-np.pi, np.pi)
    
    #Plot the second subplot - tunning curves alone
    axes[0,1].scatter(x_coords, values)
    axes[0,1].set_xlabel('Angles [-pi,pi]')
    axes[0,1].set_ylabel('Firing Rates')
    axes[0,1].set_title(f'Tuning Data from Neuron {neuron}')
    axes[0,1].set_xlim(-np.pi, np.pi)
    
    # Plot the third subplot - preferred angle
    axes[1,0].quiver(0, 0, 1 * np.cos(prefered_angle), 1 * np.sin(prefered_angle), angles='xy', scale_units='xy', scale=1, color='b', 
                     width=0.005)
    axes[1,0].set_xlim(-1, 1)
    axes[1,0].set_ylim(-1, 1)
    axes[1,0].set_title(f'Preferred Direction of {round(prefered_angle, 3)}')

    plt.tight_layout()
    plt.show()

In [ ]:
# Plots all the vectors that a bunch of neurons code for
def plot_vectorMap(vectors):
    for pair in vectors:
        plt.quiver(0, 0, 1 * np.cos(pair[1]), 1 * np.sin(pair[1]), angles='xy', scale_units='xy', scale=1, color='b', label=pair[0], 
                   width=0.003)
    plt.xlim(-1, 1) 
    plt.ylim(-1, 1)
    plt.title('All Direction (Mapping)') 
    plt.show()  

In [ ]:
#Plots the actual angle vs the predicted one in the context of covariate 1
def plot_decoding(trials_toCheck, results):
    nr_trials = len(trials_toCheck)
    nr_rows = nr_trials//3
    if(nr_trials%3 == 0): extra_row = 0
    else: extra_row = 1
    fig, axes = plt.subplots(nr_rows+extra_row, 3, figsize=(12, (nr_rows+extra_row)*3))
    for i in range(nr_rows+extra_row):
        if((extra_row == 1 and i < nr_rows+extra_row-1) or extra_row == 0):
            for j in range(3):
                predicted_angle = results[i*3+j][1]
                actual_angle = results[i*3+j][0]
                axes[i,j].quiver(0, 0, 1 * np.cos(predicted_angle), 1 * np.sin(predicted_angle), angles='xy', scale_units='xy', scale=1, 
                                 color='r', width=0.005, label='Predicted Angle')
                axes[i,j].quiver(0, 0, 1 * np.cos(actual_angle), 1 * np.sin(actual_angle), angles='xy', scale_units='xy', scale=1, 
                                 color='b', width=0.005, label='Actual Angle')
                axes[i,j].set_xlim(-1, 1)
                axes[i,j].set_ylim(-1, 1)
                axes[i,j].legend()
                axes[i,j].set_title(f'Diference of {round(np.abs(actual_angle-predicted_angle), 3)} rads')
        elif(extra_row == 1 and i == nr_rows+extra_row-1):
            for j in range(nr_trials%3):
                predicted_angle = results[i*3+j][1]
                actual_angle = results[i*3+j][0]
                axes[i,j].quiver(0, 0, 1 * np.cos(predicted_angle), 1 * np.sin(predicted_angle), angles='xy', scale_units='xy', scale=1, 
                                 color='r', width=0.005, label='Predicted Angle')
                axes[i,j].quiver(0, 0, 1 * np.cos(actual_angle), 1 * np.sin(actual_angle), angles='xy', scale_units='xy', scale=1, 
                                 color='b', width=0.005, label='Actual Angle')
                axes[i,j].set_xlim(-1, 1)
                axes[i,j].set_ylim(-1, 1)
                axes[i,j].legend()
                axes[i,j].set_title(f'Diference of {round(np.abs(actual_angle-predicted_angle), 3)} rads')
    plt.tight_layout()
    plt.show()

### Helper Functions

In [ ]:
#Gets the trial_info subset data without/with trial exclusions
def trial_infoExtractor (dataset, ignored_mask, settings):
    if(len(ignored_mask) == 0): 
        target_data = np.asarray(dataset.trial_info[['target_pos','active_target']])
        duration_data = np.asarray(dataset.trial_info[['end_time','move_onset_time']], dtype='float64')/1000000 #In ms
    else: 
        target_data = np.asarray(dataset.trial_info[['target_pos','active_target']][ignored_mask])
        duration_data = np.asarray(dataset.trial_info[['end_time','move_onset_time']][ignored_mask], dtype='float64')/1000000 #In ms
    
    target_position = []             #Saves the target position per trial
    target_angle = []                #Saves the target angle with origin per trial
    trial_duration = []              #Saves the duration per trial in ms
    for i in range(len(target_data)):
        position = target_data[i][0][target_data[i][1]]
        target_position +=  [position]
        if(settings == 'simple'): target_angle += [np.arctan2(position[1],position[0])]
        elif(settings == 'double_cos'): target_angle += [[np.cos(np.arctan2(position[1],position[0])), 
                                                          np.cos(np.arctan2(position[0],position[1]))]]
        trial_duration += [duration_data[i][0]-duration_data[i][1]]
    target_position = np.asarray(target_position) 
    #print(target_angle)

    return target_angle, trial_duration

In [ ]:
#Returns all diferent angles present in the set
def all_angles(sorted_tunning):
    size = len(sorted_tunning)
    checked_angles = []
    for i in range(size):
        if(sorted_tunning[i][0] not in checked_angles):
            checked_angles += [sorted_tunning[i][0]]
    return checked_angles

In [ ]:
#Returns the mean firing rate from each angle
def mean_firingRate(different_angles, sorted_tunning):
    final_tune = []
    size = len(different_angles)
    for j in range(size):
        store = []
        for i in range(len(sorted_tunning)):
            if(sorted_tunning[i][0] == different_angles[j]):
                store += [sorted_tunning[i][1]]
        final_tune += [[different_angles[j], np.mean(store), np.std(store)]]
    return final_tune

In [ ]:
#Gets the tunning curve and returns the preferred direction
def prefered_direction(tunning_data):
    size = len(tunning_data)
    prefered_angle = None
    biggest_rate = 0
    for i in range(size):
        if(tunning_data[i][1] > biggest_rate): 
            prefered_angle = tunning_data[i][0]
            biggest_rate = tunning_data[i][1]
    return prefered_angle

In [ ]:
#Linear regression with given loss function and given regularization that returns the weights
def linear_regression_model(trainning_var, trainning_out, loss='sse', lambaRidge=2):
    #Data preparation
    rates = trainning_var
    vel = trainning_out
    
    #Loss Function Selection
    if(loss == 'sse'):
        rates = np.insert(rates, 0, 1, axis=1)
        penrose = np.linalg.pinv(rates.astype(np.float32))
        w = np.matmul(penrose,vel)
        predicted = np.matmul(rates,w)
    elif(loss == 'ridge'):
        rates = np.insert(rates, 0, 1, axis=1)
        ratesT = np.transpose(rates)
        prod = np.matmul(ratesT,rates)
        identity = np.identity(len(prod))
        helper = prod + lambaRidge * identity
        inverse = np.linalg.inv(helper)
        penrose = np.matmul(inverse,ratesT)
        w = np.matmul(penrose,vel)
        predicted = np.matmul(rates,w)

    return w

In [ ]:
#Obtains the desired spike data (+ trial ids) and the mask used to remove the ignored trials. If wanted can generate for only one trial
def generate_spikeData(dataset, neuron, ignored_ids=None, trial=-1, extra_needed=False):
    ignored_mask = []
    if(trial != -1): 
        ignored_mask = np.all(dataset.trial_info[['trial_id']] != trial, axis=1)
        trial_data = dataset.make_trial_data(start_field='move_onset_time', ignored_trials=ignored_mask)
        if(extra_needed): 
            duration_data = np.asarray(dataset.trial_info[['end_time','move_onset_time']][~ignored_mask], dtype='float64')/1000000
            target_data = np.asarray(dataset.trial_info[['target_pos','active_target']][~ignored_mask])
    elif(ignored_ids == None): 
        trial_data = dataset.make_trial_data(start_field='move_onset_time')
    else:
        for ids in ignored_ids:
            ignored_mask += [np.all(dataset.trial_info[['trial_id']] != ids, axis=1)]
        ignored_mask = reduce(operator.and_, ignored_mask)
        trial_data = dataset.make_trial_data(start_field='move_onset_time', ignored_trials=~ignored_mask)

    neuron_preData = trial_data[['spikes','trial_id']]
    neuron_data = neuron_preData[['trial_id']].copy()
    neuron_data[neuron] = neuron_preData['spikes'][neuron].copy()
    neuron_data = np.asarray(neuron_data)

    if(extra_needed):
        position = target_data[0][0][target_data[0][1]]
        target_angle = np.arctan2(position[1],position[0])
        trial_duration = duration_data[0][0]-duration_data[0][1]
        return neuron_data, ignored_mask, trial_duration, target_angle
    return neuron_data, ignored_mask

In [ ]:
#Returns the firing rate of a given neuron in a given trial
def neuron_firingRate(neuron_data, movement_duration, trial, debugger=False):
    firing_time = movement_duration / 2
    firing_bins = firing_time // 5
    count = 0
    bins_added = 0
    j = 0
    while j < len(neuron_data) and bins_added < firing_bins:
        if(neuron_data[j][0] == trial): 
            count += neuron_data[j][1]
            bins_added += 1
        j += 1
    firing_rate = count/firing_time
    
    if(debugger):
        print('Counted: ', count)
        print('Firing Rate: ', firing_rate)
        print('----')
        print('Firing Time: ', firing_time)
        print('Firing Bins: ', firing_bins)
        print('')

    return firing_rate

In [ ]:
#Converts the cosines values to the angle
def cos2angle(cosines):
    if(cosines[1] >= 0): angle = np.arccos(cosines[0])
    else: angle = -np.arccos(cosines[0])
        
    return angle

In [ ]:
#Converts an angle to its cosines
def angle2cos(angle):
    return np.cos(angle), np.sin(angle)

### Main Functions

In [ ]:
#Allows for better visualization of the spikes of a specific trial
def raster_plot (dataset, neurons=None, trial=0, align=None):
    mask = np.all(dataset.trial_info[['trial_id']] == trial, axis=1)

    if(align == None): 
        trial_data = dataset.make_trial_data(align_field='move_onset_time', ignored_trials=(~mask))
    else:
        trial_data = dataset.make_trial_data(align_field='move_onset_time', align_range=align, ignored_trials=(~mask))

    
    if (neurons == None):
        neurons = trial_data['spikes'].columns.tolist() #Gets the name of all neurons
    neuron_spikes = []
    for neuron in neurons:
        neuron_spikes = neuron_spikes + [trial_data['spikes'][neuron]]

    #Initialize a list to store spike times for each neuron
    spike_times = [[] for _ in range(len(neuron_spikes))]
    
    #Iterate through the spike data and extract spike times for each neuron
    for neuron_idx, neuron in enumerate(neuron_spikes):
        for time_idx, spike in enumerate(neuron):
            if spike == 1:
                spike_times[neuron_idx].append(time_idx)  # Assuming time_idx represents time in ms
    
    fig, ax = plt.subplots()
    for i, spikes in enumerate(spike_times, start=1):
        ax.eventplot(spikes, colors='k', lineoffsets=i, linelengths=0.5)
    ax.set_xlabel('Time (bins)')
    #if(align != None): ax.set_xlim(align[0]//5, align[1]//5)
    ax.set_ylabel('Neuron')
    ax.set_title('Raster Plot for Neuron Firing')
    plt.show()

In [ ]:
'''
Recieves a dataset and will generate a tunning curve and, using a explicit rule, define the prefered direction of the 
given neurons. Will also return the b coeficient used in the decoding function
Co-variate: Target location
'''
def tuning_curve(dataset, neuron, ignored_ids=None, settings='maximum', debugger=False, plot=True):

    #Obtains the desired spike data (+ trial ids) and the mask used to remove the ignored trials
    neuron_data, ignored_mask = generate_spikeData(dataset, neuron, ignored_ids)
    
    #Defines the settings according to some templates
    if(settings == 'maximum'): angle_type = 'simple' #Angle is returned as the angle with vector (0,1)
    elif(settings == 'cosines'): angle_type = 'double_cos' #Angle is returned as the two cosines with vector (0,1) and (1,0)

    #Gets the angle that the activated target makes and the duration of movement for each trial 
    target_angle, trial_duration = trial_infoExtractor(dataset,ignored_mask, angle_type)

    tunning_data = []
    for i in range(len(trial_duration)):
        firing_rate = neuron_firingRate(neuron_data, trial_duration[i], i, debugger)
        tunning_data += [[target_angle[i], firing_rate]]

    #Sorts data from angles (necessary if you want a plot with connected lines)
    sorted_tunning = sorted(tunning_data, key=lambda x: x[0])     

    
    #Returns the direction with higher firing rate
    if(settings == 'maximum'): 
        different_angles = all_angles(sorted_tunning)
        results = mean_firingRate(different_angles,sorted_tunning)
        angle = prefered_direction(results)
        if(plot): plot_tuning(results, angle)
        return angle
        
    #Returns the coded direction using the equation d(M) = b + bxmx + bymy
    elif(settings == 'cosines'):
        cosines = [item[0] for item in sorted_tunning]
        freqs = [item[1] for item in sorted_tunning]
        weights = linear_regression_model(cosines, freqs)
        k = np.sqrt(pow(weights[1],2) + pow(weights[2],2))
        vectorC = [weights[1]/k, weights[2]/k]
        angle = cos2angle(vectorC)
        
        if(plot): 
            different_angles = all_angles(sorted_tunning)
            results = mean_firingRate(different_angles,sorted_tunning)
            for i in range(len(results)): results[i][0] = cos2angle(results[i][0])
            plot_tuning(results, angle)

        
        return vectorC, weights[0], angle

In [ ]:
#Returns the prefered directions of all neurons
def direction_mapping(dataset, neurons, ignored_trials=None, config='maximum', plot=True):
    results = []
    plot_data = []
    for neuron in neurons:
        tuning_data = tuning_curve(dataset,neuron, ignored_ids=ignored_trials, settings=config, plot=False)
        results += [[neuron, tuning_data[0], tuning_data[1]]]
        if(plot): plot_data += [[neuron,tuning_data[2]]]
    if(plot): plot_vectorMap(plot_data)
    return results

In [ ]:
#Recieves a frequency and the trainned data and returns the predicted direction
def population_prediction(rate, data):
    nr_neurons = len(data)
    P = 0
    for i in range(nr_neurons):
        P += (rate[i]-data[i][3])*np.array([data[i][1], data[i][2]])
    return P

In [ ]:
#Applies the decoding for multiple trials
def decode_trials(dataset, trials_toCheck, neurons, vector_mapping, plot = False):
    results = []
    for trial_check in trials_toCheck:
        rates = []
        for neuron in neurons:
            test_spikes, mask, duration, actual_angle = generate_spikeData(dataset, neuron, trial=trial_check, extra_needed=True)
            rates += [neuron_firingRate(test_spikes, duration, trial_check)]
        predicted_angle = cos2angle(population_prediction(rates, vector_mapping))
        results += [[actual_angle, predicted_angle]]
    if(plot): plot_decoding(trials_toCheck, results)
    return results

### Crafting Table - Covariate 1

In [ ]:
'''
Predictions made with decoder trained with target covariate and cosines angles (paper)
Time Required: 65 sec
'''

decoder = direction_mapping(smallDS, neurons, config='cosines', plot=False)
decoder #Takes about 35sec

#Creates the validation data
val_dataset, mask = generate_subset(smallDS, 'val')

#Predicts every angle according to a decoder
angle_predictor(val_dataset,decoder, 35, small_neurons, distance_coef=50, pred_pos=True)

#Plots the cross-correlation set
plot_crossCor(val_dataset)

#Visualizes all predictions
repeated_trials = np.asarray(val_dataset['trial_id'])
trials = []
for i in range(len(val_dataset)):
    if(repeated_trials[i] not in trials): trials += [repeated_trials[i]]

plot_allPredictions(val_dataset, trials, nr_cols=4)

In [ ]:
#Visualize the turning curve of a given neuron
neuron = neurons[1]
tuning_curve(smallDS, neuron, ignored_ids=[0,1], settings='cosines', debugger=False, plot=True)

In [ ]:
#Visualize the predicted target direction and compares to the actual target direction
trials_toCheck = [0,1,2,3,4,5,6,7,8]
decode_trials(smallDS, trials_toCheck, neurons, vector_mapping, plot=True)

### Crafting Table - Covariate 2

In [ ]:
#Changes a dataset spike to a firing rate calculated with a specific window size
def firing_rate(dataset, window_size, neurons, bin_time=5):
    spikes_data = dataset['spikes'].copy()
    size = len(spikes_data)
    for neuron in neurons:
        spikes_data[neuron] = np.convolve(spikes_data[neuron], np.ones(window_size)/window_size, mode='same')/bin_time
    dataset['spikes'] = spikes_data

In [ ]:
#Generates the Train/Val subset
def generate_subset(dataset, subset_name, start_time='move_onset_time'):
    mask = np.all(dataset.trial_info[['split']] == subset_name, axis=1)
    return dataset.make_trial_data(start_field=start_time, ignored_trials=~mask), mask

In [ ]:
def angle_decoder(train_dataset, window_size, neurons, delay=0, start_time='move_onset_time', bin_time=5):
    #Change spikes to firing rates in ms (moving average)
    firing_rate(train_dataset, window_size, bin_time=bin_time)
    train_size = len(train_dataset)
    
    #Adds the column vectorM with the cosines components to the train_dataset
    vectorM = train_dataset['hand_pos'].copy()
    vectorM = vectorM.rename(columns={'x': 'mx'}) 
    vectorM = vectorM.rename(columns={'y': 'my'}) 
    for i in range(train_size):
        if (i == 0):
            preX = 0
            preY = 0
        elif(train_dataset.loc[i,'trial_id'][0] != train_dataset.loc[i-1,'trial_id'][0]): #Find ways to optimize this loop gives 10 extra seconds
            preX = 0
            preY = 0
        else:
            preX = vectorM.loc[i - 1, 'mx']
            preY = vectorM.loc[i - 1, 'my']
        x = vectorM.loc[i, 'mx']
        y = vectorM.loc[i, 'my']
        angle = np.arctan2((y - preY), (x - preX))
        cosX, cosY = angle2cos(angle)  
        vectorM.loc[i, 'mx'] = cosX
        vectorM.loc[i, 'my'] = cosY
        vectorM.loc[i, 'angle'] = angle
    train_dataset['mx'] = vectorM['mx']
    train_dataset['my'] = vectorM['my']
    train_dataset['angle'] = vectorM['angle']
    
    #Calculate the coeficients for every neuron and generates the decoder
    ms_array = np.array([np.array([x, y]) for x, y in zip(vectorM['mx'], vectorM['my'])])
    
    #Adds the delay (if there is any)
    delayM_array = []
    for i in range(train_size):
        if(i+delay > train_size-1): delayM_array += [[0,0]]
        else: delayM_array += [ms_array[i+delay]]
    ms_array = delayM_array
    
    decoder = []
    for neuron in neurons:
        rates = train_dataset['spikes'][neuron].copy()
        weights = linear_regression_model(ms_array, np.array(rates), loss='ridge')
        k = np.sqrt(weights[1]**2 + weights[2]**2)
        cx = weights[1]/k
        cy = weights[2]/k
        angle = cos2angle([cx,cy])
        magnitude = np.sqrt(np.square(cx) + np.square(cy))
        decoder += [[neuron, [cx, cy], weights[0], magnitude, angle]]
    return decoder

In [ ]:
#Adds to the dataset predicted angle values
def angle_predictor(val_dataset, decoder, window_size, neurons, removed_neurons=None ,distance_coef=7, bin_time=5, pred_pos=False):
    #Prepares the data
    firing_rate(val_dataset, window_size, bin_time=bin_time)
    val_size = len(val_dataset)

    #Predicts the direction
    for i in range(val_size):
        if(removed_neurons != None): 
            pred_angle = population_prediction(np.asarray(val_dataset['spikes'].drop(removed_neurons, axis=1).loc[i]),decoder) #HEREEEEE
        else:
            pred_angle = population_prediction(np.asarray(val_dataset.loc[i, 'spikes']),decoder)
        val_dataset.loc[i, 'pred_mag'] = np.sqrt(np.square(pred_angle[0]) + np.square(pred_angle[1]))
        val_dataset.loc[i, 'pred_cosX'] = pred_angle[0]
        val_dataset.loc[i, 'pred_cosY'] = pred_angle[1]
        val_dataset.loc[i, 'pred_angle'] = cos2angle(pred_angle)
    
    #Predicts the position
        if(pred_pos):
            if(i == 0): 
                val_dataset.loc[i, 'pred_X'] = val_dataset.loc[i, 'hand_pos']['x']
                val_dataset.loc[i, 'pred_Y'] = val_dataset.loc[i, 'hand_pos']['y']
            elif(val_dataset.loc[i,'trial_id'][0] != val_dataset.loc[i-1,'trial_id'][0]): 
                val_dataset.loc[i, 'pred_X'] = val_dataset.loc[i, 'hand_pos']['x']
                val_dataset.loc[i, 'pred_Y'] = val_dataset.loc[i, 'hand_pos']['y']
            else:
                val_dataset.loc[i, 'pred_X'] = val_dataset.loc[i-1, 'pred_X'][0] + distance_coef * val_dataset.loc[i-1, 'pred_mag'][0] * val_dataset.loc[i-1, 'pred_cosX'][0]
                val_dataset.loc[i, 'pred_Y'] = val_dataset.loc[i-1, 'pred_Y'][0] + distance_coef * val_dataset.loc[i-1, 'pred_mag'][0] * val_dataset.loc[i-1, 'pred_cosY'][0]
    
    #Builds the vectorM for comparition
    vectorM = val_dataset['hand_pos'].copy()
    vectorM = vectorM.rename(columns={'x': 'mx'}) 
    vectorM = vectorM.rename(columns={'y': 'my'}) 
    for i in range(val_size):
        if (i == 0):
            preX = 0
            preY = 0
        elif(val_dataset.loc[i,'trial_id'][0] != val_dataset.loc[i-1,'trial_id'][0]): #Find ways to optimize this loop gives 10 extra seconds
            preX = 0
            preY = 0
        else:
            preX = vectorM.loc[i - 1, 'mx']
            preY = vectorM.loc[i - 1, 'my']
        x = vectorM.loc[i, 'mx']
        y = vectorM.loc[i, 'my']
        angle = np.arctan2((y - preY), (x - preX))
        cosX, cosY = angle2cos(angle)  
        vectorM.loc[i, 'mx'] = cosX
        vectorM.loc[i, 'my'] = cosY
        vectorM.loc[i, 'angle'] = angle
    val_dataset['mx'] = vectorM['mx']
    val_dataset['my'] = vectorM['my']
    val_dataset['angle'] = vectorM['angle']

In [ ]:
#Plots all trajectory comparitions between predicted trajectory and actual trajectory
def plot_allPredictions(dataset, trials, nr_cols=3):
    nr_trials = len(trials)
    #Prepares subplot organization
    nr_rows = nr_trials//nr_cols
    if(nr_trials%nr_cols == 0): extra_row = 0
    else: extra_row = 1
    fig, axes = plt.subplots(nr_rows+extra_row, nr_cols, figsize=(12, (nr_rows+extra_row)*3))
    print(nr_rows+extra_row)
    #Plots the data
    for i in range(nr_rows+extra_row):
        if((extra_row == 1 and i < nr_rows+extra_row-1) or extra_row == 0):
            for j in range(nr_cols):
                x_coords = []
                y_coords = []
                actual_x = []
                actual_y = []
                desired_trial = trials[i*nr_cols+j]
                for k in range(len(dataset)):
                    if(dataset.loc[k, 'trial_id'][0] == desired_trial):
                        x_coords += [dataset.loc[k, 'pred_X'][0]]
                        y_coords += [dataset.loc[k, 'pred_Y'][0]]
                        actual_x += [dataset.loc[k, 'hand_pos']['x']]
                        actual_y += [dataset.loc[k, 'hand_pos']['y']]
                axes[i,j].plot(x_coords, y_coords, label='Predicted')
                axes[i,j].plot(actual_x, actual_y, label='Actual')
                axes[i,j].set_xlabel('X')
                axes[i,j].set_ylabel('Y')
                #axes[i,j].legend()
                axes[i,j].grid(True)
                axes[i,j].set_title(f"Hand Trajectory for Trial {desired_trial}")
        elif(extra_row == 1 and i == nr_rows+extra_row-1):
            for j in range(nr_trials%nr_cols):
                x_coords = []
                y_coords = []
                actual_x = []
                actual_y = []
                desired_trial = trials[i*nr_cols+j]
                for k in range(len(dataset)):
                    if(dataset.loc[k, 'trial_id'][0] == desired_trial):
                        x_coords += [dataset.loc[k, 'pred_X'][0]]
                        y_coords += [dataset.loc[k, 'pred_Y'][0]]
                        actual_x += [dataset.loc[k, 'hand_pos']['x']]
                        actual_y += [dataset.loc[k, 'hand_pos']['y']]
                axes[i,j].plot(x_coords, y_coords, label='Predicted')
                axes[i,j].plot(actual_x, actual_y, label='Actual')
                axes[i,j].set_xlabel('X')
                axes[i,j].set_ylabel('Y')
                #axes[i,j].legend()
                axes[i,j].grid(True)
                axes[i,j].set_title(f"Hand Trajectory for Trial {desired_trial}")
    plt.tight_layout()
    plt.savefig('all_predictions.png')
    plt.show()

In [ ]:
def trial_crossCorrelation (dataset, trial, time_window=150, bin_size=5, plot=True):
    bin_window = time_window//bin_size
    if(not isinstance(trial, numbers.Integral)):
        print('Please input only one trial as an integer')
        return

    trial_dataset = trial_datasetMaker(dataset, [trial])
    predicted = np.asarray(trial_dataset['pred_angle'])
    actual = np.asarray(trial_dataset['angle'])
    k = np.sqrt((np.sum(actual*actual))*(np.sum(predicted*predicted)))
    cross_corr = np.correlate(actual, predicted, mode='full')/k
    lags = np.arange(-time_window + 1, time_window, bin_size)
    cross_corr = cross_corr[len(actual)-bin_window:len(actual)+bin_window]

    if(plot):
        plt.plot(lags, cross_corr)
        plt.xlabel('Lag (ms)')
        plt.ylabel('Pearsons Coefficient')
        plt.title(f'Cross-correlation of Trial {trial} Angle')
        plt.grid(True)
        plt.show()
        
        # Find the lag at which cross-correlation is maximum
        max_corr_index = np.argmax(cross_corr)
        max_corr_lag = lags[max_corr_index]
        max_corr = cross_corr[max_corr_index]
        print("Maximum cross-correlation at lag: ", max_corr_lag)
        print("Maximum cross-correlation of: ", max_corr)
    else:
        return cross_corr, lags

In [ ]:
def plot_desiredTrial(val_dataset, desired_trial):
    x_coords = []
    y_coords = []
    actual_x = []
    actual_y = []
    for i in range(len(val_dataset)):
        if(val_dataset.loc[i, 'trial_id'][0] == desired_trial):
            x_coords += [val_dataset.loc[i, 'pred_X'][0]]
            y_coords += [val_dataset.loc[i, 'pred_Y'][0]]
            actual_x += [val_dataset.loc[i, 'hand_pos']['x']]
            actual_y += [val_dataset.loc[i, 'hand_pos']['y']]
    colors = np.random.rand(len(y_coords))
    #plt.plot(x_coords, y_coords, label='Predicted')
    plt.scatter(x_coords, y_coords, label='Predicted', c=colors, cmap='viridis')
    plt.plot(actual_x, actual_y, label='Actual')
    plt.title(f"Cursor Trajectory for Trial {desired_trial}")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
'''
Predictions made with decoder trained with hand_pos covariate and cosines angles (paper)
Moving Average Size: 35
Time Required: 39 sec
'''

#Creates the training data
train_dataset, mask = generate_subset(smallDS, 'train')

#Generates a decoder for every neuron
decoder = angle_decoder(train_dataset, 35, small_neurons)

#Creates the validation data
val_dataset, mask = generate_subset(smallDS, 'val')

#Predicts every angle according to a decoder
angle_predictor(val_dataset,decoder, 35, small_neurons, distance_coef=50, pred_pos=True)

#Plots Cross_Correlation
plot_crossCor(val_dataset)

#Plots all trial predictions
repeated_trials = np.asarray(val_dataset['trial_id'])
trials = []
for i in range(len(val_dataset)):
    if(repeated_trials[i] not in trials): trials += [repeated_trials[i]]

plot_allPredictions(val_dataset, trials, nr_cols=4)

In [ ]:
'''
Predictions made with decoder trained with hand_pos covariate and cosines angles (paper)
Moving Average Size: 150
Time Required: 39 sec
'''

#Creates the training data
train_dataset, mask = generate_subset(smallDS, 'train')

#Generates a decoder for every neuron
decoder = angle_decoder(train_dataset, 150, small_neurons)

#Creates the validation data
val_dataset, mask = generate_subset(smallDS, 'val')

#Predicts every angle according to a decoder
angle_predictor(val_dataset,decoder, 150, small_neurons, distance_coef=50, pred_pos=True)

#Plots Cross_Correlation
plot_crossCor(val_dataset)

#Plots all trial predictions
repeated_trials = np.asarray(val_dataset['trial_id'])
trials = []
for i in range(len(val_dataset)):
    if(repeated_trials[i] not in trials): trials += [repeated_trials[i]]

plot_allPredictions(val_dataset, trials, nr_cols=4)

In [ ]:
'''
Predictions made with decoder trained with hand_pos covariate and cosines angles (paper)
Moving Average Size: 10
Time Required: 39 sec
'''

#Creates the training data
train_dataset, mask = generate_subset(smallDS, 'train')

#Generates a decoder for every neuron
decoder = angle_decoder(train_dataset, 10, small_neurons)

#Creates the validation data
val_dataset, mask = generate_subset(smallDS, 'val')

#Predicts every angle according to a decoder
angle_predictor(val_dataset,decoder, 10, small_neurons, distance_coef=50, pred_pos=True)

#Plots Cross_Correlation
plot_crossCor(val_dataset)

#Plots all trial predictions
repeated_trials = np.asarray(val_dataset['trial_id'])
trials = []
for i in range(len(val_dataset)):
    if(repeated_trials[i] not in trials): trials += [repeated_trials[i]]

plot_allPredictions(val_dataset, trials, nr_cols=4)

In [ ]:
'''
Predictions made with decoder trained with hand_pos covariate and tuning curve angles removing the neurons that 
have not the desired depth
Time Required: 37 sec
'''

#Creates the training data
#train_dataset, mask = generate_subset(smallDS, 'train')
#decoder = angle_decoder(train_dataset, 35, small_neurons)

dataset = train_dataset
neurons = small_neurons
nr_directions = 25

spike_data = dataset['spikes'].copy()
angle_data = dataset['angle'].copy()

target_angles = np.linspace(-np.pi, np.pi, nr_directions, endpoint=False)
for i, angle in enumerate(angle_data):
    closest_angle_idx = np.argmin(np.abs(target_angles - angle))
    angle_data[i] = target_angles[closest_angle_idx]

for neuron in neurons:
    neuron_data = spike_data[neurons]

mean_matrix = []
for j in range(len(target_angles)):
    store_matrix = []
    for i in range(len(spike_data)):
        if(angle_data[i] == target_angles[j]):
            store_matrix += [spike_data.loc[i]]
    mean_matrix += [[target_angles[j], np.mean(store_matrix, axis=0)]]

#Visualize
decoder = []
rejected_neurons = []
desired_neuron = 1011
rates_perAngle = np.zeros_like(target_angles)
for j, neuron in enumerate (neurons):
    for i in range(len(rates_perAngle)):
        rates_perAngle[i] = mean_matrix[i][1][j]

    # Get the maximum and minimum values
    max_value = max(rates_perAngle)
    min_value = min(rates_perAngle)
    depth = (max_value - min_value)/max_value
    max_index = np.argmax(rates_perAngle)
    prefered_angle = target_angles[max_index]
    
    if(neuron == desired_neuron):
        plt.plot(target_angles, rates_perAngle, marker='o', linestyle='-')
        plt.xlabel('Target Angles')
        plt.ylabel('Firing Rate')
        plt.title(f'Tuning Curve of Neuron {desired_neuron}')
        plt.grid(True)
        plt.show()
        print("Maximum value:", max_value)
        print("Minimum value:", min_value)
        print("Depth:", depth)
        
    if(depth < 0.7):
        rejected_neurons += [neuron]
    else:
        decoder += [[neuron, [np.cos(prefered_angle), np.sin(prefered_angle)], 0, prefered_angle]]


#Creates the validation data
val_dataset, mask = generate_subset(smallDS, 'val')

#Predicts every angle according to a decoder
angle_predictor(val_dataset,decoder, 35, small_neurons, removed_neurons=rejected_neurons, distance_coef=50, pred_pos=True)

#Plots Cross_Correlation
plot_crossCor(val_dataset)

#Plots all trial predictions
repeated_trials = np.asarray(val_dataset['trial_id'])
trials = []
for i in range(len(val_dataset)):
    if(repeated_trials[i] not in trials): trials += [repeated_trials[i]]

plot_allPredictions(val_dataset, trials, nr_cols=4)

In [ ]:
'''
Predictions made with decoder trained with hand_pos covariate and cosine angles removing the neurons that 
have not the desired depth
Time Required: 45 sec
'''

#Creates the training data
train_dataset, mask = generate_subset(smallDS, 'train')

#Generates a decoder for every neuron
decoder = angle_decoder(train_dataset, 35, small_neurons)

dataset = train_dataset
neurons = small_neurons
nr_directions = 25

spike_data = dataset['spikes'].copy()
angle_data = dataset['angle'].copy()

target_angles = np.linspace(-np.pi, np.pi, nr_directions, endpoint=False)
for i, angle in enumerate(angle_data):
    closest_angle_idx = np.argmin(np.abs(target_angles - angle))
    angle_data[i] = target_angles[closest_angle_idx]

for neuron in neurons:
    neuron_data = spike_data[neurons]

mean_matrix = []
for j in range(len(target_angles)):
    store_matrix = []
    for i in range(len(spike_data)):
        if(angle_data[i] == target_angles[j]):
            store_matrix += [spike_data.loc[i]]
    mean_matrix += [[target_angles[j], np.mean(store_matrix, axis=0)]]

#Visualize
rejected_neurons = []
rejected_index = []
desired_neuron = 1011
rates_perAngle = np.zeros_like(target_angles)
for j, neuron in enumerate (neurons):
    for i in range(len(rates_perAngle)):
        rates_perAngle[i] = mean_matrix[i][1][j]

    # Get the maximum and minimum values
    max_value = max(rates_perAngle)
    min_value = min(rates_perAngle)
    depth = (max_value - min_value)/max_value
    max_index = np.argmax(rates_perAngle)
    prefered_angle = target_angles[max_index]
    
    if(neuron == desired_neuron):
        plt.plot(target_angles, rates_perAngle, marker='o', linestyle='-')
        plt.xlabel('Target Angles')
        plt.ylabel('Firing Rate')
        plt.title(f'Tuning Curve of Neuron {desired_neuron}')
        plt.grid(True)
        plt.show()
        print("Maximum value:", max_value)
        print("Minimum value:", min_value)
        print("Depth:", depth)
        
    if(depth < 0.7):
        rejected_neurons += [neuron]
        rejected_index += [j]

#Removes those neurons from the decoder
old_decoder = decoder
decoder = []
for i in range(len(old_decoder)):
    if(old_decoder[i][0] not in rejected_neurons): decoder += [old_decoder[i]]


#Creates the validation data
val_dataset, mask = generate_subset(smallDS, 'val')

#Predicts every angle according to a decoder
angle_predictor(val_dataset,decoder, 35, small_neurons, removed_neurons=rejected_neurons, distance_coef=50, pred_pos=True)

#Plots Cross_Correlation
plot_crossCor(val_dataset)

#Plots all trial predictions
repeated_trials = np.asarray(val_dataset['trial_id'])
trials = []
for i in range(len(val_dataset)):
    if(repeated_trials[i] not in trials): trials += [repeated_trials[i]]

plot_allPredictions(val_dataset, trials, nr_cols=4)

In [ ]:
a,b = trial_crossCorrelation(val_dataset, 0, time_window=400, plot=False)
a

In [ ]:
#Returns the dataset for only one trial
def trial_datasetMaker(dataset, trials):
    trials_possible = trials_present(dataset)
    check_mask = np.isin(trials, trials_possible)

    if(check_mask.all()):
        size = len(dataset)
        lines_toRemove = []
        for i in range(size):
            if(dataset.loc[i, 'trial_id'][0] not in trials): lines_toRemove += [i]
        new_ds = dataset.drop(lines_toRemove)
        return new_ds
    else:
        for i in range(len(check_mask)):
            if(not check_mask[i]): print('Trial ', trials[i], ' not in the dataset')

In [ ]:
start_time = time.time()
#trial_datasetMaker(val_dataset, [3])
trials_present(val_dataset)
end_time = time.time()
elapsed_time = end_time-start_time

print(f'It took: {elapsed_time} seconds')

In [ ]:
#Returns the trials present in one dataset
def trials_present(dataset):
    size = len(dataset)
    trials = []
    for i in range(size):
        if(dataset.loc[i, 'trial_id'][0] not in trials): trials += [dataset.loc[i, 'trial_id'][0]]
    return trials

In [ ]:
listing = trials_present(val_dataset)
nr_cols = 4
save = True


In [ ]:
plot_ccSeperate(val_dataset, trials_present(val_dataset), nr_cols=4, save=True, compare_axis=True)

## 3º Meeting - Improve Data Visualization

### Plot-Helper Functions

In [ ]:
'''
Plots the cross-correlation
'''
def plot_crossCorrelation(cross_corr, lags, trial, compare_axis=False):
    plt.plot(lags, cross_corr)
    plt.xlabel('Lag (ms)')
    plt.ylabel('Pearsons Coefficient')
    plt.title(f'Cross-correlation of Trial {trial} ({metric[0]})')
    if(compare_axis): plt.ylim(-1.1,1.1)
    plt.grid(True)
    plt.show()
    
    # Find the lag at which cross-correlation is maximum
    max_corr, max_lag = maximaze_corrParameters(cross_corr, lags)
    print("Maximum cross-correlation at lag: ", max_lag)
    print("Maximum cross-correlation of: ", max_corr)

In [ ]:
'''
Plots the cross-correlation for each trial in a dataset in different subplots. It also shows the point of higher 
absolute pearson's value

Parameteres
-----------
dataset - pandas.Dataframe
    The required dataset with at least two columns named and with the specified metrics
trials - array_like
    The list of trials to be plotted (must be present in the dataset)
metric - array_like of strings
    The list with the column name of actual metric and the predicted metric
correlation - string, optional
    Define the correlation metric applied
    - 'cross' (default)
        Uses the normalized cross correlation
    - 'pearson'
        Uses the pearson correlation
window_size - integer, optional
    The size of the visualization window in the x axis (-window_size+1, window_size)
bin_size - float, optional
    Specifies what is the size of each bin in ms
nr_cols - int, optional
    Specifies the number of columns for the grid where all the plots will sit on
compare_axis - bool, optional
    If true the y axis will have the full correlation scope (-1 - margin, 1 + margin) 
save - bool, optional
    If true the plot will be saved in the same folder
save_name - string, optional
    If save true will define the file name and type
'''
def plot_ccSeperate(dataset, listings, metric, correlation='cross', window_size=400, bin_size=5, nr_cols=4, compare_axis=False, save=False, 
                    save_name='all_corrs.png'):
    #Prepares subplot organization
    nr_plots = len(listings)
    nr_rows = nr_plots//nr_cols
    if(nr_plots%nr_cols == 0): extra_row = 0
    else: extra_row = 1
    
    #Creates the subplots accordingly
    fig, axes = plt.subplots(nr_rows+extra_row, nr_cols, figsize=(12, (nr_rows+extra_row)*3))
        
    #Plots the data
    for i in range(nr_rows+extra_row):
        if((extra_row == 1 and i < nr_rows+extra_row-1) or extra_row == 0):
            for j in range(nr_cols):
                cross_corr, lags = trial_crossCorrelation(dataset, listings[i*nr_cols + j], metric, correlation=correlation, 
                                                          time_window=window_size, plot=False)
                max_corr, max_lag = maximaze_corrParameters(cross_corr, lags)
                
                axes[i,j].plot(lags, cross_corr)
                axes[i,j].scatter(max_lag, max_corr, color='red', label=f'Corr: {round(max_corr,2)}\nLag: {max_lag} ms')
                axes[i,j].set_xlabel('Lag (ms)')
                axes[i,j].grid(True)
                if(correlation == 'pearson'): axes[i,j].set_ylabel('Pearson\'s Correlation')   
                elif(correlation == 'cross'): axes[i,j].set_ylabel('Cross-Correlation')
                axes[i,j].set_title(f'Trial {listings[i*nr_cols + j]}', fontsize=10)
                axes[i,j].legend(loc='lower left')
                if(compare_axis): axes[i,j].set_ylim(-1.1, 1.1)
        elif(extra_row == 1 and i == nr_rows+extra_row-1):
            for j in range(nr_plots%nr_cols):
                cross_corr, lags = trial_crossCorrelation(dataset, listings[i*nr_cols + j], metric, correlation=correlation, 
                                                          time_window=window_size, plot=False)
                max_corr, max_lag = maximaze_corrParameters(cross_corr, lags)
            
                axes[i,j].plot(lags, cross_corr)
                axes[i,j].scatter(max_lag, max_corr, color='red', label=f'Corr: {round(max_corr,2)}\nLag: {max_lag} ms')
                axes[i,j].set_xlabel('Lag (ms)')
                if(correlation == 'pearson'): axes[i,j].set_ylabel('Pearson\'s Correlation')   
                elif(correlation == 'cross'): axes[i,j].set_ylabel('Cross-Correlation')
                axes[i,j].set_title(f'Trial {listings[i*nr_cols + j]}', fontsize=10)
                axes[i,j].grid(True)
                axes[i,j].legend(loc='lower left')
                if(compare_axis): axes[i,j].set_ylim(-1.1, 1.1)
    if(correlation == 'pearson'): plt.suptitle(f'Pearson\'s Correlation for All Trials ({metric[0]})', y=0.995)
    elif(correlation == 'cross'): plt.suptitle(f'Cross-Correlation for All Trials ({metric[0]})', y=0.995)
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    plt.show()

In [ ]:
'''
Plots the correlation for each trial in a dataset in the same plot.

Parameteres
-----------
dataset - pandas.Dataframe
    The required dataset with at least two columns named and with the specified metrics
trials - array_like
    The list of trials to be plotted (must be present in the dataset)
metric - array_like of strings
    The list with the column name of actual metric and the predicted metric
window_size - integer, optional
    The size of the visualization window in the x axis (-window_size+1, window_size)
correlation - string, optional
    Define the correlation metric applied
    - 'cross' (default)
        Uses the normalized cross correlation
    - 'pearson'
        Uses the pearson correlation
bin_size - float, optional
    Specifies what is the size of each bin in ms
focus_trial - {None, int, string}, optional
    Defines a plotted line that will recieve emphasis
    - ''None'' (default)
        No trial recieves emphasis
    - ''int''
        The specified trial recieves emphasis (needs to be in the trials array)
    - 'max'
        The trial with higher correlation recieves emphasis
    - 'min'
        The trial with lower correlation recieves emphasis
    - 'best'
        The trial with the best correlation recieves emphasis (positive or negative)
    - 'worst'
        The trial with the worst correlation recieves emphasis (closer to 0)
    - 'best_lag'
        The trial with the lowest lag for the highest correlation recieves emphasis (closer to 0)
    - 'worst_lag'
        The trial with the highest lag for the highest correlation recieves emphasis (positive or negative)
    - 'mean'
        A new plot line with the mean and the standard error shaded will recieve emphasis with is maximum correlation 
        for a given lag pointed out
compare_axis - bool, optional
    If true the y axis will have the full correlation scope (-1 - margin, 1 + margin) 
color_template - string, optional
    Defines the plt.colormaps used for the plot lines color (uses the full spectrum)
save - bool, optional
    If true the plot will be saved in the same folder
save_name - string, optional
    If save true will define the file name and type
    
'''
def plot_ccTogether(dataset, trials, metric, window_size=400, correlation='cross', bin_size=5, focus_trial=None, compare_axis=False, 
                    color_template='winter', save=False, save_name='all_corrs.png'):
    plt.figure(figsize=(15,10))
    nr_trials = len(trials)
    color_array = np.round(np.linspace(0, 255, nr_trials))
    if(nr_trials > 30): no_label = True
    else: no_label = False

    corr_matrix = []
    max_matrix = []
    max_correlations = []
    max_correlationsAbs = []
    max_lags = []
    max_lagsAbs = []
    for i in range(nr_trials):
        cross_corr, lags = trial_crossCorrelation(dataset, trials[i], metric, correlation=correlation, time_window=window_size, 
                                                  bin_size=bin_size, plot=False)
        max_corr, max_lag = maximaze_corrParameters(cross_corr, lags)
        max_correlations += [max_corr]
        max_correlationsAbs += [np.abs(max_corr)]
        max_lags += [max_corr]
        max_lagsAbs += [np.abs(max_lag)]
        corr_matrix += [[cross_corr, lags]]
        max_matrix += [[max_corr, max_lag]]
    corr_matrix = np.asarray(corr_matrix, dtype='float64')
    
    if(focus_trial == 'max'):
        focus_trial = trials[np.argmax(max_correlations)]
    elif(focus_trial == 'min'):
        focus_trial = trials[np.argmin(max_correlations)]
    elif(focus_trial == 'best'):
        focus_trial = trials[np.argmax(max_correlationsAbs)]
    elif(focus_trial == 'worst'):
        focus_trial = trials[np.argmin(max_correlationsAbs)]
    elif(focus_trial == 'best_lag'):
        focus_trial = trials[np.argmin(max_lagsAbs)]
    elif(focus_trial == 'worst_lag'):
        focus_trial = trials[np.argmax(max_lagsAbs)]
        
    for i in range(nr_trials):
        cross_corr = corr_matrix[i][0]
        lags = corr_matrix[i][1]
        max_corr = max_matrix[i][0]
        max_lag = max_matrix[i][1]
        if(focus_trial == None):
            if(not no_label): plt.plot(lags, cross_corr, color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), 
                                       label=f'Trial {trials[i]}', alpha=1)
            else: plt.plot(lags, cross_corr, color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), alpha=1)
        elif(focus_trial == trials[i]):
            plt.plot(lags, cross_corr, color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), label=f'Trial {trials[i]}', 
                     alpha=1)
            plt.scatter(max_lag, max_corr, color='red', label=f'Corr: {round(max_corr,2)}\nLag: {max_lag} ms')
        else:
            if(not no_label): plt.plot(lags, cross_corr, color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), 
                                       label=f'Trial {trials[i]}', alpha=0.25)
            else: plt.plot(lags, cross_corr, color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), alpha=0.1)

    if(focus_trial == 'mean'):
        mean_vector = np.mean(corr_matrix, axis=0)
        ste_vector = np.std(corr_matrix, axis=0)
        ste_values = ste_vector[0] / np.sqrt(len(ste_vector[0]))
        cross_corr = mean_vector[0]
        lags = mean_vector[1]
        max_corr, max_lag = maximaze_corrParameters(cross_corr, lags)
        plt.plot(lags, cross_corr, color='black', label='Mean Corr', alpha=1)
        plt.fill_between(lags, cross_corr-ste_values, cross_corr+ste_values, color='black', label='Stand Err', alpha=0.25)
        plt.scatter(max_lag, max_corr, color='red', label=f'Corr: {round(max_corr,2)}\nLag: {max_lag} ms')

    plt.xlabel('Lag (ms)')
    if(correlation == 'cross'):
        plt.ylabel('Cross-Correlation')
        plt.title(f'Cross-Correlation of All Trials ({metric[0]})')
    if(correlation == 'pearson'):
        plt.ylabel('Pearson\'s Correlation')
        plt.title(f'Pearson\'s Correlation of All Trials ({metric[0]})')
    if(compare_axis): plt.ylim(-1.1,1.1)
    plt.grid(True)
    plt.legend(loc='lower left')
    if(save): plt.savefig(save_name)
    plt.show()
    return corr_matrix

In [ ]:
'''
Plots in the same graph a predicetd metric and the actual metric value throughout time for a single trial
'''
def plot_metricComparition_single (dataset, trial, metric, bin_size = 5, compare_axis=True):
    trial_dataset = trial_datasetMaker(dataset, [trial])
    trial_dataset = np.asarray(trial_dataset.loc[:,[metric[0], metric[1]]], dtype='float64')
    
    actual_angle = trial_dataset[:,0]
    pred_angle = trial_dataset[:,1]
    trial_time = np.round(np.linspace(0, bin_size*len(actual_angle), len(actual_angle)))
    
    
    plt.plot(trial_time, actual_angle)
    plt.plot(trial_time, pred_angle)
    plt.xlabel('Duration (ms)')
    plt.ylabel('Angle (rads)')
    plt.title(f'{metric[0]} and {metric[1]} of Trial {trial}')
    if(compare_axis): plt.ylim(-np.pi-0.1,np.pi+0.1)
    plt.grid(True)
    plt.show()

In [ ]:
'''
Plots for all the trials in listings a parameter and its predicted version (in this case, 'angle', 'cosx' or 'cosy')
'''
def plot_metricComparition_allS(dataset, listing, metric, bin_size = 5, nr_cols = 4, compare_axis=(-np.pi-1, np.pi +1), save = False, 
                                save_name = 'test.png'):
    #Prepares subplot organization
    nr_plots = len(listing)
    nr_rows = nr_plots//nr_cols
    if(nr_plots%nr_cols == 0): extra_row = 0
    else: extra_row = 1
    
    #Creates the subplots accordingly
    fig, axes = plt.subplots(nr_rows+extra_row, nr_cols, figsize=(12, (nr_rows+extra_row)*3))
    
    #Plots the data
    for i in range(nr_rows+extra_row):
        if((extra_row == 1 and i < nr_rows+extra_row-1) or extra_row == 0):
            for j in range(nr_cols):
                trial_dataset = trial_datasetMaker(dataset, [listing[i*nr_cols + j]])
                trial_dataset = np.asarray(trial_dataset.loc[:,[metric[0], metric[1]]], dtype='float64')
                
                actual_angle = trial_dataset[:,0]
                pred_angle = trial_dataset[:,1]
                trial_time = np.round(np.linspace(0, bin_size*len(actual_angle), len(actual_angle)))
                
                axes[i,j].plot(trial_time, actual_angle, label=f'{metric[0]}')
                axes[i,j].plot(trial_time, pred_angle, label=f'{metric[1]}')
                axes[i,j].set_xlabel('Trial Duration (ms)')
                axes[i,j].set_ylabel(f'{metric[0]} (rads)')
                axes[i,j].grid(True)
                k = np.sqrt(np.sum(actual_angle*actual_angle)*np.sum(pred_angle*pred_angle))
                cross_corr = round(np.sum(actual_angle*pred_angle)/k,2)
                axes[i,j].set_title(f'Trial: {listing[i*nr_cols + j]}, Corr={cross_corr}', fontsize=10)
                axes[i,j].legend(loc='upper left')
                if(compare_axis != None): axes[i,j].set_ylim(compare_axis)
        elif(extra_row == 1 and i == nr_rows+extra_row-1):
            for j in range(nr_plots%nr_cols):
                trial_dataset = trial_datasetMaker(dataset, [listing[i*nr_cols + j]])
                trial_dataset = np.asarray(trial_dataset.loc[:,[metric[0], metric[1]]], dtype='float64')
                
                actual_angle = trial_dataset[:,0]
                pred_angle = trial_dataset[:,1]
                trial_time = np.round(np.linspace(0, bin_size*len(actual_angle), len(actual_angle)))
            
                axes[i,j].plot(trial_time, actual_angle, label=f'{metric[0]}')
                axes[i,j].plot(trial_time, pred_angle, label=f'{metric[1]}')
                axes[i,j].set_xlabel('Trial Duration (ms)')
                axes[i,j].set_ylabel(f'{metric[0]} (rads)')
                axes[i,j].grid(True)
                k = np.sqrt(np.sum(actual_angle*actual_angle)*np.sum(pred_angle*pred_angle))
                cross_corr = round(np.sum(actual_angle*pred_angle)/k,2)
                axes[i,j].set_title(f'Trial: {listing[i*nr_cols + j]}, Corr={cross_corr}', fontsize=10)
                axes[i,j].legend(loc='upper left')
                if(compare_axis != None): axes[i,j].set_ylim(compare_axis)
    plt.title(f'{metric[0]} and {metric[1]} for Multiple Trials')
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    plt.show()

In [ ]:
'''
Plots the tuning curves
'''
def plot_tuning(dataset, decoder, nr_directions, neurons, nr_cols=5, threshold=0, save=False, save_name='plot.png'):
    mean_matrix, predicted_matrix, target_angles = tune_neurons(dataset, decoder, nr_directions, neurons)
    max_rate = np.max([np.max(mean_matrix), np.max(predicted_matrix)])+1
    if(len(mean_matrix[0])%nr_cols == 0): nr_rows = len(mean_matrix[0])//nr_cols
    else: nr_rows = len(mean_matrix[0])//nr_cols + 1
    
    # Create a figure and subplots
    fig, axs = plt.subplots(nrows=nr_rows, ncols=nr_cols, figsize=(nr_cols*3, nr_rows*3))
    
    # Flatten the axs array for easier iteration
    axs = axs.flatten()
    
    # Plot each array
    for i in range(len(neurons)):
        ax = axs[i]
        observed = mean_matrix[:,i]
        predicted = predicted_matrix[i]
        height = round(np.max(predicted_matrix[i])-np.min(predicted_matrix[i]),3)
        if(height > threshold):
            ax.plot(target_angles, observed, label='Observed Tuning Curve')
            ax.plot(target_angles, predicted, label=f'Predicted Tuning Curve\nHeight: {height}')
            ax.set_title(f'Neuron {neurons[i]}', fontsize=10)
        else: 
            ax.plot(target_angles, observed, label='Observed Tuning Curve', alpha=0.3)
            ax.plot(target_angles, predicted, label=f'Predicted Tuning Curve\nHeight: {height}', alpha=0.3)
            ax.set_title(f'Neuron {neurons[i]}', fontsize=10)
        ax.set_ylim(0, max_rate)
        ax.legend()
        ax.grid(True)
    
    for ax in fig.get_axes():
        ax.label_outer()
        
    plt.suptitle(f'Tuning Curves\n with {nr_directions} angles and threshold at {threshold}', y=1, fontsize=15)
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    plt.show()

In [ ]:
'''
Plots the directions of the desired trial (NOT OPTIMIZED)
'''
def plot_desiredTrajectory(val_dataset, desired_trial):
    x_coords = []
    y_coords = []
    actual_x = []
    actual_y = []
    for i in range(len(val_dataset)):
        if(val_dataset.loc[i, 'trial_id'][0] == desired_trial):
            x_coords += [val_dataset.loc[i, 'pred_X'][0]]
            y_coords += [val_dataset.loc[i, 'pred_Y'][0]]
            actual_x += [val_dataset.loc[i, 'hand_pos']['x']]
            actual_y += [val_dataset.loc[i, 'hand_pos']['y']]
    colors = np.random.rand(len(y_coords))
    plt.plot(x_coords, y_coords, label='Predicted')
    #plt.scatter(x_coords, y_coords, label='Predicted', c=colors, cmap='viridis')
    plt.plot(actual_x, actual_y, label='Actual')
    plt.title(f"Cursor Trajectory for Trial {desired_trial}")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
'''
Plots all trajectories separate (with trial duration in ms)
'''
def plot_allTrajectoriesS(dataset, original_dataset, nr_cols=4, save=False, save_name='plot.png'):
    #Gets the hand positions (trua and pred)
    cursor_pos = np.asarray(dataset['cursor_pos'], dtype='float64')
    true_pos = np.asarray(dataset['hand_pos'], dtype='float64')
    predicted_x = np.asarray(dataset['pred_X'], dtype='float64')
    predicted_y = np.asarray(dataset['pred_Y'], dtype='float64')
    true_x = true_pos[:,0]
    true_y = true_pos[:,1]
    cursor_x = cursor_pos[:,0]
    cursor_y = cursor_pos[:,1]
    
    #Gets the trials and time
    trial_ids = np.asarray(dataset['trial_id'], dtype='int64')
    mask = np.concatenate(([True], trial_ids[1:] != trial_ids[:-1]))
    split_indices = np.where(mask)[0]
    align_time = np.asarray(dataset['align_time'])
    
    #Gets the target positions per trial
    trials = np.unique(trial_ids)
    target_pos = np.asarray(original_dataset.trial_info[['active_pos_x', 'active_pos_y']], dtype='int64')[trials]
    barrier_pos = np.asarray(original_dataset.trial_info['barrier_pos'])[trials]
    barrier_lengths = np.array([len(inner_array) for inner_array in barrier_pos])
    
    
    #Splits all arrays acording to the trials
    predicted_x = np.split(predicted_x, split_indices)[1:]
    predicted_y = np.split(predicted_y, split_indices)[1:]
    align_time = np.split(align_time, split_indices)[1:]
    true_x = np.split(true_x, split_indices)[1:]
    true_y = np.split(true_y, split_indices)[1:]
    cursor_x = np.split(cursor_x, split_indices)[1:]
    cursor_y = np.split(cursor_y, split_indices)[1:]
    
    nr_trials = len(trials)
    #The ploting starts
    if(nr_trials%nr_cols == 0): nr_rows = nr_trials//nr_cols
    else: nr_rows = nr_trials//nr_cols + 1
    
    fig, axs = plt.subplots(nrows=nr_rows, ncols=nr_cols, figsize=(nr_cols*4, nr_rows*4))
    axs = axs.flatten()
    
    #Plotting
    for i in range(nr_trials):
        ax = axs[i]
        x_coords = true_x[i]
        y_coords = true_y[i]
        x_pred = predicted_x[i]
        y_pred = predicted_y[i]
        x_cursor = cursor_x[i]
        y_cursor = cursor_y[i]
        target_x = target_pos[i,0]
        target_y = target_pos[i,1]
        duration = int(align_time[i][-1])/1000000000
        distance = round(np.sqrt(np.power(x_pred[-1]-x_coords[-1],2) + np.power(y_pred[-1]-y_coords[-1],2)),3)
        barriers = barrier_pos[i]
    
        #Gets the limit
        x_lim = np.max([np.max(x_coords), np.max(x_pred), np.max(x_cursor), np.abs(np.min(x_coords)), np.abs(np.min(x_pred)), 
                        np.abs(np.min(x_cursor))])
        y_lim = np.max([np.max(y_coords), np.max(y_pred), np.max(y_cursor), np.abs(np.min(y_coords)), np.abs(np.min(y_pred)), 
                        np.abs(np.min(y_cursor))])
        lim_value = np.max([x_lim,y_lim])+15
        
        #Plots the barriers
        for j in range(barrier_lengths[i]):
            x, y, half_width, half_height = barrier_pos[i][j]
            left = x - half_width
            bottom = y - half_height
            width = 2 * half_width
            height = 2 * half_height
            ax.add_patch(plt.Rectangle((left, bottom), width, height, facecolor='gray'))
    
        ax.plot(x_cursor, y_cursor, label='Cursor Path')
        ax.plot(x_pred, y_pred, label='Predicted Path')
        ax.plot(x_coords, y_coords, label='True Path')
        ax.scatter(target_x, target_y, label='Target Position', color='red')
        ax.scatter(x_coords[0], y_coords[0], label='Start Point', color='black')
        ax.set_title(f'Trial {trials[i]}\nLasted {duration} seconds\nDistance Final: {distance}')
        ax.set_xlim(-lim_value, lim_value)
        ax.set_ylim(-lim_value, lim_value)
        ax.legend()
        ax.grid(True)
    
    plt.suptitle(f'Trial Trajectories', y=1, fontsize=20)
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    plt.show()

### Helper Functions

In [ ]:
def maximaze_corrParameters(cross_corr, lags):
    max_corr_index = np.argmax(cross_corr)
    min_corr_index = np.argmin(cross_corr)
    max_corr_lag = lags[max_corr_index]
    min_corr_lag = lags[min_corr_index]
    max_corr = round(cross_corr[max_corr_index],2)
    min_corr = round(cross_corr[min_corr_index],2)
    if(abs(max_corr)<abs(min_corr)): 
        max_corr = min_corr
        max_corr_lag = min_corr_lag
    return max_corr, max_corr_lag

In [ ]:
'''
Computes and plots the cross-correlation for a given trial in a dataset
'''
def trial_crossCorrelation (dataset, trial, metric, time_window=150, correlation='cross', bin_size=5, plot=True, compare_axis=False):
    bin_window = time_window//bin_size
    if(not isinstance(trial, numbers.Integral)):
        print('Please input only one trial as an integer')
        return

    trial_dataset = trial_datasetMaker(dataset, [trial])
    predicted = np.asarray(trial_dataset[metric[1]], dtype='float64')
    actual = np.asarray(trial_dataset[metric[0]], dtype='float64')
    '''
    print('Predicted: ', predicted)
    print('Actual: ', actual)
    print('len of predicted: ', len(predicted))
    print('len of actual: ', len(actual))
    '''
    if(correlation == 'cross'):
        k = np.sqrt((np.sum(actual*actual))*(np.sum(predicted*predicted)))
        cross_corr = np.correlate(actual, predicted, mode='full')/k
    elif(correlation == 'pearson'):
        cross_corr = cross_pearson(actual, predicted)
    lags = np.arange(-time_window + 1, time_window, bin_size)
    cross_corr = cross_corr[len(cross_corr)//2-bin_window:len(cross_corr)//2+bin_window]

    if(plot): plot_crossCorrelation(cross_corr, lags, trial, compare_axis=compare_axis)   
    return cross_corr, lags

In [ ]:
'''
Returns the dataset for only one trial
'''
def trial_datasetMaker(dataset, trials):
    if(len(trials) == 1):
        trials_possible = trials_present(dataset)
        check_mask = np.isin(trials, trials_possible)[0]
    
        if(check_mask):
            columns = dataset.columns
            matrix = np.asarray(dataset)
            
            trial_ids = np.array(dataset['trial_id'])
            mask = trial_ids == trials[0]
            matrix = matrix[mask]
            trial_dataset = pd.DataFrame(matrix, columns=columns)
            return trial_dataset
        else:
            print('Trial ', trials[0], ' not in the dataset')

In [ ]:
'''
Returns the trials present in one dataset
'''
def trials_present(dataset):
    trial_dataset = np.asarray(dataset['trial_id'], dtype='int64')
    return np.unique(trial_dataset)

In [ ]:
'''
Returns the cross-correlation with pearson correlation values
'''
def cross_pearson(array1, array2):
    corr_p = np.zeros(len(array2) + len(array1) - 1)
    size = len(array2) - 1
    #left pad
    for index, i in enumerate(range(size, 0, -1)):
        x = np.pad(array2,(i), mode='constant')
        x = x[i*2:]
        corr_p[index] = np.corrcoef(array1,x)[0,1]
    #righ pad
    for index, i in enumerate(range(0, size+1)):
        index = index+size
        x = np.pad(array2,(i), mode='constant')
        x = x[:len(x)-i*2]
        corr_p[index] = np.corrcoef(array1,x)[0,1]
    return corr_p

In [ ]:
'''
Generates the Train/Val subset
'''
def generate_subset(dataset, subset_name, start_time='move_onset_time'):
    mask = np.all(dataset.trial_info[['split']] == subset_name, axis=1)
    return dataset.make_trial_data(start_field=start_time, end_field='reach_time', ignored_trials=~mask), mask

In [ ]:
'''
Returns the firing rate of a dataset's spikes with a specific window size
'''
def firing_rate(dataset, window_size, bin_time=5, panda=True):
    size = len(dataset)
    neurons_names = dataset['spikes'].columns
    nr_neurons = len(neurons_names)
    spikes_data = np.asarray(dataset['spikes'], dtype='float64')
    trial_ids = np.asarray(dataset['trial_id'], dtype='int64')
    firing_matrix = []
    trials = np.unique(trial_ids)
    
    for trial in trials:
        mask = trial_ids == trial
        matrix = spikes_data[mask]
        convolved = (convolve2d(matrix, (np.ones((window_size,1))/window_size), mode='same')/5)*1000
        firing_matrix.append(convolved)
    firing_matrix = np.concatenate(firing_matrix, axis=0)
    if(panda): dataset['spikes'] = pd.DataFrame(firing_matrix, columns=np.asarray(neurons_names, dtype='int'))
    else: return firing_matrix, trial_ids

In [ ]:
'''
Builds decoder by fiting the firing rate to cosines. This function does not support delay
'''
def decoder_cosinesNoDelay(dataset, window_size, neurons, bin_time=5):
    #Change spikes to firing rates in ms (moving average) ----------- Can be Optimized
    rate_data, trials_vector = firing_rate(dataset, window_size, bin_time=bin_time, panda=False)
    train_size = len(rate_data)
    nr_neurons = len(neurons)
    
    #Initiates the vectors for calculating the vector M
    hand_pos = np.asarray(dataset['hand_pos'], dtype='float64')
    cosines_matrix = np.zeros((train_size, 2))
    angle = np.zeros(train_size)
    
    #Adds the column vectorM with the cosines components to the train_dataset
    for i in range(train_size):
        if (i == 0 or ((trials_vector[i] != trials_vector[i-1]))):
            preX = 0
            preY = 0
        else:
            preX = hand_pos[i-1,0]
            preY = hand_pos[i-1,1]
        x = hand_pos[i,0]
        y = hand_pos[i,1]
        angle_temp = np.arctan2((y - preY), (x - preX))
        cosX = np.cos(angle_temp)
        cosY = np.sin(angle_temp)
        cosines_matrix[i,0] = cosX
        cosines_matrix[i,1] = cosY
        angle[i] = angle_temp
    
    #Trains the decoder fiting the data to a cos
    decoder = []
    for i in range(nr_neurons):
        neuron_data = rate_data[:,i]
        weights = linear_regression_model(cosines_matrix, neuron_data, loss='ridge')
        k = np.sqrt(np.power(weights[1],2) + np.power(weights[2],2))
        if(k == 0): 
            print(f'neuron {neurons[i]} did not fire')
            cx = 1
            cy = 0
        else:
            cx = weights[1]/k
            cy = weights[2]/k
        angleC = np.arctan2(cy, cx)
        decoder.append([neurons[i], cx, cy, weights[0], angleC, k])
    train_dataset['spikes'] = rate_data
    train_dataset['angle'] = angle
    return np.array(decoder)

In [ ]:
'''
Builds decoder by fiting the firing rate to cosines. This function applies delay
'''
def decoder_cosinesDelay(dataset, window_size, delay, bin_time=5):
    #Change spikes to firing rates in ms (moving average)
    train_size = len(dataset)
    neurons_names = dataset['spikes'].columns
    nr_neurons = len(neurons_names)
    
    spikes_data = np.asarray(dataset['spikes'], dtype='float64')
    trial_ids = np.asarray(dataset['trial_id'], dtype='int64')
    hand_pos = np.asarray(dataset['hand_pos'], dtype='float64')
    trials = np.unique(trial_ids)
    rate_data = []
    cosines_matrix = []
    angle = []
    
    for trial in trials:
        mask = trial_ids == trial
        trial_spike = spikes_data[mask]
        trial_rate = (convolve2d(trial_spike, (np.ones((window_size,1))/window_size), mode='same')/5)*1000
        rate_data.append(trial_rate)
    
        #Gets the delay if needed
        trial_pos = hand_pos[mask]
        trial_pos = np.pad(trial_pos,(delay), mode='constant')
        size = len(trial_pos)
        trial_pos = trial_pos[:size-delay*2,1*delay:-1*delay]
        
        #Gets the data for the cosines
        for i in range(len(trial_spike)):
            if (i == 0):
                preX = 0
                preY = 0
            else:
                preX = trial_pos[i-1,0]
                preY = trial_pos[i-1,1]
            x = trial_pos[i,0]
            y = trial_pos[i,1]
            angle_temp = np.arctan2((y - preY), (x - preX))
            cosX = np.cos(angle_temp)
            cosY = np.sin(angle_temp) 
            cosines_matrix.append([cosX, cosY])
            angle.append(angle_temp)
            
    rate_data = np.concatenate(rate_data, axis=0)
    
    #Trains the decoder fiting the data to a cos
    decoder = []
    for i in range(nr_neurons):
        neuron_data = rate_data[:,i]
        weights = linear_regression_model(cosines_matrix, neuron_data, loss='ridge')
        k = np.sqrt(np.power(weights[1],2) + np.power(weights[2],2))
        if(k == 0): 
            print(f'neuron {neurons[i]} did not fire')
            cx = 1
            cy = 0
        else:
            cx = weights[1]/k
            cy = weights[2]/k
        angleC = np.arctan2(cy, cx)
        decoder.append([neurons_names[i], cx, cy, weights[0], angleC, k])

    train_dataset['spikes'] = rate_data
    train_dataset['angle'] = angle
    return np.array(decoder)

In [ ]:
'''
Produces the tune curve data for a set number of angles for the observed firing rate and for the predicted firing rate
'''

def tune_neurons(dataset, decoder, nr_directions, neurons):
    rate_data = np.asarray(dataset['spikes'], dtype='float64')
    angle_data = np.asarray(dataset['angle'], dtype='float64')
    target_angles = np.linspace(-np.pi, np.pi, nr_directions, endpoint=False)
    
    closest_angle_idxs = np.argmin(np.abs(target_angles[:, np.newaxis] - angle_data), axis=0)
    angle_data = target_angles[closest_angle_idxs]
    
    #Gets the observed curve
    mean_matrix = []
    for target_angle in (target_angles):
        mask = (angle_data == target_angle)
        relevant_rates = rate_data[mask]
        mean_rate = np.mean(relevant_rates, axis=0)
        mean_matrix.append(mean_rate)
    mean_matrix = np.asarray(mean_matrix, dtype='float64')
    
    #Gets the predicted curve
    bias = decoder[:, 3]
    k = decoder[:, 5]
    angle = decoder[:, 4]
    prediced_matrix = bias[:, np.newaxis] + k[:, np.newaxis] * np.cos(target_angles - angle[:, np.newaxis])
    prediced_matrix = np.asarray(prediced_matrix, dtype='float64')
    
    return mean_matrix, prediced_matrix, target_angles

In [ ]:
'''
Removes the neurons from the decoder and prepares the val_dataset for predictions
'''
def remove_neurons(data, dataset, neurons, threshold, remove_fromDataset=True):
    #Gets the values from the metric used (depth)
    max_vector = np.max(data, axis=1)
    min_vector = np.min(data, axis=1)
    depth_vector = max_vector-min_vector
    
    #Builds arrays with the rejected and acepted neurons
    acepted_neurons = np.where(depth_vector>=threshold, neurons, None)
    rejected_neurons = np.where(depth_vector<threshold, neurons, None)
    acepted_neurons = np.asarray(np.ma.masked_equal(acepted_neurons, None).compressed(), dtype='int64')
    
    #Removes from the decoder
    new_decoder = decoder[rejected_neurons == None]
    
    #Removes those columns from the dataset
    rejected_neurons = np.asarray(np.ma.masked_equal(rejected_neurons, None).compressed(), dtype='int64')
    if(remove_fromDataset):
        col_toRemove = np.empty((len(rejected_neurons)), dtype=tuple)
        for i, neuron in enumerate(rejected_neurons):
            col_toRemove[i] = ('spikes', neuron)
        dataset.drop(columns=col_toRemove, inplace=True)
    return new_decoder, acepted_neurons

In [ ]:
'''
Linear regression with given loss function and given regularization that returns the weights
'''
def linear_regression_model(trainning_var, trainning_out, loss='sse', lambaRidge=2):
    #Data preparation
    rates = trainning_var
    vel = trainning_out
    
    #Loss Function Selection
    if(loss == 'sse'):
        rates = np.insert(rates, 0, 1, axis=1)
        penrose = np.linalg.pinv(rates.astype(np.float32))
        w = np.matmul(penrose,vel)
        predicted = np.matmul(rates,w)
    elif(loss == 'ridge'):
        rates = np.insert(rates, 0, 1, axis=1)
        ratesT = np.transpose(rates)
        prod = np.matmul(ratesT,rates)
        identity = np.identity(len(prod))
        helper = prod + lambaRidge * identity
        inverse = np.linalg.inv(helper)
        penrose = np.matmul(inverse,ratesT)
        w = np.matmul(penrose,vel)
        predicted = np.matmul(rates,w)

    return w

### Functions

In [ ]:
'''
Generates a decoder by fiting the training data to cosines (based in georgopolos et. al)
'''
def angle_decoder(dataset, window_size, neurons, delay=0, bin_size=5):
    if(delay != 0): decoder = decoder_cosinesDelay(dataset, window_size, delay, bin_time=bin_size)
    else: decoder = decoder_cosinesNoDelay(dataset, window_size, neurons, bin_time=bin_size)
    return decoder

In [ ]:
'''
Decodes the hand position (angle, cosines and trajectory) by applying a decoder fitted to the cosines
'''

def angle_predictor(dataset, decoder, window_size, neurons, bin_time=5, distance_coef=0.05, pred_pos=True):
    #Prepares the data
    rate_data, trials_ids = firing_rate(dataset, window_size, bin_time=bin_time, panda=False)
    val_size = len(dataset)
    
    #Predicts the direction
    individual_predX = (rate_data - decoder[:,3])*decoder[:,1]
    individual_predY = (rate_data - decoder[:,3])*decoder[:,2]
    pred_cosX = np.sum(individual_predX, axis=1)
    pred_cosY = np.sum(individual_predY, axis=1)
    pred_angle = np.arctan2(pred_cosY, pred_cosX)
    pred_magnitude = np.sqrt(np.power(pred_cosX,2) + np.power(pred_cosY,2))
    #Normalizes the cos and sin
    pred_cosX = np.cos(pred_angle)
    pred_cosY = np.sin(pred_angle)
    
    #Adds it to the dataset
    dataset['pred_mag'] = pred_magnitude
    dataset['pred_cosX'] = pred_cosX
    dataset['pred_cosY'] = pred_cosY
    dataset['pred_angle'] = pred_angle
    dataset['spikes'] = rate_data
    
    #Builds the vectorM for comparition
    trial_pos = np.asarray(dataset['hand_pos'], dtype='float64')
    mask = np.concatenate(([True], trials_ids[1:] != trials_ids[:-1]))
    diffs = trial_pos - np.roll(trial_pos, 1, axis=0)
    diffs = np.where(mask[:, np.newaxis], np.zeros_like(diffs), diffs)
    angles = np.arctan2(diffs[:, 1], diffs[:, 0])
    cosines_matrix = np.column_stack((np.cos(angles), np.sin(angles)))
    
    dataset['mx'] = cosines_matrix[:,0]
    dataset['my'] = cosines_matrix[:,1]
    dataset['angle'] = angles
    
    #Predicts the position if needed
    if(pred_pos):
        pred_cosNX = np.roll(pred_cosX, 1)
        pred_cosNY = np.roll(pred_cosY, 1)
        pred_magnitude = np.roll(pred_magnitude, 1)
        pred_x = np.where(mask, trial_pos[:,0], pred_cosNX * pred_magnitude * distance_coef)
        pred_y = np.where(mask, trial_pos[:,1], pred_cosNY * pred_magnitude * distance_coef)
    
        for i in range(len(pred_x)):
            if(not mask[i]): 
                pred_x[i] = pred_x[i-1] + pred_x[i]
                pred_y[i] = pred_y[i-1] + pred_y[i]
        
        dataset['pred_X'] = pred_x
        dataset['pred_Y'] = pred_y

### Crafting Table

In [ ]:
'''
Predictions made with decoder trained with hand_pos covariate and cosines angles (paper)
Moving Average Size: 35
Time Required: 1 sec
'''
dataset = smallDS
neurons = small_neurons
plot_correlation = True

#Creates the training data
train_dataset, mask = generate_subset(dataset, 'train')

#Generates a decoder for every neuron
start_time = time.time()
decoder = angle_decoder(train_dataset, 35, neurons)
end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'Decoding took {elapsed_time} sec')

#Creates the validation data
val_dataset, mask = generate_subset(dataset, 'val')

#Runs the tuning curve for every neuron
start_time = time.time()
_, predicted_matrix,_ = tune_neurons(train_dataset, decoder, 360, neurons)
end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'Tuning curve took {elapsed_time} sec')

#Removes unecessary neurons
start_time = time.time()
decoder, neurons = remove_neurons(predicted_matrix, val_dataset, neurons, 0)
end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'Neuron removal took {elapsed_time} sec')

#Predicts every angle according to a decoder
start_time = time.time()
angle_predictor(val_dataset, decoder, 35, neurons, distance_coef=0.02)
end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'Angle predictor took {elapsed_time} sec')

#Plots Correlation
metrics = [['mx', 'pred_cosX'],['my', 'pred_cosY'],['angle', 'pred_angle']]
file_names = ['mean_pearson_all_cosx_standardDS_removed1.png','mean_pearson_all_cosy_standardDS_removed1.png',
              'mean_pearson_all_angle_standardDS_removed1.png']
trials = trials_present(val_dataset)
if(plot_correlation):
    for i in range(len(metrics)):
        plot_ccTogether(val_dataset, trials , metrics[i], focus_trial='mean', correlation='pearson', window_size=150, 
                        compare_axis=True, color_template='twilight', save=True, save_name=file_names[i])

#plot_allPredictions(val_dataset, trials, nr_cols=4)

In [ ]:
metrics = [['angle', 'pred_angle'],['my', 'pred_cosY'],['mx', 'pred_cosX']]
file_names = ['pearson_angle_smallDS.png','pearson_cosy_smallDS.png','pearson_cosx_smallDS.png']
#metric = ['angle', 'pred_angle']
#metric = ['my', 'pred_cosY']
#metric = ['mx', 'pred_cosX']

for i in range(len(metrics)):
    plot_ccSeperate(val_dataset, trials, metrics[i], correlation='pearson', window_size=200, 
                compare_axis=True, save=False, save_name=file_names[i])

In [ ]:
#metric = ['angle', 'pred_angle']
#metric = ['my', 'pred_cosY']
metric = ['mx', 'pred_cosX']
#compare_axis=(-np.pi-1, np.pi +1)
compare_axis=(-1.1, 1.1)

plot_metricComparition_allS(val_dataset, trials, metric, compare_axis=compare_axis)

In [ ]:
if(len(mean_matrix[0])%nr_cols == 0): nr_rows = len(mean_matrix[0])//nr_cols
else: nr_rows = len(mean_matrix[0])//nr_cols + 1

# Create a figure and subplots
fig, axs = plt.subplots(nrows=nr_rows, ncols=nr_cols, figsize=(nr_cols*3, nr_rows*3))

# Flatten the axs array for easier iteration
axs = axs.flatten()

# Plot each array
for i in range(len(neurons)):
    ax = axs[i]
    observed = mean_matrix[:,i]
    predicted = predicted_matrix[i]
    height = round(np.max(predicted_matrix[i])-np.min(predicted_matrix[i]),3)
    if(height > threshold):
        ax.plot(target_angles, observed, label='Observed Tuning Curve')
        ax.plot(target_angles, predicted, label=f'Predicted Tuning Curve\nHeight: {height}')
        ax.set_title(f'Neuron {neurons[i]}', fontsize=10)
    else: 
        ax.plot(target_angles, observed, label='Observed Tuning Curve', alpha=0.3)
        ax.plot(target_angles, predicted, label=f'Predicted Tuning Curve\nHeight: {height}', alpha=0.3)
        ax.set_title(f'Neuron {neurons[i]}', fontsize=10)
    ax.set_ylim(0, max_rate)
    ax.legend()
    ax.grid(True)

for ax in fig.get_axes():
    ax.label_outer()
    
plt.suptitle(f'Tuning Curves\n with {nr_directions} angles and threshold at {threshold}', y=1, fontsize=15)
plt.tight_layout()
if(save): plt.savefig(save_name)
plt.show()

In [ ]:
start_time = time.time()
plot_allTrajectoriesS(val_dataset, standardDS, nr_cols=24, save=True, save_name='standardDS_trajectories.png')
end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'It took {elapsed_time} sec')

In [ ]:
'''
Computes and plots the cross-correlation for a given trial in a dataset
'''
def trial_crossCorrelation (dataset, trial, metric, time_window=150, correlation='cross', bin_size=5, plot=True, compare_axis=False):
    bin_window = time_window//bin_size
    if(not isinstance(trial, numbers.Integral)):
        print('Please input only one trial as an integer')
        return

    trial_dataset = trial_datasetMaker(dataset, [trial])
    predicted = np.asarray(trial_dataset[metric[1]], dtype='float64')
    actual = np.asarray(trial_dataset[metric[0]], dtype='float64')
    '''
    print('Predicted: ', predicted)
    print('Actual: ', actual)
    print('len of predicted: ', len(predicted))
    print('len of actual: ', len(actual))
    '''
    if(correlation == 'cross'):
        k = np.sqrt((np.sum(actual*actual))*(np.sum(predicted*predicted)))
        cross_corr = np.correlate(actual, predicted, mode='full')/k
    elif(correlation == 'pearson'):
        cross_corr = cross_pearson(actual, predicted)
    lags = np.arange(-time_window + 1, time_window, bin_size)
    cross_corr = cross_corr[len(cross_corr)//2-bin_window:len(cross_corr)//2+bin_window]

    if(plot): plot_crossCorrelation(cross_corr, lags, trial, compare_axis=compare_axis)
    print(len(cross_corr))
    return cross_corr, lags

'''
corr_matrix = []
max_matrix = []
max_correlations = []
max_correlationsAbs = []
max_lags = []
max_lagsAbs = []
for i in range(nr_trials):
    cross_corr, lags = trial_crossCorrelation(dataset, trials[i], metric, correlation=correlation, time_window=window_size, 
                                              bin_size=bin_size, plot=False)
    max_corr, max_lag = maximaze_corrParameters(cross_corr, lags)
    max_correlations += [max_corr]
    max_correlationsAbs += [np.abs(max_corr)]
    max_lags += [max_corr]
    max_lagsAbs += [np.abs(max_lag)]
    corr_matrix += [[cross_corr, lags]]
    max_matrix += [[max_corr, max_lag]]
corr_matrix = np.asarray(corr_matrix, dtype='float64')
if(focus_trial == 'mean'):
    mean_vector = np.mean(corr_matrix, axis=0)
    ste_vector = np.std(corr_matrix, axis=0)
    ste_values = ste_vector[0] / np.sqrt(len(ste_vector[0]))
    cross_corr = mean_vector[0]
    lags = mean_vector[1]
    max_corr, max_lag = maximaze_corrParameters(cross_corr, lags)
    plt.plot(lags, cross_corr, color='black', label='Mean Corr', alpha=1)
    plt.fill_between(lags, cross_corr-ste_values, cross_corr+ste_values, color='black', label='Stand Err', alpha=0.25)
    plt.scatter(max_lag, max_corr, color='red', label=f'Corr: {round(max_corr,2)}\nLag: {max_lag} ms')
'''

In [ ]:
'''
Retruns the mean correlation (cross or pearsons) and the standard error
'''

def mean_corr(dataset, metric, correlation, time_window=150, bin_size=5):
    bin_window = time_window//bin_size
    
    actual_vector = np.asarray(dataset[metric[0]], dtype='float64')
    pred_vector = np.asarray(dataset[metric[1]], dtype='float64')
    trial_id = np.asarray(dataset['trial_id'], dtype='int64')
    trials = np.unique(trial_id)
    nr_trials = len(trials)
    mask = np.concatenate(([True], trial_id[1:] != trial_id[:-1]))
    actual_vector = np.split(actual_vector, np.where(mask)[0])[1:]
    pred_vector = np.split(pred_vector, np.where(mask)[0])[1:]
    
    cross_corr_vector = []
    lags = np.arange(-time_window + 1, time_window, bin_size)
    for i in range(nr_trials):
        if(correlation == 'pearson'):
            cross_corr = cross_pearson(actual_vector[i], pred_vector[i])
        elif(correlation == 'cross'):
            k = np.sqrt((np.sum(np.power(actual_vector[i],2)))*(np.sum(np.power(pred_vector[i],2))))
            cross_corr = np.correlate(actual_vector[i], pred_vector[i])/k
        cross_corr = cross_corr[len(cross_corr)//2-bin_window:len(cross_corr)//2+bin_window]
        cross_corr_vector += [np.array(cross_corr)]
    
    cross_corr_vector = np.array(cross_corr_vector)
    mean_correlation = np.mean(cross_corr_vector, axis=0)
    return maximaze_corrParameters(mean_correlation, lags)

start_time = time.time()
max_corr, max_lag = mean_corr(val_dataset, metric, 'pearson')
end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'It took {elapsed_time} sec')

In [ ]:
'''
This will run the decoder multiple times and return the different mean correlations and lags
'''

dataset = smallDS
neurons = small_neurons
bin_size = 5
window_size = [2,5,10,35,50]
thresholds = [0.5,1,1.5,2,2.5]
metrics = [['mx', 'pred_cosX'],['my', 'pred_cosY'],['angle', 'pred_angle']]
correlations = ['pearson']
results = []
parameters = []

for i in window_size:
    for j in thresholds:
        neurons = small_neurons
        #Creates the training data
        train_dataset, mask = generate_subset(dataset, 'train')
        decoder = angle_decoder(train_dataset, i, neurons)
        val_dataset, mask = generate_subset(dataset, 'val')
        _, predicted_matrix,_ = tune_neurons(train_dataset, decoder, 360, neurons)
        decoder, neurons = remove_neurons(predicted_matrix, val_dataset, neurons, j)
        angle_predictor(val_dataset, decoder, i, neurons, distance_coef=0.02, pred_pos=False)
        for l in correlations:
            for k in metrics:
                max_corr, max_lag = mean_corr(val_dataset, k, l)
                results += [[max_corr, max_lag]]
                parameters += [[i*bin_size, j, k[1], l]]
                print(f'Parameteres:\nwindow_size = {i*bin_size} ms\nthreshold = {j} depth\nmetric = {k[1]}\ncorrelation = {l}')
                print('')
                print(f'Results:\nCorrelation = {max_corr}\nLag = {max_lag}')
                print('---------')
                print('')

In [ ]:
results = np.array(results)
max_value = np.max(results[:,0])
max_pos = np.argmax(results[:,0])

print(f'Parameteres:\nwindow_size = {parameters[max_pos][0]} ms\nthreshold = {parameters[max_pos][1]} depth\nmetric = {parameters[max_pos][2]}\ncorrelation = {parameters[max_pos][3]}')
print(f'Result\nCorrelation: {max_value}\nLag: {results[max_pos, 1]}')

In [ ]:
results

In [ ]:
35*5

In [ ]:
len(neurons)

## 4º Meeting - Naive Bayes

### Functions

#### Tuning Curves Functions

In [ ]:
'''
Returns the 3d tunning curves data with a given resolution and the predicted fitted curve --- Optimization Needed
'''
def tune_neurons3D(dataset, model='Gaussian', resolution=50):
    #Extracts data
    vel_x = np.array(dataset['hand_vel']['x'], dtype='float64')
    vel_y = np.array(dataset['hand_vel']['y'], dtype='float64')
    rate_data = np.array(dataset['spikes'], dtype='float64')
    
    #Gets limit points for binarization
    max_velX = np.max(vel_x)
    max_velY = np.max(vel_y)
    min_velX = np.min(vel_x)
    min_velY = np.min(vel_y)
    maximum = np.max([max_velX, max_velY])
    minimum = np.max([min_velX, min_velY])
    vels_resolution = np.linspace(minimum, maximum, resolution, endpoint=False)
    grid_x, grid_y = np.mgrid[minimum:maximum:resolution*1j, minimum:maximum:resolution*1j]
    
    #Binarizes the velocities
    closest_velX_idxs = np.argmin(np.abs(vels_resolution[:, np.newaxis] - vel_x), axis=0)
    closest_velY_idxs = np.argmin(np.abs(vels_resolution[:, np.newaxis] - vel_y), axis=0)
    vel_x = vels_resolution[closest_velX_idxs]
    vel_y = vels_resolution[closest_velY_idxs]
    
    grid1, grid2 = np.meshgrid(vels_resolution, vels_resolution)
    vels_pairs = np.vstack([grid1.ravel(), grid2.ravel()]).T
    velocity_vector = np.column_stack((vel_x, vel_y))
    
    #Average rates in same velocity quadrant ----- Optimize
    mean_matrix = []
    found_pairs = []
    for pair in vels_pairs:
        mask = (pair == velocity_vector)
        mask = np.logical_and(mask[:,0], mask[:,1])
        if(any(mask)):
            relevant_rates = rate_data[mask]
            mean_rate = np.mean(relevant_rates, axis=0)
            mean_matrix.append(mean_rate)
            found_pairs.append(pair)

    mean_matrix = np.asarray(mean_matrix, dtype='float64')
    found_pairs = np.asarray(found_pairs, dtype='float64')

    #Fits a Gaussian Curve ---- Optimize
    storage_gridZ = []
    predicted = []
    if(model == 'Gaussian'):
        bound = ([0,np.nanmin(grid_x), np.nanmin(grid_y), 0,0, -np.inf, -1], 
                         [100,np.nanmax(grid_x), np.nanmax(grid_y), 1000, 1000, np.inf, 100])
    elif(model == 'Cauchy'):
        bound = ([0,np.nanmin(grid_x), np.nanmin(grid_y), 0, -1], 
                         [100,np.nanmax(grid_x), np.nanmax(grid_y), 1000, 100])
    nr_neurons = len(mean_matrix[0])

    #Percentage indicator
    tf_cal = False
    ft_cal = False
    st_cal = False
    hd_cal = False

    for i in range(nr_neurons):
        neuron_data = mean_matrix[:, i]
        grid_z = griddata((found_pairs[:,0], found_pairs[:,1]), neuron_data, (grid_x, grid_y), method='linear')
        gz = grid_z.ravel()
        total_spikes = np.sum(neuron_data)
        
        x_mean = np.sum((neuron_data * found_pairs[:,0]))/total_spikes
        y_mean = np.sum(neuron_data * found_pairs[:,1])/total_spikes
        
        N = np.count_nonzero(neuron_data)
        if(N == 0): print(f'N equal to 0 at {i}')
        if(N == 1): 
            print(f'N equal to 1 at {i}')
            x_std = np.sqrt(np.sum(neuron_data * (found_pairs[:, 0] - x_mean)**2) / (((N)*total_spikes)/N))
            y_std = np.sqrt(np.sum(neuron_data * (found_pairs[:, 1] - y_mean)**2) / (((N)*total_spikes)/N))
        else:
            x_std = np.sqrt(np.sum(neuron_data * (found_pairs[:, 0] - x_mean)**2) / (((N-1)*total_spikes)/N))
            y_std = np.sqrt(np.sum(neuron_data * (found_pairs[:, 1] - y_mean)**2) / (((N-1)*total_spikes)/N))

        mean_std = np.mean([x_std, y_std])

        if(model == 'Gaussian'):
            initial_guess = (np.nanmax(grid_z),x_mean,y_mean,x_std,y_std,0,np.nanmin(grid_z))
    
            try:
                popt, pcov = curve_fit(ex_twoD_Gaussian, (grid_x.ravel(), grid_y.ravel()), gz.ravel(), 
                               p0=initial_guess, maxfev=10000, bounds=bound, nan_policy='omit')
            except Exception as e:
                popt = initial_guess
                print(f"-> Fitting failed: {e}. Returning initial guess.")
        elif(model == 'Cauchy'):
            initial_guess = (np.nanmax(grid_z),x_mean,y_mean,mean_std,np.nanmin(grid_z))
    
            try:
                popt, pcov = curve_fit(cauchy_surface, (grid_x.ravel(), grid_y.ravel()), gz.ravel(), 
                               p0=initial_guess, maxfev=100000, bounds=bound, nan_policy='omit')
            except Exception as e:
                popt = initial_guess
                print(f"-> Fitting failed: {e}. Returning initial guess.")
            
        if(np.round((i+1)/nr_neurons, 2) == 0.25 and not tf_cal): 
            print('25% Fitted')
            tf_cal = True
        elif(np.round((i+1)/nr_neurons, 2) == 0.5 and not ft_cal): 
            print('50% Fitted')
            ft_cal = True
        elif(np.round((i+1)/nr_neurons, 2) == 0.75 and not st_cal): 
            print('75% Fitted')
            st_cal = True
        elif(np.round((i+1)/nr_neurons, 2) == 1 and not hd_cal): 
            print('100% Fitted')
            hd_cal = True
        '''
        try:
            popt, pcov = curve_fit(twoD_Gaussian, (grid_x, grid_y), gz, p0=initial_guess, maxfev=2000)
        except RuntimeError or popt[0] > 200:
            print(f"Unbounded fit failed for neuron {i}, trying bounded fit.")
            popt, pcov = curve_fit(twoD_Gaussian, (grid_x, grid_y), gz, p0=initial_guess, maxfev=2000, bounds=bound)
        if(popt[0] > 100):
            print(f'Wamp Wamp, wrong dimension found for amplitude at Neuron {i}. You got {np.round(popt[0],2)} for an aplitude')
            popt, pcov = curve_fit(twoD_Gaussian, (grid_x, grid_y), gz, p0=initial_guess, maxfev=2000, bounds=bound)
        '''
        if(popt[-1] < 0): 
            popt = list(popt)
            popt[-1] = 0 #Avoids negative values
            popt = tuple(popt)
        
        storage_gridZ.append(grid_z)
        predicted.append(popt)
        
    storage_gridZ = np.asarray(storage_gridZ, dtype='float64')
    predicted = np.asarray(predicted, dtype='float64')
    grids = [grid_x, grid_y] 
    observed = storage_gridZ
    
    return mean_matrix, found_pairs, velocity_vector, grids, observed, predicted

In [ ]:
'''
Gaussian surface generator (used to fit the data)
'''
def twoD_Gaussian(xy, param):
    amplitude, xo, yo, sigma_x, sigma_y, theta, offset = param
    x, y = xy
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

In [ ]:
'''
Gaussian surface generator (used to fit the data)
'''
def ex_twoD_Gaussian(xy, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    x, y = xy
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

In [ ]:
'''
Gaussian surface generator (used to fit the data)
'''
def ex_twoD_Gaussian2(xy, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    x, y = xy
    xo = float(xo)
    yo = float(yo)

    a = (np.cos(theta)**2) / (2 * sigma_x**2) + (np.sin(theta)**2) / (2 * sigma_y**2)
    b = -(np.sin(2 * theta)) / (4 * sigma_x**2) + (np.sin(2 * theta)) / (4 * sigma_y**2)
    c = (np.sin(theta)**2) / (2 * sigma_x**2) + (np.cos(theta)**2) / (2 * sigma_y**2)

    g = offset + amplitude * np.exp(- (a * ((x - xo)**2) + 2 * b * (x - xo) * (y - yo) + c * ((y - yo)**2)))
    return g

In [ ]:
'''
Vectorized two_gaussian. This works as well for single input. Migh depecrate the other version
'''
def vect_twoD_Gaussian(xy, param):
    amplitude, xo, yo, sigma_x, sigma_y, theta, offset = param
    x, y = xy
    xo = float(xo)
    yo = float(yo)
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude * np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) + c*((y-yo)**2)))
    return g

In [ ]:
'''
Vectorized two_gaussian. This works as well for single input. Migh depecrate the other version -> 2
'''
def vect2_twoD_Gaussian(xy, params):
    x, y = xy
    amplitude, xo, yo, sigma_x, sigma_y, theta, offset = params
    #xo = float(xo)
    #yo = float(yo)
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude * np.exp(- (a * ((x - xo)**2) + 2 * b * (x - xo) * (y - yo) + c * ((y - yo)**2)))
    return g

#### Bayes Decoder

In [ ]:
'''
Returns all components needed for the Simple Naive Bayses Decoder
'''
def SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, model='Gaussian', spike_range=None, bin_size=5, window_size=35):
    #Unpacking data
    tau = bin_size * window_size * 0.001
    probability_s = all_prob_output(found_pairs, velocity_vector)
    rate_data = np.array(train_dataset['spikes'])
    spike_data = np.asarray(rate_data * tau, dtype='int')
    if(spike_range == None): spike_range = np.max(spike_data) + 1
    nr_neurons = len(spike_data[0])
    
    #Gets all neuron probability to all velocities for all spikes count in the train
    all_conditional = []
    for i, pair in enumerate(found_pairs):
        if(model == 'Gaussian'): fs = vect2_twoD_Gaussian(([found_pairs[i,0]], [found_pairs[i,1]]), predicted.T)
        elif(model == 'Cauchy'): fs = cauchy_surface(([found_pairs[i,0]], [found_pairs[i,1]]), *predicted.T)
        all_store = []
        for j in range(nr_neurons):
            f = fs[j] * tau
            all_prob = []
            for k in range(spike_range):
                probability = (np.exp(-f)*np.power(f, k))/math.factorial(k)
                all_prob.append(probability)
            all_store.append(all_prob)
        all_conditional.append(all_store)
    all_conditional = np.array(all_conditional) #found_pair, neuron, spike_count
    return all_conditional, probability_s, found_pairs

In [ ]:
'''
Applies the Simple Naive Bayes Decoder
'''
def SNB_apply(dataset, decoder, bin_size=5, window_size=35):
    #Data Extraction
    all_conditional, probability_s, found_pairs = decoder
    tau = bin_size * window_size * 0.001
    rate_data = np.array(dataset['spikes'])
    spike_data = np.asarray(rate_data * tau, dtype='int')

    #Loop Apply - Tries all s for each entry and returns the most likely -> Can be optimized (parallelization)
    max_prob = []
    directions = []
    i_found_pairs = np.arange(len(found_pairs))
    i_neurons = np.arange(len(spike_data[0]))
    for k in range(len(spike_data)): #Maybe can be optimized
        spikes = spike_data[k]
        values = all_conditional[i_found_pairs[:, np.newaxis], i_neurons, spikes]
        conditional_vector = np.prod(values, axis=1)
        probability_vector = conditional_vector * probability_s
        direction_loc = np.argmax(probability_vector)
        max_prob.append(probability_vector[direction_loc])
        directions.append(found_pairs[direction_loc])
    max_prob = np.array(max_prob)
    directions = np.array(directions)

    return directions, max_prob

In [ ]:
'''
Removed Applies the Simple Naive Bayes Decoder
'''
def RNB_apply(dataset, decoder, bin_size=5, window_size=35):
    #Data Extraction
    all_conditional, probability_s, found_pairs = decoder
    tau = bin_size * window_size * 0.001
    rate_data = np.array(dataset['spikes'])
    spike_data = np.asarray(rate_data * tau, dtype='int')

    #Loop Apply - Tries all s for each entry and returns the most likely -> Can be optimized (parallelization)
    max_prob = []
    directions = []
    i_found_pairs = np.arange(len(found_pairs))
    i_neurons = np.arange(len(spike_data[0]))
    for k in range(len(spike_data)): #Maybe can be optimized
        spikes = spike_data[k]
        values = all_conditional[i_found_pairs[:, np.newaxis], i_neurons, spikes]
        conditional_vector = np.prod(values, axis=1)
        probability_vector = conditional_vector
        direction_loc = np.argmax(probability_vector)
        max_prob.append(probability_vector[direction_loc])
        directions.append(found_pairs[direction_loc])
    max_prob = np.array(max_prob)
    directions = np.array(directions)

    return directions, max_prob

In [ ]:
'''
Returns all components needed for the Naive Bayses Decoder
'''
def NB_decoder(train_dataset, found_pairs, velocity_vector, predicted, model='Gaussian', spike_range=None, bin_size=5, window_size=35):
    #Unpacking data
    tau = bin_size * window_size * 0.001
    probability_s = all_prob_output(found_pairs, velocity_vector)
    rate_data = np.array(train_dataset['spikes'])
    spike_data = np.asarray(rate_data * tau, dtype='int')
    if(spike_range == None): spike_range = np.max(spike_data)
    nr_neurons = len(spike_data[0])
    
    #Gets all neuron probability to all velocities for all spikes count in the train
    all_conditional = []
    for i, pair in enumerate(found_pairs):
        if(model == 'Gaussian'): fs = vect2_twoD_Gaussian(([found_pairs[i,0]], [found_pairs[i,1]]), predicted.T)
        elif(model == 'Cauchy'): fs = cauchy_surface(([found_pairs[i,0]], [found_pairs[i,1]]), *predicted.T)
        all_store = []
        for j in range(nr_neurons):
            f = fs[j] * tau
            all_prob = []
            for k in range(spike_range):
                probability = (np.exp(-f)*np.power(f, k))/math.factorial(k)
                all_prob.append(probability)
            all_store.append(all_prob)
        all_conditional.append(all_store)
    all_conditional = np.array(all_conditional) #found_pair, neuron, spike_count

    #Gets the distance probability matrix
    std = get_train_std(train_dataset)
    probability_dist = prob_distances(found_pairs, std*50)
    return all_conditional, probability_s, probability_dist, found_pairs

In [ ]:
'''
Applies the Naive Bayes Decoder
'''
def NB_apply(dataset, decoder, bin_size=5, window_size=35):
    #Data Extraction
    all_conditional, probability_s, probability_dist, found_pairs = decoder
    tau = bin_size * window_size * 0.001
    rate_data = np.array(dataset['spikes'])
    spike_data = np.asarray(rate_data * tau, dtype='int')

    #Gets all trial changes boolean
    _, first_occurrences = np.unique(trial_ids, return_index=True)
    trial_change = np.zeros_like(trial_ids, dtype=bool)
    trial_change[first_occurrences] = True
    
    #Loop Apply - Tries all s for each entry and returns the most likely -> Can be optimized (parallelization)
    max_prob = []
    directions = []
    i_found_pairs = np.arange(len(found_pairs))
    i_neurons = np.arange(len(spike_data[0]))
    
    for k in range(len(spike_data)):
        spikes = spike_data[k]
        values = all_conditional[i_found_pairs[:, np.newaxis], i_neurons, spikes]
        conditional_vector = np.prod(values, axis=1)
        if(trial_change[k]): probability_vector = conditional_vector #Firs entry of each trial should not check past
        else: 
            previos_loc = directions[-1]
            loc_index = np.where(np.all(found_pairs == directions[-1], axis=1))[0][0]
            probability_vector = conditional_vector * probability_dist[loc_index]
        direction_loc = np.argmax(probability_vector)
        max_prob.append(probability_vector[direction_loc])
        directions.append(found_pairs[direction_loc])
    
    max_prob = np.array(max_prob)
    directions = np.array(directions)

    return directions, max_prob

#### Naive Bayses Functions

In [ ]:
'''
Calculates the P(s) -> Probability Distribution of Output variables
'''
def prob_output(s, found_pairs, velocity_vector):
    #Binarize the input into the found_pairs binarization
    distances = np.linalg.norm(found_pairs - s, axis=1)
    position = np.argmin(distances)
    s = found_pairs[position]
    
    #Count the number of times the found bin appears in velocity_vector
    s_count = np.sum((velocity_vector[:,0] == s[0]) & (velocity_vector[:,1] == s[1]))
    
    #Divide by len of velocity_vector
    size = len(velocity_vector)
    probability = s_count / size
    return probability

In [ ]:
'''
Calculates the fi(s) of the ith neuron -> Fitted tuning curve value extraction
'''
def tune_value(predicted, neuron, s):
    parameters = predicted[neuron]
    f = twoD_Gaussian((s[0], s[1]), parameters)
    return f

In [ ]:
'''
Calculates the P(ri|s) for i neuron -> Probability of a given rate given a certain velocity state based on a 2D Poisson
'''
def poisson(predicted, neuron, s, spike_count, tau, bin_size=5, window_size=35, print_info=False):
    if not isinstance(spike_count, np.integer):
        print('Not integer, going to round')
        spike_count = np.round(spike_count)
    f = tune_value(predicted, neuron, s) * tau
    probability = (np.exp(-f)*np.power(f, spike_count))/math.factorial(spike_count)
    if(print_info):
        print(f'n = {spike_count}')
        print(f'f(s) = {f}')
        print(f'P(n|s) = {probability}')
        print()
    return probability[0]

In [ ]:
'''
Gets the full P(r|s) by product -> Independent neurons
'''
def independent_conditional(predicted, s, spike_vector, print_info=False, bin_size=5, window_size=35):
    nr_neurons = len(predicted)
    product = 1
    tau = bin_size*window_size*0.001
    for i in range(nr_neurons):
        if(print_info): print(f'Neuron nº {i}')
        value = poisson(predicted, i, s, spike_vector[i], tau, print_info=print_info)
        product *= value
    return product

In [ ]:
'''
Generates all the P(s) for all possible states --- Optimize
'''
def all_prob_output(found_pairs, velocity_vector):
    nr_pairs = len(found_pairs)
    probability_vector = []
    for i in range(nr_pairs):
        s = found_pairs[i]
        probability_vector.append(prob_output(s, found_pairs, velocity_vector))
    probability_vector = np.array(probability_vector)
    
    ''' Find why doesnt work
    nr_pairs = len(found_pairs)
    probability_vector = np.where(found_pairs == found_pairs, prob_output(found_pairs, found_pairs, velocity_vector), 'NA')[:,0]
    probability_vector = np.asarray(probability_vector, dtype='float64')
    '''
    return probability_vector

In [ ]:
'''
Fits a gaussian to the distances and visualizes them
'''
def prob_distances(found_pairs, std, plot=None, cmap='hot', dist_alpha=0, prob_alpha=1, save=False, save_name='plot.png'):
    #Get the distances between each pair in a square matrix (found_pairs x found_pairs)
    general_dists = squareform(pdist(found_pairs))
    extra_prob = norm.pdf(general_dists, 0, std)
    
    #Ploting if not None
    if(plot == '3D'):
        fig = plt.figure(figsize=(20,20))
        ax = fig.add_subplot(projection='3d')
        size = len(found_pairs)
        grid_x, grid_y = np.mgrid[0:size:size*1j, 0:size:size*1j]
        if(dist_alpha != 0):
            ax.plot_surface(grid_x, grid_y, general_dists, cmap=cmap, edgecolor='none', alpha=dist_alpha)
        ax.plot_surface(grid_x, grid_y, extra_prob, cmap=cmap, edgecolor='none', alpha=prob_alpha)
        ax.set_xlabel('Found_Pairs')
        ax.set_ylabel('Found_Pairs - 1')
        ax.set_zlabel('Distances')
        
        if(view != None):
            ax.view_init(view[0], view[1], view[2])
    
        plt.title('Distances', fontsize=20)
        if(save): plt.savefig(save_name)
        plt.show()
        
    elif(plot == 'heat'):
        # Create a figure with two subplots
        fig, axs = plt.subplots(1, 2, figsize=(20, 8))
        
        # Plot the first heatmap on the left subplot
        im1 = axs[0].imshow(extra_prob.T, cmap=cmap, origin='lower')
        cb_max1 = np.nanmax(extra_prob)
        cb_min1 = np.nanmin(extra_prob)
        cb_range1 = np.linspace(cb_min1, cb_max1, 10)
        fig.colorbar(im1, ax=axs[0], ticks=cb_range1, label='P(s|s-1)')
        axs[0].set_title('Heat Map of Distance Probability', fontsize=20)
        axs[0].set_xlabel('Found Pairs')
        axs[0].set_ylabel('Found Pairs - 1')
        
        # Plot the second heatmap on the right subplot
        im2 = axs[1].imshow(general_dists.T, cmap=cmap, origin='lower', extent=(0,1790,0,1790))  
        cb_max2 = np.nanmax(general_dists)  
        cb_min2 = np.nanmin(general_dists)  
        cb_range2 = np.linspace(cb_min2, cb_max2, 10)  
        fig.colorbar(im2, ax=axs[1], ticks=cb_range2, label='Distance')  
        axs[1].set_title('Heat Map of Distances', fontsize=20)
        axs[1].set_xlabel('Found Pairs')
        axs[1].set_ylabel('Found Pairs - 1')
    
        #Save plot
        if save:
            plt.savefig(save_name)
        plt.show()
    return extra_prob

In [ ]:
'''
Gets the distances (euclidean) between all training dataset velocities and their std
'''
def get_train_std(train_dataset, get_distance=False):
    #Data extraction
    velocities = np.array(train_dataset['hand_vel'])
    delta = velocities[1:] - velocities[:-1]
    distances = np.sqrt(np.sum(delta**2, axis=1))
    std = np.std(distances)

    if(get_distance): return std, distances
    else: return std

#### Analysis

In [ ]:
'''
Converts the predicted velocities into coordinates
'''
def convert_vel2pos(dataset, bin_size=5, save_pandas=True):
    #Data Extraction
    trials = trials_present(dataset)
    all_trials = np.array(dataset['trial_id'])
    pred_vel_x = np.array(dataset['pred_vel_X'])
    pred_vel_y = np.array(dataset['pred_vel_Y'])
    pos_x = np.array(dataset['cursor_pos']['x'])
    pos_y = np.array(dataset['cursor_pos']['y'])
    
    #Data Preparation
    single_duration = bin_size * 0.001
    trial_mask = np.insert(all_trials[1:] != all_trials[:-1], 0, True)
    pred_pos_x = np.zeros_like(pred_vel_x)
    pred_pos_y = np.zeros_like(pred_vel_y)
    
    #Position Calculation
    pred_pos_x = np.where(trial_mask, pos_x, 0)
    pred_pos_y = np.where(trial_mask, pos_y, 0)
    for i in range(len(pred_pos_x)): # Optimize this loop
        if(not trial_mask[i]):
            position_x = pred_pos_x[i-1]
            position_y = pred_pos_y[i-1]
            velocity_x = pred_vel_x[i-1]
            velocity_y = pred_vel_y[i-1]
            new_x = position_x + velocity_x * single_duration
            new_y = position_y + velocity_y * single_duration
            pred_pos_x[i] = new_x
            pred_pos_y[i] = new_y
    pred_pos = np.stack((pred_pos_x, pred_pos_y), axis=-1)
    
    #Pandas Saving (Optional)
    if(save_pandas):
        apply_2D_data(dataset, pred_pos, col_name='pred')
    else: return pred_pos

In [ ]:
'''
Returns the mean correlation
'''
def mean_corr(dataset, metric, correlation, time_window=150, bin_size=5):
    bin_window = time_window//bin_size
    
    actual_vector = np.asarray(dataset[metric[0]], dtype='float64')
    pred_vector = np.asarray(dataset[metric[1]], dtype='float64')
    trial_id = np.asarray(dataset['trial_id'], dtype='int64')
    trials = np.unique(trial_id)
    nr_trials = len(trials)
    mask = np.concatenate(([True], trial_id[1:] != trial_id[:-1]))
    actual_vector = np.split(actual_vector, np.where(mask)[0])[1:]
    pred_vector = np.split(pred_vector, np.where(mask)[0])[1:]
    
    cross_corr_vector = []
    lags = np.arange(-time_window + 1, time_window, bin_size)
    for i in range(nr_trials):
        if(correlation == 'pearson'):
            cross_corr = cross_pearson(actual_vector[i], pred_vector[i])
        elif(correlation == 'cross'):
            k = np.sqrt((np.sum(np.power(actual_vector[i],2)))*(np.sum(np.power(pred_vector[i],2))))
            cross_corr = np.correlate(actual_vector[i], pred_vector[i])/k
        cross_corr = cross_corr[len(cross_corr)//2-bin_window:len(cross_corr)//2+bin_window]
        cross_corr_vector += [np.array(cross_corr)]
    
    cross_corr_vector = np.array(cross_corr_vector)
    mean_correlation = np.mean(cross_corr_vector, axis=0)
    std_corr = np.std(cross_corr_vector, axis=0)
    return mean_correlation, std_corr

#### Extras

In [ ]:
'''
Applies 2D to the pandas dataset
'''
def apply_2D_data(dataset, data, col_name='pred_vel'):
    dataset[col_name + '_X'] = data[:,0]
    dataset[col_name + '_Y'] = data[:,1]

In [ ]:
'''
Removes the neurons according to a certain threshold
'''
def remove_neurons(datasets, neurons, predicted, method, min_depth=None, max_std=None, remove_fromDataset=True):
    #Finding the neurons
    if(method == 'depth'):
        amplitude = predicted[:,0]
        mask = np.where(amplitude >= min_depth, True, False)
    elif(method == 'std'):
        x_std = np.array(predicted[:,3])
        y_std = np.array(predicted[:,4])
        mean_std = (x_std + y_std)/2
        mask = np.where(mean_std <= max_std, True, False)
    elif(method == 'depth&std'):
        amplitude = predicted[:,0]
        x_std = np.array(predicted[:,3])
        y_std = np.array(predicted[:,4])
        mean_std = (x_std + y_std)/2
        
        mask1 = np.where(amplitude >= min_depth, True, False)
        mask2 = np.where(mean_std <= max_std, True, False)
        mask = np.logical_and(mask1, mask2)

    #Removing the neurons
    neurons = np.array(neurons)
    removed_neurons = neurons[~mask]
    acepted_neurons = neurons[mask]
    acepted_predicted = predicted[mask]
    
    if(remove_fromDataset):
        for j, dataset in enumerate(datasets):
            col_toRemove = np.empty((len(removed_neurons)), dtype=tuple)
            for i, neuron in enumerate(removed_neurons):
                col_toRemove[i] = ('spikes', neuron)
            dataset.drop(columns=col_toRemove, inplace=True)
    return acepted_neurons, acepted_predicted

#### Visualize Tuning Curves

In [ ]:
'''
Visualizes the 3d tunning curve (heat map) of a given neuron
'''
def plot_Heatmap_single(mean_matrix, found_pairs, neuron, grids, observed, predicted, cmap='hot', fill_nan=False, plot_predicted=True):
    minimum = np.min(found_pairs)
    maximum = np.max(found_pairs)
    resolution = len(observed[0])
    
    grid_x, grid_y = grids
    grid_z = observed[neuron]
    popt = predicted[neuron]
    data_fitted = twoD_Gaussian((grid_x, grid_y), popt)
    
    if(fill_nan): grid_z = np.nan_to_num(grid_z)
    max_locationF = np.nanargmax(grid_z)
    max_location = np.unravel_index(max_locationF, grid_z.shape)
    
    plt.figure(figsize=(20,15))
    im = plt.imshow(grid_z.T, cmap=cmap, origin='lower', extent=(minimum, maximum, minimum, maximum))
    plt.scatter(grid_x[max_location], grid_y[max_location], c='white', linewidths=50, marker='o', alpha=0.7,
               label=f'X: {np.round(grid_x[max_location],0)}\nY: {np.round(grid_y[max_location],0)}')
    if(plot_predicted): plt.contour(grid_x, grid_y, data_fitted.reshape(resolution, resolution), 8, colors='w')

    cb_max = np.nanmax(grid_z)
    cb_range = np.linspace(0, cb_max, 10)
    plt.colorbar(im, ticks=cb_range, label='Firing Rate')
    plt.title('Heat Map of Neuron Firing Rates')
    plt.xlabel('X Velocity')
    plt.ylabel('Y Velocity')
    #plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

In [ ]:
'''
Visualizes all the tuning curves 3D (heat map)
'''
def plot_heatmap_all(mean_matrix, neurons, found_pairs, grids, observed, predicted, nr_cols=5, model='Gaussian', uniform=False, 
                     cmap='hot', fill_nan=False, save=False, save_name='plot.png'):
    max_rate = np.max(mean_matrix)
    if(len(mean_matrix[0])%nr_cols == 0): nr_rows = len(mean_matrix[0])//nr_cols
    else: nr_rows = len(mean_matrix[0])//nr_cols + 1
        
    nr_neurons = len(neurons)
    minimum = np.min(found_pairs)
    maximum = np.max(found_pairs)
    grid_x, grid_y = grids
    resolution = len(observed[0])
    
    # Create a figure and subplots
    fig, axs = plt.subplots(nrows=nr_rows, ncols=nr_cols, figsize=(nr_cols*5, nr_rows*5))
    
    # Flatten the axs array for easier iteration
    axs = axs.flatten()
    
    # Plot each array
    for i in range(nr_neurons):
        ax = axs[i]
        grid_z = observed[i]
        if(fill_nan): grid_z = np.nan_to_num(grid_z)
        if(uniform): im = ax.imshow(grid_z.T, cmap=cmap, origin='lower', extent=(minimum, maximum, minimum, maximum), vmin=0, vmax=max_rate)
        else: im = ax.imshow(grid_z.T, cmap=cmap, origin='lower', extent=(minimum, maximum, minimum, maximum))

        popt = predicted[i]
        if(model == 'Gaussian'): data_fitted = twoD_Gaussian((grid_x, grid_y), popt)
        elif(model == 'Cauchy'): data_fitted = cauchy_surface((grid_x, grid_y), *popt)
        ax.contour(grid_x, grid_y, data_fitted.reshape(resolution, resolution), 8, colors='#18E789')
        
        cb_max = np.nanmax(grid_z)
        max_locationF = np.nanargmax(grid_z)
        max_location = np.unravel_index(max_locationF, grid_z.shape)
        ax.scatter(grid_x[max_location], grid_y[max_location], c='#18E789', linewidths=5, marker='o', alpha=0.7,
                   label=f'X: {np.round(grid_x[max_location],0)}\nY: {np.round(grid_y[max_location],0)}\nPredicted Peak: {np.round(popt[0], 2)}')
        
        ax.set_title(f'Neuron {neurons[i]}')
        ax.set_xlabel('X Velocity')
        ax.set_ylabel('Y Velocity')
    
        cb_range = np.linspace(0, cb_max, 5)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical', ticks=cb_range)
        ax.legend()
        
    #for ax in fig.get_axes():
    #    ax.label_outer()
    
    
    plt.suptitle(f'Tuning Curves\n in a {resolution}x{resolution} grid ', y=0.999, fontsize=20)
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    plt.show()

In [ ]:
'''
Plots the 3d representation of the tunning curve of  agiven neuron
'''
def plot_tune3D_single(mean_matrix, found_pairs, neuron, grids, observed, predicted, model='Gaussian',
                       view=None, fill_nan=False, view_points=False, observed_alpha=0.05, predicted_alpha=1):
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(projection='3d')
    length = len(np.unique(found_pairs[:,0]))
    neuron_data = mean_matrix[:,neuron]
    parameters = predicted[neuron]
    resolution = len(observed[0])
    
    grid_x, grid_y = grids
    grid_z = observed[neuron]
    if(model == 'Gaussian'): predicted_data = twoD_Gaussian((grid_x, grid_y), parameters)
    elif(model == 'Cauchy'): predicted_data = cauchy_surface((grid_x, grid_y), *parameters)
    if(fill_nan): grid_z = np.nan_to_num(grid_z)
        
    if(view_points): ax.scatter(found_pairs[:,0], found_pairs[:,1], mean_matrix[:,0], marker='o', linewidths=1.5, alpha=0.5)
    ax.plot_surface(grid_x, grid_y, predicted_data.reshape(resolution, resolution), cmap='viridis', edgecolor='none', alpha=predicted_alpha)
    ax.plot_surface(grid_x, grid_y, grid_z, cmap='hot', edgecolor='none', alpha=observed_alpha)
    ax.set_xlabel('Velocity X')
    ax.set_ylabel('Velocity Y')
    ax.set_zlabel('Firing Rate')
    
    if(view != None):
        ax.view_init(view[0], view[1], view[2])
    plt.show()

In [ ]:
'''
View all predicted and observed tuning curves in 3D
'''
def plot_tune3D_all(mean_matrix, neurons, found_pairs, grids, observed, predicted, nr_cols=5, uniform=False, cmap='viridis', fill_nan=False, 
                     observed_alpha=0.05, predicted_alpha=1, model='Gaussian', save=False, save_name='plot.png'):
    #Size calculations for subplot
    max_rate = np.max(mean_matrix)
    if(len(mean_matrix[0])%nr_cols == 0): nr_rows = len(mean_matrix[0])//nr_cols
    else: nr_rows = len(mean_matrix[0])//nr_cols + 1
    
    #Parameters extraction
    nr_neurons = len(neurons)
    minimum = np.min(found_pairs)
    maximum = np.max(found_pairs)
    grid_x, grid_y = grids
    resolution = len(observed[0])
    
    # Create a figure and subplots
    fig = plt.figure(figsize=(nr_cols*4, nr_rows*4))
    
    # Flatten the axs array for easier iteration
    #axs = axs.flatten()
    
    # Plot each array
    for i in range(nr_neurons):
        ax = fig.add_subplot(nr_rows, nr_cols, i+1, projection='3d')
        grid_z = observed[i]
        if(fill_nan): grid_z = np.nan_to_num(grid_z)
            
        if(uniform): ax.plot_surface(grid_x, grid_y, grid_z, cmap='hot', edgecolor='none', alpha=observed_alpha, vmax=max_rated)
        else: ax.plot_surface(grid_x, grid_y, grid_z, cmap='hot', edgecolor='none', alpha=observed_alpha)
    
        popt = predicted[i]
        if(model == 'Gaussian'): predicted_data = twoD_Gaussian((grid_x, grid_y), popt)
        elif(model == 'Cauchy'): predicted_data = cauchy_surface((grid_x, grid_y), *popt)
        if(uniform): ax.plot_surface(grid_x, grid_y, predicted_data.reshape(resolution, resolution), cmap=cmap, edgecolor='none', 
                                     alpha=predicted_alpha, vmax=max_rate)
        else: ax.plot_surface(grid_x, grid_y, predicted_data.reshape(resolution, resolution), cmap=cmap, edgecolor='none', 
                                     alpha=predicted_alpha)
        
        ax.set_title(f'Neuron {neurons[i]}')
        ax.set_xlabel('Velocity X')
        ax.set_ylabel('Velocity Y')
        ax.set_zlabel('Firing Rate')
    
    plt.suptitle(f'Tuning Curves\n in a {resolution}x{resolution} 3D grid ', y=0.999, fontsize=20)
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    plt.show()

#### Visualize Probabilities

In [ ]:
'''
Visualizes the P(s) for all possible states
'''
def plot_prob_output(probability_vector, found_pairs, grids, fill_nan=False, cmap='hot', logged=True, save=False, save_name='plot.png'):
    #Log or not log values
    if(logged): logged_prob = np.log(probability_vector)
    else: logged_prob = probability_vector
        
    #Unpack Input
    grid_x, grid_y = grids
    grid_probability = griddata((found_pairs[:,0], found_pairs[:,1]), logged_prob, (grid_x, grid_y), method='linear')
    minimum = np.min(found_pairs)
    maximum = np.max(found_pairs)
    resolution = len(grid_x)
    if(fill_nan): grid_probability = np.nan_to_num(grid_probability, nan=np.nanmin(logged_prob))
    max_locationF = np.nanargmax(grid_probability)
    max_location = np.unravel_index(max_locationF, grid_probability.shape)

    #Plotting General
    plt.figure(figsize=(20,15))
    plt.scatter(grid_x[max_location], grid_y[max_location], c='lightgreen', linewidths=1.5, marker='o', alpha=1,
               label=f'X: {np.round(grid_x[max_location],0)}\nY: {np.round(grid_y[max_location],0)}')
    im = plt.imshow(grid_probability.T, cmap=cmap, origin='lower', extent=(minimum, maximum, minimum, maximum))

    #Plotting Colorbar
    cb_max = np.nanmax(grid_probability)
    cb_min = np.nanmin(grid_probability)
    cb_range = np.linspace(cb_min, cb_max, 10)
    if(logged): plt.colorbar(im, ticks=cb_range, label='Log P(s)')
    else: plt.colorbar(im, ticks=cb_range, label='P(s)')

    #Plotting Titles, etc
    if(logged): plt.title('Heat Map of Log P(s)', fontsize=20)
    else: plt.title('Heat Map of P(s)', fontsize=20)
    plt.xlabel('X Velocity')
    plt.ylabel('Y Velocity')
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    
    #Plotting Saving
    if(save): plt.savefig(save_name)
    plt.show()

In [ ]:
'''
Visualizes the P(s) for all possible states in 3D
'''
def plot_prob_output3D(probability_vector, found_pairs, grids, fill_nan=True, cmap='hot', logged=True, view_points=False,
                       main_alpha=1, dots_alpha=0.5, save=False, save_name='plot.png'):
    #Log or not log values
    if(logged): logged_prob = np.log(probability_vector)
    else: logged_prob = probability_vector
    
    grid_x, grid_y = grids
    grid_probability = griddata((found_pairs[:,0], found_pairs[:,1]), logged_prob, (grid_x, grid_y), method='linear')
    minimum = np.min(found_pairs)
    maximum = np.max(found_pairs)
    resolution = len(grid_x)
    if(fill_nan): grid_probability = np.nan_to_num(grid_probability, nan=np.nanmin(logged_prob))
    
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(projection='3d')
    

        
    if(view_points): ax.scatter(found_pairs[:,0], found_pairs[:,1], logged_prob, marker='o', linewidths=1.5, alpha=dots_alpha)
    ax.plot_surface(grid_x, grid_y, grid_probability, cmap=cmap, edgecolor='none', alpha=main_alpha)
    ax.set_xlabel('Velocity X')
    ax.set_ylabel('Velocity Y')
    if(logged): ax.set_zlabel('Log P(s)')
    else: ax.set_zlabel('P(s)')
    
    if(view != None):
        ax.view_init(view[0], view[1], view[2])

    if(logged): plt.title('3D Plot of Log P(s)', fontsize=20)
    else: plt.title('3D Plot of P(s)', fontsize=20)

    if(save): plt.savefig(save_name)
    plt.show()

In [ ]:
'''
Visualizes the spike count distribution
'''
def plot_spike_count(dataset, bin_size=5, window_size=35, save=False, save_name='plot.png'):
    #Data Extraction ≈ 2sec
    rate_data = np.array(dataset['spikes'])
    tau = bin_size * window_size * 0.001
    spike_data = np.asarray(rate_data * tau, dtype='int')
    unique_spikes, counts = np.unique(spike_data, return_counts=True)
    
    #Data Visualization
    plt.figure(figsize=(10,5))
    plt.bar(unique_spikes, counts, width=0.6, color='white', edgecolor='black')
    
    plt.xlabel('Spike Counts')
    plt.ylabel('Histogram')
    plt.title('Histogram of Spike Counts in the Dataset')
    if(save): plt.savefig(save_name)
    plt.show()

### Crafting Table

In [ ]:
%time train_sets, val_sets, test_dataset = generate_sets(standardDS, 0.1, 1)

train_dataset = train_sets[0]
val_dataset = val_sets[0]

neurons = standard_neurons

#Converting sets to firing rate -> Otimization Needed!!!
%time firing_rate(val_dataset, 35, panda=True) 
%time firing_rate(train_dataset, 35, panda=True) 
%time firing_rate(test_dataset, 35, panda=True) 

In [ ]:
#Fitting Gaussian Curves to Dataset
%time mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 50)

In [ ]:
%time decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)

In [ ]:
%time pred_vel, max_prob = SNB_apply(val_dataset, decoder)

In [ ]:
%time apply_2D_data(val_dataset, pred_vel)
%time apply_2D_data(val_dataset, np.array(val_dataset['hand_vel']), col_name='vel')

In [ ]:
%time convert_vel2pos(val_dataset)

In [ ]:
(len(test_dataset)*0.01)/(60)

In [ ]:
datasets = [train_dataset, val_dataset, test_dataset]
%time acepted_neurons, acepted_predicted = remove_neurons(datasets, neurons, predicted, 'depth&std', min_depth=1, max_std=700)

In [ ]:
%time get_train_std(train_dataset)

In [ ]:
%time prob_distances(found_pairs, get_train_std(train_dataset))

In [ ]:
%time decoder = NB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)

In [ ]:
correlation_matrix = std_corr_Y
deviation = std_std_Y
lag = lags
model_names = std_thresholds
title_name = 'Deviation Threshold Hacking (VelY)'
save = False
save_name = 'std_thr_hack_velY.png'

start_time = time.time()
'''
Visualizes the different models preformance
'''
#Data extraction -> then add error
nr_models = len(correlation_matrix)
max_max = []
mm_std = []
max_lag = []
zero_max = []
zm_std = []
lag_len = len(lag)
for i in range(nr_models):
    max_max.append(np.max(correlation_matrix[i]))
    max_lag.append(lag[np.argmax(correlation_matrix[i])])
    mm_std.append(deviation[i][np.argmax(correlation_matrix[i])])
    zero_max.append(correlation_matrix[i][int(lag_len/2)])
    zm_std.append(deviation[i][int(lag_len/2)])
model_names = np.asarray(model_names, dtype='int')
model_names = np.asarray(model_names, dtype='str')
model_names = np.insert(model_names, 0, 'Default')
results = {
    'Absolute Maximum': max_max,
    'Maximum at Zero': zero_max
}

#Plotting
model_x = np.arange(len(model_names))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0.5
colors = ['red', 'blue']

deviation = [mm_std, zm_std]

fig, ax = plt.subplots(figsize=(15,6), layout='constrained')

for i, (attribute, measurement) in enumerate(results.items()):
    offset = width * multiplier
    rects = ax.errorbar(model_x + offset, measurement, deviation[i], linewidth=width*5, ls='none', capsize=5, fmt='o', 
                        label=attribute, color=colors[i])
    #ax.bar_label(rects, padding=3)
    multiplier += 1


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Pearson')
ax.set_xlabel('Models')
ax.set_title(title_name)
ax.set_xticks(model_x + width, model_names)
ax.legend(loc='upper right', ncols=3)
ax.set_ylim(-1, 1)
ax.grid(True)

if(save): plt.savefig(save_name)
plt.show()

elapsed_time = round(time.time() - start_time,3)
print(f'It took {elapsed_time} sec')

In [ ]:
xo = predicted[:,1]
yo = predicted[:,2]

vector = np.column_stack((xo, yo)) / np.linalg.norm(np.column_stack((xo, yo)), axis=1)[:, np.newaxis]

plt.figure(figsize=(8, 6))
plt.quiver(np.zeros(len(xo)), np.zeros(len(yo)), vector[:,0], vector[:,1], angles='xy', scale_units='xy', scale=1, color='blue')
plt.xlim(-1, 1)
plt.ylim(-1, 1)

In [ ]:
plt.figure(figsize=(8, 6))
vector = np.column_stack((xo, yo))
plt.scatter(vector[:,0], vector[:,1], color='blue')

In [ ]:
plt.figure(figsize=(20, 16))
vector = np.column_stack((xo, yo)) / np.linalg.norm(np.column_stack((xo, yo)), axis=1)[:, np.newaxis]
plt.scatter(vector[:,0], vector[:,1], linewidths=0.1, color='blue')

In [ ]:
start_time = time.time()
'''
Divides the original training set into a train set and a validation set
'''


elapsed_time = round(time.time() - start_time,3)
print(f'It took {elapsed_time} sec')

### Results

In [ ]:
nr_neurons = len(neurons)
for i in range(nr_neurons):
    if(neurons[i] == 2401): print(i)
neurons[128]

In [ ]:
'''
Simple Bayes Decoder - Parametrization Hacking #1 (Neurons)
'''

#Separates the dataset into 3 sets
train_sets, val_sets, otest_dataset = generate_sets(standardDS, 0.1, 1)
otrain_dataset = train_sets[0]
oval_dataset = val_sets[0]
neurons = standard_neurons
print('Dataset is now divided')

#Converting sets to firing rate -> Otimization Needed!!!
firing_rate(oval_dataset, 35, panda=True) 
firing_rate(otrain_dataset, 35, panda=True) 
firing_rate(otest_dataset, 35, panda=True) 
print('Dataset is now in rates and not spike count')

#Fitting the Gaussian Tuning Curve
mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(otrain_dataset, 50)
print('All neurons have been fitted')
lags = np.arange(-150 + 1, 150, 5)

#Run Control
depth_corr_X = []
depth_corr_Y = []
depth_std_X = []
depth_std_Y = []
depth_neurons = []
std_corr_X = []
std_corr_Y = []
std_std_X = []
std_std_Y = []
std_neurons = []
print('Starting Control Model were no neurons are eliminated')
train_dataset = otrain_dataset.copy()
val_dataset = oval_dataset.copy()
test_dataset = otest_dataset.copy()
datasets = [train_dataset, val_dataset, test_dataset]

#Decoding Moment
decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
pred_vel, max_prob = SNB_apply(val_dataset, decoder)
print('Decoder Applied')
#Column Management
apply_2D_data(val_dataset, pred_vel)
apply_2D_data(val_dataset, np.array(val_dataset['hand_vel']), col_name='vel')

#Correlation calculation
mean_corrX, std_corrX = mean_corr(val_dataset, ['vel_X', 'pred_vel_X'], 'pearson')
mean_corrY, std_corrY = mean_corr(val_dataset, ['vel_Y', 'pred_vel_Y'], 'pearson')
print('Correlation Calculated')
print(f'(VelX) Maximum Pearson of {np.round(np.max(np.abs(mean_corrX)),2)} at {lags[np.argmax(np.abs(mean_corrX))]}')
print(f'(VelY) Maximum Pearson of {np.round(np.max(np.abs(mean_corrY)),2)} at {lags[np.argmax(np.abs(mean_corrY))]}')

#Stores the correlation values
depth_corr_X.append(mean_corrX)
depth_corr_Y.append(mean_corrY)
depth_std_X.append(std_corrX)
depth_std_Y.append(std_corrY)
depth_neurons.append(neurons)
std_corr_X.append(mean_corrX)
std_corr_Y.append(mean_corrY)
std_std_X.append(std_corrX)
std_std_Y.append(std_corrY)
std_neurons.append(neurons)

print()
print('-----------------------------')
start_depth = time.time()
#Removing Neurons with depth threshold
print('Trying Depth Thresholding')
depth_thresholds = np.concatenate(([0.01, 0.1 , 0.2, 0.5],np.round(np.linspace(1, 50, 25))))
for i, depth in enumerate(depth_thresholds):
    print()
    print(f'Starting Model were Neurons with depth smaller than: {depth} were eliminated')
    #Neuron Removal
    train_dataset = otrain_dataset.copy()
    val_dataset = oval_dataset.copy()
    test_dataset = otest_dataset.copy()
    
    datasets = [train_dataset, val_dataset, test_dataset]
    acepted_neurons, acepted_predicted = remove_neurons(datasets, neurons, predicted, 'depth', min_depth=depth)
    print('Neurons Removed')
    
    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, acepted_predicted, spike_range=28)
    pred_vel, max_prob = SNB_apply(val_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(val_dataset, pred_vel)
    apply_2D_data(val_dataset, np.array(val_dataset['hand_vel']), col_name='vel')

    #Correlation calculation
    mean_corrX, std_corrX = mean_corr(val_dataset, ['vel_X', 'pred_vel_X'], 'pearson')
    mean_corrY, std_corrY = mean_corr(val_dataset, ['vel_Y', 'pred_vel_Y'], 'pearson')
    print('Correlation Calculated')
    print(f'(VelX) Maximum Pearson of {np.round(np.max(np.abs(mean_corrX)),2)} at {lags[np.argmax(np.abs(mean_corrX))]}')
    print(f'(VelY) Maximum Pearson of {np.round(np.max(np.abs(mean_corrY)),2)} at {lags[np.argmax(np.abs(mean_corrY))]}')
    print(f'Number of Removed Neurons: {len(neurons) - len(acepted_neurons)}')
    
    #Stores the correlation values
    depth_corr_X.append(mean_corrX)
    depth_corr_Y.append(mean_corrY)
    depth_std_X.append(std_corrX)
    depth_std_Y.append(std_corrY)
    depth_neurons.append(acepted_neurons)
print('Depth Thresholding Done')
elapsed_depth = round(time.time() - start_depth,3)
print(f'It took {elapsed_depth} sec')
print('-------------------------')

start_std = time.time()
#Removing Neurons with std threshold
print('Trying STD Thresholding')
std_thresholds = np.round(np.linspace(300, 1000, 29))
for i, std in enumerate(std_thresholds):
    print()
    print(f'Starting Model were Neurons with mean std bigger than: {std} were eliminated')
    #Neuron Removal
    train_dataset = otrain_dataset.copy()
    val_dataset = oval_dataset.copy()
    test_dataset = otest_dataset.copy()
    datasets = [train_dataset, val_dataset, test_dataset]
    acepted_neurons, acepted_predicted = remove_neurons(datasets, neurons, predicted, 'std', max_std=std)
    print('Neurons Removed')
    
    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, acepted_predicted, spike_range=28)
    pred_vel, max_prob = SNB_apply(val_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(val_dataset, pred_vel)
    apply_2D_data(val_dataset, np.array(val_dataset['hand_vel']), col_name='vel')

    #Correlation calculation
    mean_corrX, std_corrX = mean_corr(val_dataset, ['vel_X', 'pred_vel_X'], 'pearson')
    mean_corrY, std_corrY = mean_corr(val_dataset, ['vel_Y', 'pred_vel_Y'], 'pearson')
    print('Correlation Calculated')
    print(f'(VelX) Maximum Pearson of {np.round(np.max(np.abs(mean_corrX)),2)} at {lags[np.argmax(np.abs(mean_corrX))]}')
    print(f'(VelY) Maximum Pearson of {np.round(np.max(np.abs(mean_corrY)),2)} at {lags[np.argmax(np.abs(mean_corrY))]}')
    print(f'Number of Removed Neurons: {len(neurons) - len(acepted_neurons)}')

    #Stores the correlation values
    std_corr_X.append(mean_corrX)
    std_corr_Y.append(mean_corrY)
    std_std_X.append(std_corrX)
    std_std_Y.append(std_corrY)
    std_neurons.append(acepted_neurons)
elapsed_std = round(time.time() - start_std,3)
print(f'It took {elapsed_std} sec')

In [ ]:
np.concatenate((np.round(np.linspace(300, 750, 10)), np.round(np.linspace(775, 1000, 10))))
np.round(np.linspace(300, 1000, 29))

In [ ]:
trials = trials_present(val_dataset)
%time plot_ccTogether(val_dataset, trials, ['vel_X', 'pred_vel_X'], focus_trial='mean', save=False, save_name='SNB_val_standardDS_velX.png')
%time plot_ccTogether(val_dataset, trials, ['vel_Y', 'pred_vel_Y'], focus_trial='mean', save=False, save_name='SNB_val_standardDS_velY.png')

In [ ]:
%time plot_allTrajectoriesS(val_dataset, standardDS, nr_cols=6, save=True, save_name='NB_val_standardDS_traj.pdf')

In [ ]:
%time plot_heatmap_all(mean_matrix, neurons, found_pairs, grids, observed, predicted, nr_cols=6, cmap='hot', fill_nan=True, save=True, save_name='test.pdf')

In [ ]:
%time plot_tune3D_all(mean_matrix, standard_neurons, found_pairs, grids, observed, predicted, nr_cols=6, fill_nan=False, save=False, save_name='3d_tuningcurve_standard_scroll50_nanfixed.pdf', observed_alpha=0.1)

In [ ]:
view_mode = 0

if(view_mode == 0):
    view=None
elif(view_mode == 1):
    view = [0, -90, 0]
elif(view_mode == 2):
    view = [90, -90, 0]
elif(view_mode == 3):
    view = [0, 0, 0]


%time plot_tune3D_single(mean_matrix, found_pairs, 37, grids, observed, predicted, view, True, observed_alpha=0.01)

In [ ]:
%time plot_Heatmap_single(mean_matrix, found_pairs, 37, grids, observed, predicted, fill_nan=True, plot_predicted=True)

In [ ]:
%time plot_prob_output(probability_vector, found_pairs, grids, fill_nan=True, logged=True, cmap='inferno', save=False, save_name='logP(s).png')

In [ ]:
%time plot_prob_output3D(probability_vector, found_pairs, grids, cmap='inferno', fill_nan=False, logged=True, save=False, save_name='logP(s)_3D.png')

In [ ]:
%time plot_spike_count(train_dataset)

In [ ]:
#rate_vector = np.asarray(np.random.rand(nr_neurons,1)*50, dtype='int')[:,0]
rate_vector = np.asarray(np.array(train_dataset['spikes'].iloc[0])*35*5*0.001, dtype='int')
s = np.array(train_dataset['hand_vel'].iloc[0])
#s = [500,500]

%time product = independent_conditional(predicted, [500,500], rate_vector, print_info=False)

In [ ]:
%time poi = poisson(predicted, 1, [500,500], 5.809)
poi

In [ ]:
%time probability = prob_output(np.array([100,-100]), found_pairs, velocity_vector)

In [ ]:
%time f = tune_value(predicted, 1, [500,500])
f

In [ ]:
%time probability_vector = all_prob_output(found_pairs, velocity_vector)

# Final Results and Plots

## Methods

In [ ]:
'''
Shows the population vector
'''
%time train_sets, val_sets, test_dataset = generate_sets(standardDS, 0.1, 1)
train_dataset = train_sets[0]
%time val_dataset = trial_datasetMaker(test_dataset, [0])

neurons = standard_neurons
plot_correlation = True

#Generates a decoder for every neuron
decoder = angle_decoder(train_dataset, 35, neurons)

#Runs the tuning curve for every neuron
_, predicted_matrix,_ = tune_neurons(train_dataset, decoder, 360, neurons)

#Removes unecessary neurons
#decoder, neurons = remove_neurons(predicted_matrix, val_dataset, neurons, 0)

#Predicts every angle according to a decoder
ns = angle_predictor(val_dataset, decoder, 35, neurons, distance_coef=0.02)

In [ ]:
'''
Builds the Population Vector Plot
'''
def plot_population_vector(ns, entry=0, trial=0, ratio=1, save=False, save_name='plot.png'):
    n_X, n_Y = ns
    first_entry = np.stack((n_X[entry], n_Y[entry]), axis=-1)
    #size = np.max([np.max(np.abs(first_entry)), np.max(np.abs(pop_vect))]) + 1
    size = np.max(np.abs(first_entry)) + 1
    pop_vect = np.sum(first_entry, axis=0)
    
    plt.figure(figsize=(5*ratio, 4.5*ratio))
    for i in range(len(first_entry)):
        plt.quiver(0, 0, first_entry[i, 0], first_entry[i, 1], angles='xy', scale_units='xy', scale=1, width=0.0005)
    
    plt.quiver(0, 0, pop_vect[0], pop_vect[1], angles='xy', scale_units='xy', color='r', scale=1, width=0.002, 
               label=f'Population Vector: {np.round(pop_vect,2)}')
    
    plt.xlim(-size, size)
    plt.ylim(-size, size)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(f'Population Vector for entry {entry} of trial {trial}')
    plt.grid()
    plt.legend()
    if(save): plt.savefig(save_name)
    plt.show()

In [ ]:
plot_population_vector(ns, 10, ratio=(4/3), save=True, save_name='population_vector_0_10.png')

In [ ]:
6/4.5

In [ ]:
%time train_sets, val_sets, test_dataset = generate_sets(standardDS, 0.1, 1)
train_dataset = train_sets[0]
val_dataset = val_sets[0]
neurons = standard_neurons

#Converting sets to firing rate -> Otimization Needed!!!
%time firing_rate(val_dataset, 35, panda=True) 
%time firing_rate(train_dataset, 35, panda=True) 
%time firing_rate(test_dataset, 35, panda=True) 

In [ ]:
'''
Returns the 3d tunning curves data with a given resolution and the predicted fitted curve --- Optimization Needed
'''
def tune_neurons3D(dataset, resolution=50):
    #Extracts data
    vel_x = np.array(dataset['hand_vel']['x'], dtype='float64')
    vel_y = np.array(dataset['hand_vel']['y'], dtype='float64')
    rate_data = np.array(dataset['spikes'], dtype='float64')
    
    #Gets limit points for binarization
    max_velX = np.max(vel_x)
    max_velY = np.max(vel_y)
    min_velX = np.min(vel_x)
    min_velY = np.min(vel_y)
    maximum = np.max([max_velX, max_velY])
    minimum = np.max([min_velX, min_velY])
    vels_resolution = np.linspace(minimum, maximum, resolution, endpoint=False)
    grid_x, grid_y = np.mgrid[minimum:maximum:resolution*1j, minimum:maximum:resolution*1j]
    
    #Binarizes the velocities
    closest_velX_idxs = np.argmin(np.abs(vels_resolution[:, np.newaxis] - vel_x), axis=0)
    closest_velY_idxs = np.argmin(np.abs(vels_resolution[:, np.newaxis] - vel_y), axis=0)
    vel_x = vels_resolution[closest_velX_idxs]
    vel_y = vels_resolution[closest_velY_idxs]
    
    grid1, grid2 = np.meshgrid(vels_resolution, vels_resolution)
    vels_pairs = np.vstack([grid1.ravel(), grid2.ravel()]).T
    velocity_vector = np.column_stack((vel_x, vel_y))
    
    #Average rates in same velocity quadrant ----- Optimize
    mean_matrix = []
    found_pairs = []
    for pair in vels_pairs:
        mask = (pair == velocity_vector)
        mask = np.logical_and(mask[:,0], mask[:,1])
        if(any(mask)):
            relevant_rates = rate_data[mask]
            mean_rate = np.mean(relevant_rates, axis=0)
            mean_matrix.append(mean_rate)
            found_pairs.append(pair)

    mean_matrix = np.asarray(mean_matrix, dtype='float64')
    found_pairs = np.asarray(found_pairs, dtype='float64')

    #Fits a Gaussian Curve ---- Optimize
    storage_gridZ = []
    predicted = []
    bound = ([0,np.nanmin(grid_x), np.nanmin(grid_y), 0,0, -np.inf, -1], 
                     [100,np.nanmax(grid_x), np.nanmax(grid_y), 1000, 1000, np.inf, 100])
    nr_neurons = len(mean_matrix[0])
    for i in range(nr_neurons):
        neuron_data = mean_matrix[:, i]
        grid_z = griddata((found_pairs[:,0], found_pairs[:,1]), neuron_data, (grid_x, grid_y), method='linear')
        gz = grid_z.ravel()
        total_spikes = np.sum(neuron_data)
        
        x_mean = np.sum((neuron_data * found_pairs[:,0]))/total_spikes
        y_mean = np.sum(neuron_data * found_pairs[:,1])/total_spikes
        max_locationF = np.nanargmax(grid_z)
        max_location = np.unravel_index(max_locationF, grid_z.shape)
        x_peak = grid_x[max_location]
        y_peak = grid_y[max_location]
        
        N = np.count_nonzero(neuron_data)
        if(N == 0): print(f'N equal to 0 at {i}')
        if(N == 1): 
            print(f'N equal to 1 at {i}')
            x_std = np.sqrt(np.sum(neuron_data * (found_pairs[:, 0] - x_mean)**2) / (((N)*total_spikes)/N))
            y_std = np.sqrt(np.sum(neuron_data * (found_pairs[:, 1] - y_mean)**2) / (((N)*total_spikes)/N))
        else:
            x_std = np.sqrt(np.sum(neuron_data * (found_pairs[:, 0] - x_mean)**2) / (((N-1)*total_spikes)/N))
            y_std = np.sqrt(np.sum(neuron_data * (found_pairs[:, 1] - y_mean)**2) / (((N-1)*total_spikes)/N))
        if(np.isnan(x_std)): x_std = 500
        if(np.isnan(y_std)): y_std = 500
            
        #initial_guess = (np.nanmax(grid_z),x_peak,y_peak,x_std,y_std,0,np.nanmin(grid_z))
        initial_guess = (np.nanmax(grid_z),x_mean,y_mean,x_std,y_std,0,np.nanmin(grid_z))
        #initial_guess = (np.nanmax(grid_z),0,0,512.5644445100842,483.7434837601396,0,np.nanmin(grid_z))
        popt, pcov = curve_fit(ex_twoD_Gaussian, (grid_x.ravel(), grid_y.ravel()), gz, p0=initial_guess, maxfev=5000, bounds=bound, 
                               nan_policy='omit')
        if(np.round((i+1)/nr_neurons, 2) == 0.25): print('25% Fitted')
        elif(np.round((i+1)/nr_neurons, 2) == 0.5): print('50% Fitted')
        elif(np.round((i+1)/nr_neurons, 2) == 0.75): print('75% Fitted')
        elif(np.round((i+1)/nr_neurons, 2) == 1): print('100% Fitted')
        '''
        try:
            popt, pcov = curve_fit(twoD_Gaussian, (grid_x, grid_y), gz, p0=initial_guess, maxfev=2000)
        except RuntimeError or popt[0] > 200:
            print(f"Unbounded fit failed for neuron {i}, trying bounded fit.")
            popt, pcov = curve_fit(twoD_Gaussian, (grid_x, grid_y), gz, p0=initial_guess, maxfev=2000, bounds=bound)
        if(popt[0] > 100):
            print(f'Wamp Wamp, wrong dimension found for amplitude at Neuron {i}. You got {np.round(popt[0],2)} for an aplitude')
            popt, pcov = curve_fit(twoD_Gaussian, (grid_x, grid_y), gz, p0=initial_guess, maxfev=2000, bounds=bound)
        '''
        if(popt[-1] < 0): popt[-1] = 0 #Avoids negative values
        
        storage_gridZ.append(grid_z)
        predicted.append(popt)
        
    storage_gridZ = np.asarray(storage_gridZ, dtype='float64')
    predicted = np.asarray(predicted, dtype='float64')
    grids = [grid_x, grid_y] 
    observed = storage_gridZ
    
    return mean_matrix, found_pairs, velocity_vector, grids, observed, predicted

'''
Gaussian surface generator (used to fit the data)
'''
def ex_twoD_Gaussian(xy, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    x, y = xy
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

#Fitting Gaussian Curves to Dataset
%time mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 50)

In [ ]:
'''
Visualizes the P(s) for all possible states
'''
def plot_prob_output(probability_vector, found_pairs, grids, ratio=1, fill_nan=False, cmap='hot', logged=True, save=False, save_name='plot.png'):
    #Log or not log values
    if(logged): logged_prob = np.log(probability_vector)
    else: logged_prob = probability_vector
        
    #Unpack Input
    grid_x, grid_y = grids
    grid_probability = griddata((found_pairs[:,0], found_pairs[:,1]), logged_prob, (grid_x, grid_y), method='linear')
    minimum = np.min(found_pairs)
    maximum = np.max(found_pairs)
    resolution = len(grid_x)
    if(fill_nan): grid_probability = np.nan_to_num(grid_probability, nan=np.nanmin(logged_prob))
    max_locationF = np.nanargmax(grid_probability)
    max_location = np.unravel_index(max_locationF, grid_probability.shape)

    #Plotting General
    plt.figure(figsize=(20*ratio,15*ratio))
    #plt.scatter(grid_x[max_location], grid_y[max_location], c='lightgreen', linewidths=1.5, marker='o', alpha=1,
    #           label=f'X: {np.round(grid_x[max_location],0)}\nY: {np.round(grid_y[max_location],0)}')
    im = plt.imshow(grid_probability.T, cmap=cmap, origin='lower', extent=(minimum, maximum, minimum, maximum))

    #Plotting Colorbar
    cb_max = np.nanmax(grid_probability)
    cb_min = np.nanmin(grid_probability)
    cb_range = np.linspace(cb_min, cb_max, 10)
    if(logged): plt.colorbar(im, ticks=cb_range, label='Log P(s)')
    else: plt.colorbar(im, ticks=cb_range, label='P(s)')

    #Plotting Titles, etc
    if(logged): plt.title('Heat Map of Log P(s)', fontsize=20)
    else: plt.title('Heat Map of P(s)', fontsize=20)
    plt.xlabel('X Velocity')
    plt.ylabel('Y Velocity')
    plt.gca().set_aspect('equal', adjustable='box')
    #plt.legend()
    
    #Plotting Saving
    if(save): plt.savefig(save_name)
    print((20*ratio,15*ratio))
    plt.show()

'''
Calculates the P(s) -> Probability Distribution of Output variables
'''
def prob_output(s, found_pairs, velocity_vector):
    #Binarize the input into the found_pairs binarization
    distances = np.linalg.norm(found_pairs - s, axis=1)
    position = np.argmin(distances)
    s = found_pairs[position]
    
    #Count the number of times the found bin appears in velocity_vector
    s_count = np.sum((velocity_vector[:,0] == s[0]) & (velocity_vector[:,1] == s[1]))
    
    #Divide by len of velocity_vector
    size = len(velocity_vector)
    probability = s_count / size
    return probability

'''
Generates all the P(s) for all possible states --- Optimize
'''
def all_prob_output(found_pairs, velocity_vector):
    nr_pairs = len(found_pairs)
    probability_vector = []
    for i in range(nr_pairs):
        s = found_pairs[i]
        probability_vector.append(prob_output(s, found_pairs, velocity_vector))
    probability_vector = np.array(probability_vector)
    
    ''' Find why doesnt work
    nr_pairs = len(found_pairs)
    probability_vector = np.where(found_pairs == found_pairs, prob_output(found_pairs, found_pairs, velocity_vector), 'NA')[:,0]
    probability_vector = np.asarray(probability_vector, dtype='float64')
    '''
    return probability_vector

probability_vector = all_prob_output(found_pairs, velocity_vector)
plot_prob_output(probability_vector, found_pairs, grids, ratio=0.4, fill_nan=False, logged=False, cmap='inferno', 
                 save=True, save_name='prior_s.png')

In [ ]:
'''
Visualizes the 3d tunning curve (heat map) of a given neuron
'''
def plot_Heatmap_single(mean_matrix, found_pairs, neuron, neurons, grids, observed, predicted, cmap='hot', ratio=1, fill_nan=False, 
                        plot_predicted=True, save=False, save_name='plot.png'):
    minimum = np.min(found_pairs)
    maximum = np.max(found_pairs)
    resolution = len(observed[0])
    
    grid_x, grid_y = grids
    grid_z = observed[neuron]
    popt = predicted[neuron]
    data_fitted = twoD_Gaussian((grid_x, grid_y), popt)
    
    if(fill_nan): grid_z = np.nan_to_num(grid_z)
    max_locationF = np.nanargmax(grid_z)
    max_location = np.unravel_index(max_locationF, grid_z.shape)
    
    plt.figure(figsize=(20*ratio,15*ratio))
    im = plt.imshow(grid_z.T, cmap=cmap, origin='lower', extent=(minimum, maximum, minimum, maximum))
    plt.scatter(popt[1], popt[2], c='#18E789', linewidths=1, marker='o', alpha=0.7,
               label=f'Predicted Peak\nX: {np.round(popt[1],0)}\nY: {np.round(popt[2],0)}\nAmplitude: {np.round(popt[0], 2)}')
    if(plot_predicted): plt.contour(grid_x, grid_y, data_fitted.reshape(resolution, resolution), 8, colors='#18E789')

    cb_max = np.nanmax(grid_z)
    cb_range = np.linspace(0, cb_max, 8)
    plt.colorbar(im, ticks=cb_range, label='Firing Rate (Hz)')
    plt.title(f'Heat Map of Neuron {neurons[neuron]} Firing Rates')
    plt.xlabel('X Velocity (px/s)')
    plt.ylabel('Y Velocity (px/s)')
    plt.legend()
    if(save): plt.savefig(save_name)
    print((20*ratio, 15*ratio))
    plt.show()

plot_Heatmap_single(mean_matrix, found_pairs, 75, standard_neurons, grids, observed, predicted, ratio=0.4, 
                    save=False, save_name='tuning3D_example2.png')

In [ ]:
'''
Visualizes all the tuning curves 3D (heat map)
'''
def plot_heatmap_all(mean_matrix, neurons, full_neurons, found_pairs, grids, observed, predicted, nr_cols=5, ratio=4, uniform=False, 
                     cmap='viridis', fill_nan=False, save=False, save_name='plot.png'):
    max_rate = np.max(mean_matrix)
    if(len(mean_matrix[0])%nr_cols == 0): nr_rows = len(neurons)//nr_cols
    else: nr_rows = len(neurons)//nr_cols + 1
        
    nr_neurons = len(neurons)
    minimum = np.min(found_pairs)
    maximum = np.max(found_pairs)
    grid_x, grid_y = grids
    resolution = len(observed[0])
    
    # Create a figure and subplots
    fig, axs = plt.subplots(nrows=nr_rows, ncols=nr_cols, figsize=(nr_cols*ratio, nr_rows*ratio))
    
    # Flatten the axs array for easier iteration
    axs = axs.flatten()
    
    # Plot each array
    for i, neuron in enumerate(neurons):
        ax = axs[i]
        grid_z = observed[neuron]
        if(fill_nan): grid_z = np.nan_to_num(grid_z)
        if(uniform): im = ax.imshow(grid_z.T, cmap=cmap, origin='lower', extent=(minimum, maximum, minimum, maximum), vmin=0, vmax=max_rate)
        else: im = ax.imshow(grid_z.T, cmap=cmap, origin='lower', extent=(minimum, maximum, minimum, maximum))

        popt = predicted[neuron]
        data_fitted = twoD_Gaussian((grid_x, grid_y), popt)
        ax.contour(grid_x, grid_y, data_fitted.reshape(resolution, resolution), 8, colors='#18E789')
        ax.scatter(popt[1], popt[2], c='#18E789', linewidths=0.1, marker='o', alpha=0.7,
                   label=f'Predicted Peak\n     X: {np.round(popt[1],0)}\n     Y: {np.round(popt[2],0)}\n     Amplitude: {np.round(popt[0], 2)}')
        
        cb_max = np.nanmax(grid_z)
        max_locationF = np.nanargmax(grid_z)
        max_location = np.unravel_index(max_locationF, grid_z.shape)
        
        if(i==0): ax.set_title(f'Tuning Surface of Neuron {full_neurons[neuron]}')
        if(i==1): ax.set_title(f'Fitted Tuning Surface of Neuron {full_neurons[neuron]}')
        ax.set_xlabel('X Velocity (px/s)')
        ax.set_ylabel('Y Velocity (px/s)')
    
        cb_range = np.linspace(0, cb_max, 7)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        cbar = fig.colorbar(im, cax=cax, orientation='vertical', ticks=cb_range, label='Firing Rate (Hz)')
        if(i==1): ax.legend()
        
    for ax in fig.get_axes():
        ax.label_outer()
    
    
    #plt.suptitle(f'Tuning Curves\n in a {resolution}x{resolution} grid ', y=0.999, fontsize=20)
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    print((nr_cols*ratio, nr_rows*ratio))
    plt.show()

'''
Gaussian surface generator (used to fit the data)
'''
def twoD_Gaussian(xy, param):
    amplitude, xo, yo, sigma_x, sigma_y, theta, offset = param
    x, y = xy
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

plot_heatmap_all(mean_matrix, [70, 75], standard_neurons, found_pairs, grids, observed, predicted, nr_cols=2, ratio=6, cmap='hot', 
                 fill_nan=False, save=True, save_name='tuning3D_example3.png')

In [ ]:
nr_neurons = len(standard_neurons)
for i in range(nr_neurons):
    if(standard_neurons[i] == 1663): print(i)

In [ ]:
'''
Plots the trajectory of one trial (with trial duration in ms)
'''
def plot_single_traj(original_dataset, trial_id, nr_cols=4, ratio=4, save=False, save_name='plot.png'):
    #Gets the trial_set
    mask = np.array([False] * len(original_dataset.trial_info))
    mask[trial_id] = True
    dataset = original_dataset.make_trial_data(start_field='move_onset_time', end_field='reach_time', ignored_trials=~mask)
    
    #Gets the hand positions (trua and pred)
    cursor_pos = np.asarray(dataset['cursor_pos'], dtype='float64')
    hand_pos = np.asarray(dataset['hand_pos'], dtype='float64')
    
    hand_x = hand_pos[:,0]
    hand_y = hand_pos[:,1]
    cursor_x = cursor_pos[:,0]
    cursor_y = cursor_pos[:,1]
    
    #Gets the trials and time
    trial_ids = np.asarray(dataset['trial_id'], dtype='int64')
    mask = np.concatenate(([True], trial_ids[1:] != trial_ids[:-1]))
    split_indices = np.where(mask)[0]
    align_time = np.asarray(dataset['align_time'])
    
    #Gets the target positions per trial
    trials = np.unique(trial_ids)
    target_pos = np.asarray(original_dataset.trial_info[['active_pos_x', 'active_pos_y']], dtype='int64')[trials]
    barrier_pos = np.asarray(original_dataset.trial_info['barrier_pos'])[trials]
    barrier_lengths = np.array([len(inner_array) for inner_array in barrier_pos])
    
    #Splits all arrays acording to the trials
    align_time = np.split(align_time, split_indices)[1:]
    hand_x = np.split(hand_x, split_indices)[1:]
    hand_y = np.split(hand_y, split_indices)[1:]
    cursor_x = np.split(cursor_x, split_indices)[1:]
    cursor_y = np.split(cursor_y, split_indices)[1:]
    
    nr_trials = len(trials)
    #The ploting starts
    if(nr_trials%nr_cols == 0): nr_rows = nr_trials//nr_cols
    else: nr_rows = nr_trials//nr_cols + 1
    
    fig, axs = plt.subplots(nrows=nr_rows, ncols=nr_cols, figsize=(nr_cols*ratio, nr_rows*ratio))
    axs = axs.flatten()
    
    #Plotting
    for i in range(nr_trials):
        ax = axs[i]
        x_coords = hand_x[i]
        y_coords = hand_y[i]
        x_cursor = cursor_x[i]
        y_cursor = cursor_y[i]
        target_x = target_pos[i,0]
        target_y = target_pos[i,1]
        duration = int(align_time[i][-1])/1000000000
        barriers = barrier_pos[i]
    
        #Gets the limit
        x_lim = np.max([np.max(x_coords), np.max(x_cursor), np.abs(np.min(x_coords)), np.abs(np.min(x_cursor))])
        y_lim = np.max([np.max(y_coords), np.max(y_cursor), np.abs(np.min(y_coords)), np.abs(np.min(y_cursor))])
        lim_value = np.max([x_lim,y_lim])+15
        
        #Plots the barriers
        for j in range(barrier_lengths[i]):
            x, y, half_width, half_height = barrier_pos[i][j]
            left = x - half_width
            bottom = y - half_height
            width = 2 * half_width
            height = 2 * half_height
            ax.add_patch(plt.Rectangle((left, bottom), width, height, facecolor='gray'))
    
        ax.plot(x_cursor, y_cursor, label='Cursor Path')
        ax.plot(x_coords, y_coords, label='Hand Path', alpha=0.4, color='grey')
        ax.scatter(target_x, target_y, label='Target Position', color='red')
        ax.scatter(x_cursor[0], y_cursor[0], label='Start Point', color='black')
        ax.set_title(f'Trial {trials[i]} (lasted {duration} seconds)')
        ax.set_xlim(-lim_value, lim_value)
        ax.set_ylim(-lim_value, lim_value)
        ax.set_xlabel('Pixels')
        ax.set_ylabel('Pixels')
        ax.legend(loc='lower left')
        ax.grid(True)
    
    #plt.suptitle(f'Trial Trajectories', y=1, fontsize=20)
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    print((nr_cols*ratio, nr_rows*ratio))
    plt.show()

In [ ]:
#plot_single_traj(standardDS, np.arange(2295), nr_cols=6, ratio=4.5, save=False, save_name='traj_all(page).pdf')
#plot_single_traj(standardDS, np.arange(2295), nr_cols=48, ratio=4.5, save=False, save_name='traj_all(square).pdf')

In [ ]:
def raster_plot(dataset, neurons=None, trial=0, align=None, ratio=6, save=False, save_name='raster.png'):
    mask = np.all(dataset.trial_info[['trial_id']] == trial, axis=1)

    if align is None: 
        trial_data = dataset.make_trial_data(ignored_trials=(~mask))
    else:
        trial_data = dataset.make_trial_data(start_field='move_onset_time', end_field='reach_time', align_range=align, ignored_trials=(~mask))

    if neurons is None:
        neurons = trial_data['spikes'].columns.tolist()  # Gets the name of all neurons
    
    neuron_spikes = []
    for neuron in neurons:
        neuron_spikes.append(trial_data['spikes'][neuron])

    # Initialize a list to store spike times for each neuron
    spike_times = [[] for _ in range(len(neuron_spikes))]
    
    # Iterate through the spike data and extract spike times for each neuron
    for neuron_idx, neuron in enumerate(neuron_spikes):
        for time_idx, spike in enumerate(neuron):
            if spike == 1:
                spike_times[neuron_idx].append(time_idx * 5)  # Multiply by 5 to convert bins to ms

    #Get the color
    color_array = np.round(np.linspace(0, 255, 5))
    color_template = 'turbo'
    
    # Gets the tot timestamp in milliseconds
    tot_str = dataset.trial_info.loc[trial, 'target_on_time']
    gct_str = dataset.trial_info.loc[trial, 'go_cue_time']
    mot_str = dataset.trial_info.loc[trial, 'move_onset_time']
    rt_str = dataset.trial_info.loc[trial, 'reach_time']
    tot = pd.to_timedelta(tot_str).total_seconds() * 1000  # Convert to milliseconds
    gct = pd.to_timedelta(gct_str).total_seconds() * 1000  # Convert to milliseconds
    mot = pd.to_timedelta(mot_str).total_seconds() * 1000  # Convert to milliseconds
    rt = pd.to_timedelta(rt_str).total_seconds() * 1000  # Convert to milliseconds
    
    fig, ax = plt.subplots(figsize=(1*ratio, 3/4*ratio))
    for i, spikes in enumerate(spike_times, start=1):
        ax.eventplot(spikes, colors='k', lineoffsets=i, linelengths=0.5)
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Neurons')
    ax.set_title(f'Raster Plot of Trial {trial}')
    
    # Set x-ticks to show time in ms
    num_bins = trial_data['spikes'].shape[0]  # Total number of bins
    max_time_ms = num_bins * 5  # Convert total bins to ms
    ax.set_xlim(0, max_time_ms)
    xticks = np.arange(0, max_time_ms + 1, 500)  # Set x-ticks every 500 ms
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticks)
    
    # Add a vertical line at the 'tot' timestamp
    ax.axvline(tot, color=plt.colormaps.get_cmap(color_template)(int(color_array[0])), linestyle='--', linewidth=1, 
               label=f'Target On Time: {tot/1000:.3f} sec')
    ax.axvline(gct, color=plt.colormaps.get_cmap(color_template)(int(color_array[1])), linestyle='--', linewidth=1, 
               label=f'Go Cue Time: {gct/1000:.3f} sec')
    ax.axvline(mot, color=plt.colormaps.get_cmap(color_template)(int(color_array[2])), linestyle='--', linewidth=1, 
               label=f'Move Onset Time: {mot/1000:.3f} sec')
    ax.axvline(rt, color=plt.colormaps.get_cmap(color_template)(int(color_array[3])), linestyle='--', linewidth=1, 
               label=f'Reach Time: {rt/1000:.3f} sec')
    ax.legend()

    if save:
        plt.savefig(save_name)
    plt.show()


In [ ]:
raster_plot(standardDS, standard_neurons, trial=0, ratio=6, save=True, save_name='raster_full_example.png')

In [ ]:
dataset = standardDS
trial = 0
nr_neurons = 65

'''
Plot all frequencies of a trial
'''
#Extract Data
mask = np.all(dataset.trial_info[['trial_id']] == trial, axis=1)
trial_data = dataset.make_trial_data(start_field='move_onset_time', end_field='reach_time', ignored_trials=(~mask))
firing_rate(trial_data, 35, panda=True) 
data = np.array(trial_data['spikes'])[:,0:nr_neurons].T
X = np.linspace(-1, 1, data.shape[-1])
G = 0.25

#Colors
color_array = np.round(np.linspace(100, 255, nr_neurons))
#np.random.shuffle(color_array)
color_template = 'Blues'


#Starts Plotting
fig = plt.figure(figsize=(10,6))

# Generate line plots
lines = []
for i in range(len(data)):
    # Small reduction of the X extents to get a cheap perspective effect
    xscale = 1 - i / 200.
    # Same for linewidth (thicker strokes on the bottom)
    lw = 1.5 - i / 500.0
    line, = plt.plot(xscale * X, i*2 + G * data[i], color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), lw=lw)
    lines.append(line)

plt.ylim(-1, 70)

# No ticks
plt.xticks([])
plt.yticks([])
plt.xlabel('Time (ms)')
plt.ylabel('Neurons')
plt.title(f'Trial {trial} Firing Rate')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Sample dataset and parameters for illustration
dataset = standardDS
neurons = standard_neurons
trial = 0
nr_neurons = 182
ratio = 0.75

# Extract Data
mask = np.all(dataset.trial_info[['trial_id']] == trial, axis=1)
trial_data = dataset.make_trial_data(start_field='move_onset_time', end_field='reach_time', ignored_trials=(~mask))
# Assuming `firing_rate` is a function that processes `trial_data` (adjust accordingly if needed)
firing_rate(trial_data, 35, panda=True)
data = np.array(trial_data['spikes'])[:, 0:nr_neurons].T
X = np.linspace(0, data.shape[-1] * 5, data.shape[-1])

# Colors
color_array = np.round(np.linspace(100, 255, nr_neurons))
color_template = 'hsv'

# Focus adjustment
for j in range(nr_neurons):
    focus = j
    numbered_neurons = np.arange(len(neurons))
    numbered_neurons = np.delete(numbered_neurons, focus)
    
    # Start Plotting
    fig, ax = plt.subplots(figsize=(10 * ratio, 6 * ratio))
    
    # Generate line plots
    lines = []
    for i in numbered_neurons:
        if(np.mean(data[i]) < 20): line, = ax.plot(X, data[i], color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), alpha=0.005)
        else: line, = ax.plot(X, data[i], color=plt.colormaps.get_cmap(color_template)(int(color_array[i])), alpha=0.05)
        lines.append(line)
    focus_line, = ax.plot(X, data[focus], color=plt.colormaps.get_cmap(color_template)(int(color_array[focus])),
                          label=f'Neuron {standard_neurons[focus]}')
    lines.append(focus_line)

    # Extract the legend handles and labels
    handles, labels = ax.get_legend_handles_labels()

    # Sort the labels and handles based on the sorted labels
    sorted_labels_handles = sorted(zip(labels, handles), key=lambda x: int(x[0].split()[1]))
    sorted_labels, sorted_handles = zip(*sorted_labels_handles)

    # Create the legend with sorted labels
    ax.legend(sorted_handles, sorted_labels)

    # No ticks
    plt.xlabel('Time (ms)')
    plt.ylabel('Firing Rate')
    plt.title(f'Trial {trial} Firing Rate')
    plt.savefig(f'trial0_neuron{focus}.png')
    plt.show()


In [ ]:
np.round(np.rad2deg(1.51),2)

In [ ]:
'''
Produces the tune curve data for a set number of angles for the observed firing rate and for the predicted firing rate
'''

def tune_neurons(dataset, decoder, nr_directions, neurons):
    rate_data = np.asarray(dataset['spikes'], dtype='float64')
    angle_data = np.asarray(dataset['angle'], dtype='float64')
    target_angles = np.linspace(-np.pi, np.pi, nr_directions, endpoint=False)
    
    closest_angle_idxs = np.argmin(np.abs(target_angles[:, np.newaxis] - angle_data), axis=0)
    angle_data = target_angles[closest_angle_idxs]
    
    #Gets the observed curve
    mean_matrix = []
    for target_angle in (target_angles):
        mask = (angle_data == target_angle)
        relevant_rates = rate_data[mask]
        mean_rate = np.mean(relevant_rates, axis=0)
        mean_matrix.append(mean_rate)
    mean_matrix = np.asarray(mean_matrix, dtype='float64')
    
    #Gets the predicted curve
    bias = decoder[:, 3]
    k = decoder[:, 5]
    angle = decoder[:, 4]
    prediced_matrix = bias[:, np.newaxis] + k[:, np.newaxis] * np.cos(target_angles - angle[:, np.newaxis])
    prediced_matrix = np.asarray(prediced_matrix, dtype='float64')
    
    return mean_matrix, prediced_matrix, target_angles

'''
Generates a decoder by fiting the training data to cosines (based in georgopolos et. al)
'''
def angle_decoder(dataset, window_size, neurons, delay=0, bin_size=5):
    if(delay != 0): decoder = decoder_cosinesDelay(dataset, window_size, delay, bin_time=bin_size)
    else: decoder = decoder_cosinesNoDelay(dataset, window_size, neurons, bin_time=bin_size)
    return decoder

'''
Builds decoder by fiting the firing rate to cosines. This function does not support delay
'''
def decoder_cosinesNoDelay(dataset, window_size, neurons, bin_time=5):
    #Change spikes to firing rates in ms (moving average) ----------- Can be Optimized
    rate_data, trials_vector = firing_rate(dataset, window_size, bin_time=bin_time, panda=False)
    train_size = len(rate_data)
    nr_neurons = len(neurons)
    
    #Initiates the vectors for calculating the vector M
    hand_pos = np.asarray(dataset['hand_pos'], dtype='float64')
    cosines_matrix = np.zeros((train_size, 2))
    angle = np.zeros(train_size)
    
    #Adds the column vectorM with the cosines components to the train_dataset
    for i in range(train_size):
        if (i == 0 or ((trials_vector[i] != trials_vector[i-1]))):
            preX = 0
            preY = 0
        else:
            preX = hand_pos[i-1,0]
            preY = hand_pos[i-1,1]
        x = hand_pos[i,0]
        y = hand_pos[i,1]
        angle_temp = np.arctan2((y - preY), (x - preX))
        cosX = np.cos(angle_temp)
        cosY = np.sin(angle_temp)
        cosines_matrix[i,0] = cosX
        cosines_matrix[i,1] = cosY
        angle[i] = angle_temp
    
    #Trains the decoder fiting the data to a cos
    decoder = []
    for i in range(nr_neurons):
        neuron_data = rate_data[:,i]
        weights = linear_regression_model(cosines_matrix, neuron_data, loss='ridge')
        k = np.sqrt(np.power(weights[1],2) + np.power(weights[2],2))
        if(k == 0): 
            print(f'neuron {neurons[i]} did not fire')
            cx = 1
            cy = 0
        else:
            cx = weights[1]/k
            cy = weights[2]/k
        angleC = np.arctan2(cy, cx)
        decoder.append([neurons[i], cx, cy, weights[0], angleC, k])
    dataset['spikes'] = rate_data
    dataset['angle'] = angle
    return np.array(decoder)

'''
Linear regression with given loss function and given regularization that returns the weights
'''
def linear_regression_model(trainning_var, trainning_out, loss='sse', lambaRidge=2):
    #Data preparation
    rates = trainning_var
    vel = trainning_out
    
    #Loss Function Selection
    if(loss == 'sse'):
        rates = np.insert(rates, 0, 1, axis=1)
        penrose = np.linalg.pinv(rates.astype(np.float32))
        w = np.matmul(penrose,vel)
        predicted = np.matmul(rates,w)
    elif(loss == 'ridge'):
        rates = np.insert(rates, 0, 1, axis=1)
        ratesT = np.transpose(rates)
        prod = np.matmul(ratesT,rates)
        identity = np.identity(len(prod))
        helper = prod + lambaRidge * identity
        inverse = np.linalg.inv(helper)
        penrose = np.matmul(inverse,ratesT)
        w = np.matmul(penrose,vel)
        predicted = np.matmul(rates,w)

    return w

'''
Plots the tuning curves
'''
def plot_tuning(dataset, decoder, nr_directions, neurons, full_neurons, nr_cols=5, ratio=4, threshold=0, save=False, save_name='plot.png'):
    mean_matrix, predicted_matrix, target_angles = tune_neurons(dataset, decoder, nr_directions, neurons)
    max_rate = np.max([np.max(mean_matrix), np.max(predicted_matrix)])+1
    if(len(mean_matrix[0])%nr_cols == 0): nr_rows = len(neurons)//nr_cols
    else: nr_rows = len(neurons)//nr_cols + 1
    
    # Create a figure and subplots
    fig, axs = plt.subplots(nrows=nr_rows, ncols=nr_cols, figsize=(nr_cols*ratio, nr_rows*ratio))
    
    # Flatten the axs array for easier iteration
    axs = axs.flatten()
    
    # Plot each array
    for i, neuron in enumerate(neurons):
        ax = axs[i]
        observed = mean_matrix[:,neuron]
        predicted = predicted_matrix[neuron]
        max_angle = target_angles[np.argmax(predicted)]
        height = round(np.max(predicted_matrix[neuron])-np.min(predicted_matrix[neuron]),3)
        if(height > threshold):
            ax.plot(target_angles, observed, label='Observed Tuning Curve')
            if(i==1):ax.plot(target_angles, predicted, label=f'Fitted Tuning Curve\nPreferred Angle: {np.round(max_angle,2)} rads')
            if(i==0):ax.set_title(f'Tuning curve of neuron {full_neurons[neuron]}', fontsize=10)
            else:ax.set_title(f'Fitted tuning curve of neuron {full_neurons[neuron]}', fontsize=10)
        else: 
            ax.plot(target_angles, observed, label='Observed Tuning Curve', alpha=0.3)
            ax.plot(target_angles, predicted, label=f'Predicted Tuning Curve\nPreferred Angle: {np.round(max_angle,2)} rads', alpha=0.3)
            ax.set_title(f'Tuning curve of neuron {full_neurons[neuron]}', fontsize=10)
        ax.set_xlabel('Angle (rads)')
        ax.set_ylabel('Firing Rate (Hz)')
        ax.set_ylim(0, max_rate)
        ax.legend(loc='lower left')
        ax.grid(True)
    
    for ax in fig.get_axes():
        ax.label_outer()
        
    #plt.suptitle(f'Tuning Curves\n with {nr_directions} angles and threshold at {threshold}', y=1, fontsize=15)
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    print((nr_cols*ratio, nr_rows*ratio))
    plt.show()

In [ ]:
%time train_sets, val_sets, test_dataset = generate_sets(standardDS, 0.1, 1)
train_dataset = train_sets[0]
val_dataset = val_sets[0]
decoder = angle_decoder(train_dataset, 35, standard_neurons)
plot_tuning(train_dataset, decoder, 100, [181,181], standard_neurons, nr_cols=2, ratio=4.5, 
            save=True, save_name='tuning2D_example_comparition.png')

In [ ]:
nr_neurons = len(standard_neurons)
for i in range(nr_neurons):
    if(standard_neurons[i] == 2961): print(i)

In [ ]:
np.sqrt(182)

In [ ]:
len(standard_neurons)

In [ ]:
len(np.unique(train_dataset['trial_id']))/len(np.unique(standardDS.trial_info['trial_id'])), len(np.unique(val_dataset['trial_id']))/len(np.unique(standardDS.trial_info['trial_id'])), len(np.unique(test_dataset['trial_id']))/len(np.unique(standardDS.trial_info['trial_id']))

In [ ]:
np.round([0.6749455337690632, 0.07494553376906318, 0.25010893246187366],4)*100

In [ ]:
len(np.unique(train_dataset['trial_id'])), len(np.unique(val_dataset['trial_id'])), len(np.unique(test_dataset['trial_id']))

## Results

In [ ]:
'''
Decodes the hand position (angle, cosines and trajectory) by applying a decoder fitted to the cosines
'''

def angle_predictor(dataset, decoder, window_size, neurons, bin_time=5, distance_coef=0.05, pred_pos=True):
    #Prepares the data
    rate_data, trials_ids = firing_rate(dataset, window_size, bin_time=bin_time, panda=False)
    val_size = len(dataset)
    #Predicts the direction
    individual_predX = (rate_data - decoder[:,3])*decoder[:,1]
    individual_predY = (rate_data - decoder[:,3])*decoder[:,2]
    pred_cosX = np.sum(individual_predX, axis=1)
    pred_cosY = np.sum(individual_predY, axis=1)
    pred_angle = np.arctan2(pred_cosY, pred_cosX)
    pred_magnitude = np.sqrt(np.power(pred_cosX,2) + np.power(pred_cosY,2))
    #Normalizes the cos and sin
    pred_cosX = np.cos(pred_angle)
    pred_cosY = np.sin(pred_angle)
    
    #Adds it to the dataset
    dataset['pred_mag'] = pred_magnitude
    dataset['pred_cosX'] = pred_cosX
    dataset['pred_cosY'] = pred_cosY
    dataset['pred_angle'] = pred_angle
    dataset['spikes'] = rate_data
    
    #Builds the vectorM for comparition
    trial_pos = np.asarray(dataset['cursor_pos'], dtype='float64')
    mask = np.concatenate(([True], trials_ids[1:] != trials_ids[:-1]))
    diffs = trial_pos - np.roll(trial_pos, 1, axis=0)
    diffs = np.where(mask[:, np.newaxis], np.zeros_like(diffs), diffs)
    angles = np.arctan2(diffs[:, 1], diffs[:, 0])
    cosines_matrix = np.column_stack((np.cos(angles), np.sin(angles)))
    
    dataset['mx'] = cosines_matrix[:,0]
    dataset['my'] = cosines_matrix[:,1]
    dataset['angle'] = angles
    
    #Predicts the position if needed
    if(pred_pos):
        pred_cosNX = np.roll(pred_cosX, 1)
        pred_cosNY = np.roll(pred_cosY, 1)
        pred_magnitude = np.roll(pred_magnitude, 1)
        pred_x = np.where(mask, trial_pos[:,0], pred_cosNX * pred_magnitude * distance_coef)
        pred_y = np.where(mask, trial_pos[:,1], pred_cosNY * pred_magnitude * distance_coef)
    
        for i in range(len(pred_x)):
            if(not mask[i]): 
                pred_x[i] = pred_x[i-1] + pred_x[i]
                pred_y[i] = pred_y[i-1] + pred_y[i]
        
        dataset['pred_X'] = pred_x
        dataset['pred_Y'] = pred_y

    return individual_predX, individual_predY

'''
Produces the tune curve data for a set number of angles for the observed firing rate and for the predicted firing rate
'''

def tune_neurons(dataset, decoder, nr_directions, neurons):
    rate_data = np.asarray(dataset['spikes'], dtype='float64')
    angle_data = np.asarray(dataset['angle'], dtype='float64')
    target_angles = np.linspace(-np.pi, np.pi, nr_directions, endpoint=False)
    
    closest_angle_idxs = np.argmin(np.abs(target_angles[:, np.newaxis] - angle_data), axis=0)
    angle_data = target_angles[closest_angle_idxs]
    
    #Gets the observed curve
    mean_matrix = []
    for target_angle in (target_angles):
        mask = (angle_data == target_angle)
        relevant_rates = rate_data[mask]
        mean_rate = np.mean(relevant_rates, axis=0)
        mean_matrix.append(mean_rate)
    mean_matrix = np.asarray(mean_matrix, dtype='float64')
    
    #Gets the predicted curve
    bias = decoder[:, 3]
    k = decoder[:, 5]
    angle = decoder[:, 4]
    prediced_matrix = bias[:, np.newaxis] + k[:, np.newaxis] * np.cos(target_angles - angle[:, np.newaxis])
    prediced_matrix = np.asarray(prediced_matrix, dtype='float64')
    
    return mean_matrix, prediced_matrix, target_angles

'''
Generates a decoder by fiting the training data to cosines (based in georgopolos et. al)
'''
def angle_decoder(dataset, window_size, neurons, delay=0, bin_size=5):
    if(delay != 0): decoder = decoder_cosinesDelay(dataset, window_size, delay, bin_time=bin_size)
    else: decoder = decoder_cosinesNoDelay(dataset, window_size, neurons, bin_time=bin_size)
    return decoder

'''
Builds decoder by fiting the firing rate to cosines. This function does not support delay
'''
def decoder_cosinesNoDelay(dataset, window_size, neurons, bin_time=5):
    #Change spikes to firing rates in ms (moving average) ----------- Can be Optimized
    rate_data, trials_vector = firing_rate(dataset, window_size, bin_time=bin_time, panda=False)
    train_size = len(rate_data)
    nr_neurons = len(neurons)
    
    #Initiates the vectors for calculating the vector M
    hand_pos = np.asarray(dataset['cursor_pos'], dtype='float64')
    cosines_matrix = np.zeros((train_size, 2))
    angle = np.zeros(train_size)
    
    #Adds the column vectorM with the cosines components to the train_dataset
    for i in range(train_size):
        if (i == 0 or ((trials_vector[i] != trials_vector[i-1]))):
            preX = 0
            preY = 0
        else:
            preX = hand_pos[i-1,0]
            preY = hand_pos[i-1,1]
        x = hand_pos[i,0]
        y = hand_pos[i,1]
        angle_temp = np.arctan2((y - preY), (x - preX))
        cosX = np.cos(angle_temp)
        cosY = np.sin(angle_temp)
        cosines_matrix[i,0] = cosX
        cosines_matrix[i,1] = cosY
        angle[i] = angle_temp
    
    #Trains the decoder fiting the data to a cos
    decoder = []
    for i in range(nr_neurons):
        neuron_data = rate_data[:,i]
        weights = linear_regression_model(cosines_matrix, neuron_data, loss='ridge')
        k = np.sqrt(np.power(weights[1],2) + np.power(weights[2],2))
        if(k == 0): 
            print(f'neuron {neurons[i]} did not fire')
            cx = 1
            cy = 0
        else:
            cx = weights[1]/k
            cy = weights[2]/k
        angleC = np.arctan2(cy, cx)
        decoder.append([neurons[i], cx, cy, weights[0], angleC, k])
    dataset['spikes'] = rate_data
    dataset['angle'] = angle
    return np.array(decoder)

'''
Linear regression with given loss function and given regularization that returns the weights
'''
def linear_regression_model(trainning_var, trainning_out, loss='sse', lambaRidge=2):
    #Data preparation
    rates = trainning_var
    vel = trainning_out
    
    #Loss Function Selection
    if(loss == 'sse'):
        rates = np.insert(rates, 0, 1, axis=1)
        penrose = np.linalg.pinv(rates.astype(np.float32))
        w = np.matmul(penrose,vel)
        predicted = np.matmul(rates,w)
    elif(loss == 'ridge'):
        rates = np.insert(rates, 0, 1, axis=1)
        ratesT = np.transpose(rates)
        prod = np.matmul(ratesT,rates)
        identity = np.identity(len(prod))
        helper = prod + lambaRidge * identity
        inverse = np.linalg.inv(helper)
        penrose = np.matmul(inverse,ratesT)
        w = np.matmul(penrose,vel)
        predicted = np.matmul(rates,w)

    return w

'''
Plots the tuning curves
'''
def plot_tuning(dataset, decoder, nr_directions, neurons, full_neurons, nr_cols=5, ratio=4, threshold=0, save=False, save_name='plot.png'):
    mean_matrix, predicted_matrix, target_angles = tune_neurons(dataset, decoder, nr_directions, neurons)
    max_rate = np.max([np.max(mean_matrix), np.max(predicted_matrix)])+1
    if(len(mean_matrix[0])%nr_cols == 0): nr_rows = len(neurons)//nr_cols
    else: nr_rows = len(neurons)//nr_cols + 1
    
    # Create a figure and subplots
    fig, axs = plt.subplots(nrows=nr_rows, ncols=nr_cols, figsize=(nr_cols*ratio, nr_rows*ratio))
    
    # Flatten the axs array for easier iteration
    axs = axs.flatten()
    
    # Plot each array
    for i, neuron in enumerate(neurons):
        ax = axs[i]
        observed = mean_matrix[:,neuron]
        predicted = predicted_matrix[neuron]
        max_angle = target_angles[np.argmax(predicted)]
        height = round(np.max(predicted_matrix[neuron])-np.min(predicted_matrix[neuron]),3)
        if(height > threshold):
            ax.plot(target_angles, observed, label='Observed Tuning Curve')
            if(i==1):ax.plot(target_angles, predicted, label=f'Fitted Tuning Curve\nPreferred Angle: {np.round(max_angle,2)} rads')
            if(i==0):ax.set_title(f'Tuning curve of neuron {full_neurons[neuron]}', fontsize=10)
            else:ax.set_title(f'Fitted tuning curve of neuron {full_neurons[neuron]}', fontsize=10)
        else: 
            ax.plot(target_angles, observed, label='Observed Tuning Curve', alpha=0.3)
            ax.plot(target_angles, predicted, label=f'Predicted Tuning Curve\nPreferred Angle: {np.round(max_angle,2)} rads', alpha=0.3)
            ax.set_title(f'Tuning curve of neuron {full_neurons[neuron]}', fontsize=10)
        ax.set_xlabel('Angle (rads)')
        ax.set_ylabel('Firing Rate (Hz)')
        ax.set_ylim(0, max_rate)
        ax.legend(loc='lower left')
        ax.grid(True)
    
    for ax in fig.get_axes():
        ax.label_outer()
        
    #plt.suptitle(f'Tuning Curves\n with {nr_directions} angles and threshold at {threshold}', y=1, fontsize=15)
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    print((nr_cols*ratio, nr_rows*ratio))
    plt.show()

In [ ]:
'''
Predictions made with decoder trained with hand_pos covariate and cosines angles (paper)
Moving Average Size: 35
Time Required: 1 sec
'''
%time train_sets, val_sets, test_dataset = generate_sets(standardDS, 0.1, 1)
train_dataset = train_sets[0]
val_dataset = val_sets[0]
neurons = standard_neurons
plot_correlation = True

#Generates a decoder for every neuron
decoder = angle_decoder(train_dataset, 35, neurons)

#Runs the tuning curve for every neuron
_, predicted_matrix,_ = tune_neurons(train_dataset, decoder, 360, neurons)

#Removes unecessary neurons
#decoder, neurons = remove_neurons(predicted_matrix, val_dataset, neurons, 0)

#Predicts every angle according to a decoder
angle_predictor(val_dataset, decoder, 35, neurons, distance_coef=0.02)

#Plots Correlation
metrics = [['mx', 'pred_cosX'],['my', 'pred_cosY'],['angle', 'pred_angle']]
file_names = ['mean_pearson_all_cosx_standardDS_nonOptimized.png','mean_pearson_all_cosy_standardDS_nonOptimized.png',
              'mean_pearson_all_angle_standardDS_nonOptimized.png']
trials = trials_present(val_dataset)
if(plot_correlation):
    for i in range(len(metrics)):
        plot_ccTogether(val_dataset, trials , metrics[i], focus_trial='mean', correlation='pearson', window_size=150, 
                        compare_axis=True, color_template='twilight', save=False, save_name=file_names[i])

## Population Vector

### Window Size Hacking

In [ ]:
'''
Optimizes the Population Vector for window size
'''
total_time = time.time()

#|----------Gets the Datasets Info----------|
print('Starting Dataset Preparation')
start_time = time.time()

train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, 1)
o_train_dataset = train_sets[0]
o_val_dataset = val_sets[0]
neurons = standard_neurons

end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'It took {elapsed_time} sec')

#|----------Preparing for Hacking----------|
print('Starting Hacking Preparation')
start_time = time.time()

window_size = np.linspace(2, 50, 49, dtype=int)
metrics = [['mx', 'pred_cosX'],['my', 'pred_cosY'],['angle', 'pred_angle']]
parameters = []
all_entries = []

end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'It took {elapsed_time} sec')

#|----------Hyperparametrization Hacking----------|
print('Hacking Starting\n')
for size in window_size:
    print(f'|----- Trying window size of {size} -----|')
    start_time = time.time()
    
    #Generates a decoder
    train_dataset = o_train_dataset.copy()
    val_dataset = o_val_dataset.copy()
    decoder = angle_decoder(train_dataset, size, neurons)

    #Predicts every angle according to a decoder
    angle_predictor(val_dataset, decoder, size, neurons, pred_pos=False)

    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(val_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

    end_time = time.time()
    elapsed_time = round(end_time - start_time,3)
    print(f'It took {elapsed_time} sec\n')

parameters = np.array(parameters)
all_entries = np.array(all_entries)
print(f'It took a total time of {round(time.time() - total_time,3)}')

In [ ]:
title_name = 'Window Size Hacking for Population Vector'
label_name = 'Window Size (nº of bins)'
save_name = 'ppv_window_size_hack_r2.png'
eval='R2'
compare_models(parameters, all_entries, window_size, eval=eval, title_name=title_name, label_name=label_name, save=True, save_name=save_name)

### Neuron Hacking

In [ ]:
'''
Optimizes the Population Vector for angle resolution
'''
total_time = time.time()

#|----------Gets the Datasets Info----------|
print('Starting Dataset Preparation')
start_time = time.time()

train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, 1)
train_dataset = train_sets[0]
o_val_dataset = val_sets[0]
neurons = standard_neurons

end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'It took {elapsed_time} sec')

#|----------Preparing for Hacking----------|
print('Starting Hacking Preparation')
start_time = time.time()

#Gets thresholds
metrics = [['mx', 'pred_cosX'],['my', 'pred_cosY'],['angle', 'pred_angle']]
parameters = []
all_entries = []

#Generates a decoder
o_neurons = standard_neurons
o_decoder = angle_decoder(train_dataset, 31, neurons)

#Runs the tuning curve for every neuron
observed_matrix, predicted_matrix,_ = tune_neurons(train_dataset, o_decoder, 360, o_neurons)
diff = np.sqrt(np.sum(np.power(predicted_matrix - observed_matrix.T, 2), axis=1))
threshold = np.linspace(np.max(diff), np.min(diff), 50, endpoint=False)

end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'It took {elapsed_time} sec')

#|----------Hyperparametrization Hacking----------|
print('Hacking Starting\n')
for i, thr in enumerate(threshold):
    print(f'|----- Trying difference threshold of {round(thr, 2)} -----|')
    start_time = time.time()
    
    #Copies variables needed
    val_dataset = o_val_dataset.copy()
    neurons = o_neurons
    decoder = o_decoder
    
    #Updates the neurons
    decoder, neurons = remove_neurons(predicted_matrix, observed_matrix, val_dataset, neurons, thr, metric='difference')
    print(f'Nº of accepted neurons: {len(neurons)}')
    print(f'Nº of removed neurons: {len(standard_neurons)-len(neurons)}')

    #Predicts every angle according to a decoder
    angle_predictor(val_dataset, decoder, 31, neurons, pred_pos=False)

    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(val_dataset, metric, extra_metrics=True)
        if(np.isnan([mean, std, r2_mean, r2_std]).any()): 
            threshold = threshold[:i]
            print('NaN found -> Early Stop')
            break
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)
        
    if(np.isnan([mean, std, r2_mean, r2_std]).any()): break
        
    data = np.array(data)
    entries = np.array(entries)
    parameters.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

    end_time = time.time()
    elapsed_time = round(end_time - start_time,3)
    print(f'It took {elapsed_time} sec\n')

parameters = np.array(parameters)
all_entries = np.array(all_entries)
print(f'It took a total time of {round(time.time() - total_time,3)}')

In [ ]:
title_name = 'Neuron Hacking for Population Vector'
label_name = 'Difference Threshold'
save_name = 'ppv_neuron_removal_hack_r2.png'
eval='R2'
compare_models(parameters, all_entries, np.round(threshold, 2), eval=eval, title_name=title_name, label_name=label_name, 
               save=True, save_name=save_name)

### Angle Resolution Hacking

In [ ]:
os.system('afplay /System/Library/Sounds/Glass.aiff')
'''
Optimizes the Population Vector for angle resolution (With previous R2 optimizations)
'''
total_time = time.time()

#|----------Gets the Datasets Info----------|
print('Starting Dataset Preparation')
start_time = time.time()

train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, 1)
o_train_dataset = train_sets[0]
o_val_dataset = val_sets[0]
o_neurons = standard_neurons

end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'It took {elapsed_time} sec')

#|----------Preparing for Hacking----------|
print('Starting Hacking Preparation')
start_time = time.time()

thresholds = np.linspace(6, 360, 50, dtype=int)
metrics = [['mx', 'pred_cosX'],['my', 'pred_cosY'],['angle', 'pred_angle']]
parameters = []
all_entries = []

o_decoder = angle_decoder(o_train_dataset, 31, o_neurons)

end_time = time.time()
elapsed_time = round(end_time - start_time,3)
print(f'It took {elapsed_time} sec')

#|----------Hyperparametrization Hacking----------|
print('Hacking Starting\n')
for thr in thresholds:
    print(f'|----- Angle Resolution of {thr} -----|')
    start_time = time.time()
    
    #Generates a decoder
    train_dataset = o_train_dataset.copy()
    val_dataset = o_val_dataset.copy()
    neurons = o_neurons
    decoder = o_decoder
    
    #Removes Neurons
    observed_matrix, predicted_matrix,_ = tune_neurons(train_dataset, decoder, thr, neurons)
    decoder, neurons = remove_neurons(predicted_matrix, observed_matrix, val_dataset, neurons, 11.4, metric='difference')
    
    #Predicts every angle according to a decoder
    angle_predictor(val_dataset, decoder, 31, neurons, pred_pos=False)

    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(val_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

    end_time = time.time()
    elapsed_time = round(end_time - start_time,3)
    print(f'It took {elapsed_time} sec\n')

os.system('afplay /System/Library/Sounds/Glass.aiff')
parameters = np.array(parameters)
all_entries = np.array(all_entries)
print(f'It took a total time of {round(time.time() - total_time,3)}')

In [ ]:
title_name = 'Angle Resolution Hacking for Population Vector'
label_name = 'Resolution (nº of angles)'
save_name = 'ppv_angle_res_hack_pearson.png'
eval='Pearson'
compare_models(parameters, all_entries, np.round(thresholds, 2), eval=eval, title_name=title_name, label_name=label_name, 
               save=True, save_name=save_name)

## Naive Bayes

### Resolution Hacking

In [ ]:
'''
Simple Bayes Decoder - Parametrization Hacking #1 (Velocity Resolution)
'''

#Separates the dataset into 3 sets
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, 1)
o_train_dataset = train_sets[0]
o_val_dataset = val_sets[0]
neurons = standard_neurons
metrics = [['vel_X', 'pred_vel_X'], ['vel_Y', 'pred_vel_Y']]
print('Dataset is now divided')

#Converting sets to firing rate -> Otimization Needed!!!
firing_rate(o_val_dataset, 35, panda=True) 
firing_rate(o_train_dataset, 35, panda=True) 
firing_rate(o_test_dataset, 35, panda=True) 
print('Dataset is now in rates and not spike count')

#Run Control
parameters = []
all_entries = []
thresholds = np.linspace(10, 50, 9, dtype=int)
thresholds = np.append(thresholds,100)

print()
print('-----------------------------')
hack_time = time.time()
#Removing Neurons with depth threshold
print('Trying Velocity Resolution Hacking')
for i, thr in enumerate(thresholds):
    print()
    print(f'Starting Model were velocity\'s resolution used was {thr}')

    #Start Counting Time
    start_time = time.time()
    
    #Neuron Removal
    train_dataset = o_train_dataset.copy()
    val_dataset = o_val_dataset.copy()
    test_dataset = o_test_dataset.copy()
    
    #datasets = [train_dataset, val_dataset, test_dataset]

    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, thr)
    print('All neurons have been fitted')

    '''
    print('mean_matrix: ' + str(np.isnan(mean_matrix).any()))
    print('found_pairs: ' + str(np.isnan(found_pairs).any()))
    print('velocity_vector: ' + str(np.isnan(velocity_vector).any()))
    print('grids: ' + str(np.isnan(grids).any()))
    print('observed: ' + str(np.isnan(observed).any()))
    print('predicted: ' + str(np.isnan(predicted).any()))
    '''

    
    #acepted_neurons, acepted_predicted = remove_neurons(datasets, neurons, predicted, 'depth', min_depth=depth)
    #print('Neurons Removed')
    
    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
    pred_vel, max_prob = SNB_apply(val_dataset, decoder)
    print('Decoder Applied')
    '''
    print('decoder 0: ' + str(np.isnan(decoder[0]).any()))
    print('decoder 1: ' + str(np.isnan(decoder[1]).any()))
    print('pred_vel: ' + str(np.isnan(pred_vel).any()))
    print('max_prob: ' + str(np.isnan(max_prob).any()))
    '''

    
    #Column Management
    apply_2D_data(val_dataset, pred_vel)
    apply_2D_data(val_dataset, np.array(val_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(val_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')
    
    elapsed_depth = round(time.time() - start_time,3)
    print(f'It took {elapsed_depth} sec')
    os.system('afplay /System/Library/Sounds/Glass.aiff')

print('Velocity Resolution Hacking DONE')
elapsed_depth = round(time.time() - hack_time,3)
print(f'It took {elapsed_depth} sec')

parameters = np.array(parameters)
all_entries = np.array(all_entries)

os.system('afplay /System/Library/Sounds/Glass.aiff')
os.system('afplay /System/Library/Sounds/Glass.aiff')
os.system('afplay /System/Library/Sounds/Glass.aiff')

In [ ]:
title_name = 'Velocity Resolution for Naïve Bayes'
label_name = 'Resolution (nº of bins)'
save_name = 'nb_vel_res_hack_pearson.png'
eval='Pearson'
compare_models(parameters, all_entries, thresholds, eval=eval, title_name=title_name, label_name=label_name, save=True, save_name=save_name)

### Time Resolution Hacking

In [ ]:
'''
Simple Bayes Decoder - Parametrization Hacking #1 (Time Resolution)
'''

#Separates the dataset into 3 sets
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, 1)
o_train_dataset = train_sets[0]
o_val_dataset = val_sets[0]
neurons = standard_neurons
metrics = [['vel_X', 'pred_vel_X'], ['vel_Y', 'pred_vel_Y']]
print('Dataset is now divided')

#Run Control
parameters = []
all_entries = []
thresholds = np.linspace(1, 50, 50, dtype=int)

print()
print('-----------------------------')
hack_time = time.time()
#Removing Neurons with depth threshold
print('Trying Velocity Resolution Hacking')
for i, thr in enumerate(thresholds):
    print()
    print(f'Starting Model were time resolution used was {thr}')

    #Start Counting Time
    start_time = time.time()
    
    #Neuron Removal
    train_dataset = o_train_dataset.copy()
    val_dataset = o_val_dataset.copy()

    #Firing Rate
    firing_rate(val_dataset, thr, panda=True) 
    firing_rate(train_dataset, thr, panda=True) 

    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 15)
    print('All neurons have been fitted')
    
    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, window_size=thr)
    pred_vel, max_prob = SNB_apply(val_dataset, decoder, window_size=thr)
    print('Decoder Applied')
    
    #Column Management
    apply_2D_data(val_dataset, pred_vel)
    apply_2D_data(val_dataset, np.array(val_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(val_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')
    
    elapsed_depth = round(time.time() - start_time,3)
    print(f'It took {elapsed_depth} sec')
    os.system('afplay /System/Library/Sounds/Glass.aiff')

parameters = np.array(parameters)
all_entries = np.array(all_entries)

print('Velocity Resolution Hacking DONE')
elapsed_depth = round(time.time() - hack_time,3)
print(f'It took {elapsed_depth} sec')
os.system('afplay /System/Library/Sounds/Glass.aiff')
os.system('afplay /System/Library/Sounds/Glass.aiff')
os.system('afplay /System/Library/Sounds/Glass.aiff')

In [ ]:
title_name = 'Time Resolution for Naïve Bayes'
label_name = 'Window Size (nº of bins)'
save_name = 'nb_time_res_hack_r2.png'
eval='R2'
compare_models(parameters, all_entries, thresholds, eval=eval, title_name=title_name, label_name=label_name, save=True, save_name=save_name)

### Neurons to Remove

In [ ]:
'''
Simple Bayes Decoder - Parametrization Hacking #3 (Neurons)
'''

#Separates the dataset into 3 sets
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, 1)
o_train_dataset = train_sets[0]
o_val_dataset = val_sets[0]
neurons = standard_neurons
metrics = [['vel_X', 'pred_vel_X'], ['vel_Y', 'pred_vel_Y']]
print('Dataset is now divided')

#Converting sets to firing rate -> Otimization Needed!!!
firing_rate(o_val_dataset, 35, panda=True) 
firing_rate(o_train_dataset, 35, panda=True) 
firing_rate(o_test_dataset, 35, panda=True) 
print('Dataset is now in rates and not spike count')

#Run Control
parameters = []
all_entries = []

#Fitting the Gaussian Tuning Curve
mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(o_train_dataset, 15)
print('All neurons have been fitted')
dist = get_tune_distance(grids, observed, predicted)
thresholds = np.linspace(np.max(dist), np.min(dist), 50, endpoint=False)

print()
print('-----------------------------')
hack_time = time.time()
#Removing Neurons with depth threshold
print('Trying Neuron Hacking')
for i, thr in enumerate(thresholds):
    print()
    print(f'Starting Model were Neuron Threshold Difference used was {thr}')

    #Start Counting Time
    start_time = time.time()
    
    #Dataset Copy
    train_dataset = o_train_dataset.copy()
    val_dataset = o_val_dataset.copy()
    test_dataset = o_test_dataset.copy()
    datasets = [train_dataset, val_dataset, test_dataset]

    #Removes
    acepted_neurons, acepted_predicted = nb_remove_neurons(datasets, neurons, predicted, observed, grids, 'diff', threshold=thr)
    print('Neurons Removed')
    print(f'Nº of accepted neurons: {len(acepted_neurons)}')
    print(f'Nº of removed neurons: {len(neurons)-len(acepted_neurons)}')
    
    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, acepted_predicted)
    pred_vel, max_prob = SNB_apply(val_dataset, decoder)
    print('Decoder Applied')

    
    #Column Management
    apply_2D_data(val_dataset, pred_vel)
    apply_2D_data(val_dataset, np.array(val_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(val_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')
    
    elapsed_depth = round(time.time() - start_time,3)
    print(f'It took {elapsed_depth} sec')
    os.system('afplay /System/Library/Sounds/Glass.aiff')

print('Velocity Resolution Hacking DONE')
elapsed_depth = round(time.time() - hack_time,3)
print(f'It took {elapsed_depth} sec')

parameters = np.array(parameters)
all_entries = np.array(all_entries)

os.system('afplay /System/Library/Sounds/Glass.aiff')
os.system('afplay /System/Library/Sounds/Glass.aiff')
os.system('afplay /System/Library/Sounds/Glass.aiff')

In [ ]:
title_name = 'Neuron Removal Hacking for Naïve Bayes'
label_name = 'Distance Threhsold'
save_name = 'nb_neuron_removal_hack_pearson.png'
eval='Pearson'
compare_models(parameters, all_entries, np.round(thresholds,2), eval=eval, title_name=title_name, label_name=label_name, save=False, save_name=save_name)

### Smoothing Parameter

In [ ]:
'''
Returns all components needed for the Naive Bayses Decoder
'''
def NB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=None, bin_size=5, window_size=35):
    #Unpacking data
    tau = bin_size * window_size * 0.001
    probability_s = all_prob_output(found_pairs, velocity_vector)
    rate_data = np.array(train_dataset['spikes'])
    spike_data = np.asarray(rate_data * tau, dtype='int')
    if(spike_range == None): spike_range = np.max(spike_data)
    nr_neurons = len(spike_data[0])
    
    #Gets all neuron probability to all velocities for all spikes count in the train
    all_conditional = []
    for i, pair in enumerate(found_pairs):
        fs = vect2_twoD_Gaussian(([found_pairs[i,0]], [found_pairs[i,1]]), predicted.T)
        all_store = []
        for j in range(nr_neurons):
            f = fs[j] * tau
            all_prob = []
            for k in range(spike_range):
                probability = (np.exp(-f)*np.power(f, k))/math.factorial(k)
                all_prob.append(probability)
            all_store.append(all_prob)
        all_conditional.append(all_store)
    all_conditional = np.array(all_conditional) #found_pair, neuron, spike_count

    #Gets the distance probability matrix
    std = get_train_std(train_dataset)
    probability_dist = prob_distances(found_pairs, std*50)
    return all_conditional, probability_s, probability_dist, found_pairs

In [ ]:
'''
Fits a gaussian to the distances and visualizes them
'''
def prob_distances(found_pairs, std, plot=None, cmap='hot', dist_alpha=0, prob_alpha=1, save=False, save_name='plot.png'):
    #Get the distances between each pair in a square matrix (found_pairs x found_pairs)
    general_dists = squareform(pdist(found_pairs))
    print('general_dists')
    print(general_dists.shape)
    print(type(general_dists))
    print(general_dists)
    print()
    extra_prob = norm.pdf(general_dists, 0, std)
    
    #Ploting if not None
    if(plot == '3D'):
        fig = plt.figure(figsize=(20,20))
        ax = fig.add_subplot(projection='3d')
        size = len(found_pairs)
        grid_x, grid_y = np.mgrid[0:size:size*1j, 0:size:size*1j]
        if(dist_alpha != 0):
            ax.plot_surface(grid_x, grid_y, general_dists, cmap=cmap, edgecolor='none', alpha=dist_alpha)
        ax.plot_surface(grid_x, grid_y, extra_prob, cmap=cmap, edgecolor='none', alpha=prob_alpha)
        ax.set_xlabel('Found_Pairs')
        ax.set_ylabel('Found_Pairs - 1')
        ax.set_zlabel('Distances')
        
        if(view != None):
            ax.view_init(view[0], view[1], view[2])
    
        plt.title('Distances', fontsize=20)
        if(save): plt.savefig(save_name)
        plt.show()
        
    elif(plot == 'heat'):
        # Create a figure with two subplots
        fig, axs = plt.subplots(1, 2, figsize=(20, 8))
        
        # Plot the first heatmap on the left subplot
        im1 = axs[0].imshow(extra_prob.T, cmap=cmap, origin='lower')
        cb_max1 = np.nanmax(extra_prob)
        cb_min1 = np.nanmin(extra_prob)
        cb_range1 = np.linspace(cb_min1, cb_max1, 10)
        fig.colorbar(im1, ax=axs[0], ticks=cb_range1, label='P(s|s-1)')
        axs[0].set_title('Heat Map of Distance Probability', fontsize=20)
        axs[0].set_xlabel('Found Pairs')
        axs[0].set_ylabel('Found Pairs - 1')
        
        # Plot the second heatmap on the right subplot
        im2 = axs[1].imshow(general_dists.T, cmap=cmap, origin='lower', extent=(0,1790,0,1790))  
        cb_max2 = np.nanmax(general_dists)  
        cb_min2 = np.nanmin(general_dists)  
        cb_range2 = np.linspace(cb_min2, cb_max2, 10)  
        fig.colorbar(im2, ax=axs[1], ticks=cb_range2, label='Distance')  
        axs[1].set_title('Heat Map of Distances', fontsize=20)
        axs[1].set_xlabel('Found Pairs')
        axs[1].set_ylabel('Found Pairs - 1')
    
        #Save plot
        if save:
            plt.savefig(save_name)
        plt.show()
    return extra_prob

In [ ]:
'''
Gets the distances (euclidean) between all training dataset velocities and their std
'''
def get_train_std(train_dataset, found_pairs, get_distance=False):
    #Data extraction
    velocities = np.array(train_dataset['hand_vel'])

    #Get distance
    for i in range(n-1):
        dx[i]=np.sqrt((y_train[i+1,0]-y_train[i,0])**2+(y_train[i+1,1]-y_train[i,1])**2)
    
    delta = velocities[1:] - velocities[:-1]
    distances = np.sqrt(np.sum(delta**2, axis=1))
    std = np.std(distances)

    if(get_distance): return std, distances
    else: return std

In [ ]:
'''
Applies the Naive Bayes Decoder
'''
def NB_apply(dataset, decoder, bin_size=5, window_size=35):
    #Data Extraction
    all_conditional, probability_s, probability_dist, found_pairs = decoder
    tau = bin_size * window_size * 0.001
    rate_data = np.array(dataset['spikes'])
    spike_data = np.asarray(rate_data * tau, dtype='int')

    #Gets all trial changes boolean
    _, first_occurrences = np.unique(trial_ids, return_index=True)
    trial_change = np.zeros_like(trial_ids, dtype=bool)
    trial_change[first_occurrences] = True
    
    #Loop Apply - Tries all s for each entry and returns the most likely -> Can be optimized (parallelization)
    max_prob = []
    directions = []
    i_found_pairs = np.arange(len(found_pairs))
    i_neurons = np.arange(len(spike_data[0]))
    
    for k in range(len(spike_data)):
        spikes = spike_data[k]
        values = all_conditional[i_found_pairs[:, np.newaxis], i_neurons, spikes]
        conditional_vector = np.prod(values, axis=1)
        if(trial_change[k]): probability_vector = conditional_vector #Firs entry of each trial should not check past
        else: 
            previos_loc = directions[-1]
            loc_index = np.where(np.all(found_pairs == directions[-1], axis=1))[0][0]
            probability_vector = conditional_vector * probability_dist[loc_index]
        direction_loc = np.argmax(probability_vector)
        max_prob.append(probability_vector[direction_loc])
        directions.append(found_pairs[direction_loc])
    
    max_prob = np.array(max_prob)
    directions = np.array(directions)

    return directions, max_prob

In [ ]:
prob_distances(found_pairs, 10)

In [ ]:
n=100
dx = np.zeros([n-1,1])
noise = np.random.uniform(low=-0.0001, high=0.0001, size=dx.shape)
dx = dx + noise
np.sqrt(np.mean(dx**2))

In [ ]:
y_train = np.random.rand(100,2)
y_train

In [ ]:
n=y_train.shape[0]
dx=np.zeros([n-1,1])
for i in range(n-1):
    dx[i]=np.sqrt((y_train[i+1,0]-y_train[i,0])**2+(y_train[i+1,1]-y_train[i,1])**2) #Change in state across time steps
std=np.sqrt(np.mean(dx**2))

In [ ]:
std

In [ ]:
dx

### Surface Hacking

In [ ]:
'''
Cauchy Surface used to fit tuning curves
'''
def cauchy_surface(xy, amplitude, xo, yo, l, offset):
    x, y = xy
    up1 = l**2
    down1 = (x-xo)**2+(y-yo)**2+up1
    down2 = np.pi * l
    return (up1/down1)*(1/down2)*amplitude + offset

In [ ]:
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, 1)
o_train_dataset = train_sets[0]
o_val_dataset = val_sets[0]
firing_rate(o_val_dataset, 35, panda=True) 
firing_rate(o_train_dataset, 35, panda=True) 
firing_rate(o_test_dataset, 35, panda=True)
neurons = standard_neurons

In [ ]:
train_dataset = o_train_dataset.copy()
val_dataset = o_val_dataset.copy()

In [ ]:
mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, model='Cauchy', resolution=15)

In [ ]:
plot_heatmap_all(mean_matrix, neurons, found_pairs, grids, observed, predicted, nr_cols=5, model='Cauchy', uniform=False, 
                 save=False, save_name='cauchy_tuning2.pdf')

In [ ]:
plot_tune3D_single(mean_matrix, found_pairs, 1, grids, observed, predicted, model='Cauchy',
                       view=None, fill_nan=False, view_points=False, observed_alpha=0.1, predicted_alpha=0.6)

In [ ]:
metrics = [['vel_X', 'pred_vel_X'], ['vel_Y', 'pred_vel_Y']]

decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, model='Cauchy')
pred_vel, max_prob = SNB_apply(val_dataset, decoder)

#Column Management
apply_2D_data(val_dataset, pred_vel)
apply_2D_data(val_dataset, np.array(val_dataset['hand_vel']), col_name='vel')
print('Column Management Done')

#Gets correlation values
data = []
entries = []
for metric in metrics:
    mean, std, r2_mean, r2_std, entry = get_pearson(val_dataset, metric, extra_metrics=True)
    data.append([mean, std, r2_mean, r2_std])
    entries.append(entry)

data = np.array(data)
entries = np.array(entries)
print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

## Final Models

In [ ]:
'''
Run 10 fold comparison model
'''

#Separates the dataset into 3 sets
folds = 10
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, folds) 
o_neurons = standard_neurons
print('Dataset is now divided')

#Prepare Arrays
parameters = []
all_entries = []

#Runs the 10 fold for population vector
metrics = [['mx', 'pred_cosX'],['my', 'pred_cosY']]

print()
print('--------------Population Vector---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_test_dataset.copy()
    neurons = o_neurons

    #Build the decoder
    decoder = angle_decoder(train_dataset, 31, neurons)
   
    #Updates the neurons
    observed_matrix, predicted_matrix,_ = tune_neurons(train_dataset, decoder, 345, neurons)
    decoder, neurons = remove_neurons(predicted_matrix, observed_matrix, test_dataset, neurons, 11.4, metric='difference')
    
    #Predicts every angle according to a decoder
    angle_predictor(test_dataset, decoder, 31, neurons, pred_pos=False)
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

#Runs the 10 fold for Naive Bayes
metrics = [['vel_X', 'pred_vel_X'], ['vel_Y', 'pred_vel_Y']]
firing_rate(o_test_dataset, 35, panda=True)

print()
print('--------------Naive Bayes---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_test_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, 35, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
    pred_vel, max_prob = SNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')
parameters = np.array(parameters)
all_entries = np.array(all_entries)

In [ ]:
parameters
all_entries
model_names = ['Population Vector (PPV)', 'Naive Bayes (NB)', 'Golden Standard (MINT)']
nr_folds = 10
eval='R2'
title_name='Final Models Analysis'
label_name='Final Models'
save=False
save_name='compare_ppv.png'

'''
Compare final models
'''

# Data extraction --> Models
model_1_parameters = parameters[0:10]
model_2_parameters = parameters[10:]

nr_trials = len(parameters)
if(eval == 'Pearson'):
    x_std = np.array([np.std(model_1_parameters[:,0,1]), np.std(model_2_parameters[:,0,1])])
    y_std = np.array([np.std(model_1_parameters[:,1,1]), np.std(model_2_parameters[:,1,1])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,0]), np.mean(model_2_parameters[:,0,0])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,0]), np.mean(model_2_parameters[:,1,0])])
    f_model1 = np.mean((model_1_parameters[:,0,0], model_1_parameters[:,1,0]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,0], model_2_parameters[:,1,0]), axis=0)

elif(eval == 'R2'):
    x_std = np.array([np.std(model_1_parameters[:,0,3]), np.std(model_2_parameters[:,0,3])])
    y_std = np.array([np.std(model_1_parameters[:,1,3]), np.std(model_2_parameters[:,1,3])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,2]), np.mean(model_2_parameters[:,0,2])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,2]), np.mean(model_2_parameters[:,1,2])])
    f_model1 = np.mean((model_1_parameters[:,0,2], model_1_parameters[:,1,2]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,2], model_2_parameters[:,1,2]), axis=0)

m_mean = np.mean((x_mean, y_mean), axis=0)
m_std = np.mean((x_std, y_std), axis=0)

# Extra Variables
error = m_std / np.sqrt(nr_trials)
error = m_std / np.sqrt(nr_trials)

#Begin Plotting
plt.figure(figsize=(6, 6), layout='constrained')
colors = ['tomato', 'dodgerblue', 'orange', 'dodgerblue', 'black']

#Scatter plotting
for i in range(nr_folds):
    plt.scatter(0, f_model1[i], color=colors[0], marker='x', alpha=0.5)
    plt.scatter(1, f_model2[i], color=colors[1], marker='x', alpha=0.5)

#Error bae plotting
plt.errorbar(0, m_mean[0], error[0], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[0], 
             label=f'Mean {eval} for PPV: {round(m_mean[0],2)}')
'''
plt.errorbar(1, m_mean[1], error[1], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[1],
             label=f'Mean {eval} for NB: {round(m_mean[1],2)}')
#plt.errorbar(2, 0.85, error[1], ls='none', elinewidth=1, capthick=1, capsize=10, color=colors[2], label=f'Gold Standard {eval}: 0.85')
plt.scatter(2, 0.85, color=colors[2], alpha=1, marker='x', linewidth=10, label=f'Gold Standard {eval}: 0.85')
'''

# Add labels, title, and custom x-axis tick labels for the left y-axis
plt.ylabel(eval)
plt.title(title_name)
#ax1.set_xlabel(label_name)
plt.xticks(np.arange(len(model_names)), labels=np.array(model_names))
plt.ylim(m_mean[0]-0.01, m_mean[0]+0.01)
#plt.ylim(0, 1)
plt.xlim(-0.5, 0.5)
plt.legend(loc='upper left')

# Customize the grid
plt.grid(True)
'''
plt.grid(which='major', color='k', linestyle='-', linewidth=0.5)
plt.grid(which='minor', color='k', linestyle='-', linewidth=0.5, alpha=0.25)

# Enable minor ticks
plt.gca().minorticks_on()

# Customize the number of minor ticks
plt.gca().xaxis.set_minor_locator(AutoMinorLocator(10))  # 4 minor ticks between major ticks on x-axis
plt.gca().yaxis.set_minor_locator(AutoMinorLocator(4))  # 5 minor ticks between major ticks on y-axis
'''
# Save figure if required
if save: plt.savefig(save_name)

# Show plot
plt.show()

In [ ]:
found_pairs[2,0] - found_pairs[1,0]

## p_Values

In [ ]:
# Perform one-sample t-test
t_statistic, p_value = stats.ttest_1samp(f_model1, 0)

# Print the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret the result
if p_value < 0.005:  # assuming a significance level of 0.05
    print("The mean of the data is significantly greater than 0.")
else:
    print("The mean of the data is not significantly different from 0.")

In [ ]:
# Perform two-sample t-test
t_statistic, p_value = stats.ttest_ind(f_model2, f_model1, alternative='greater')

# Print the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret the result
if p_value < 0.05:  # assuming a significance level of 0.05
    print("The mean of f_model2 is significantly greater than the mean of f_model1.")
else:
    print("The mean of f_model2 is not significantly greater than the mean of f_model1.")

In [ ]:
t_statistic, p_value = stats.ttest_1samp(f_model2, 0.85, alternative='less')

# Print the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret the result
if p_value < 0.05:  # assuming a significance level of 0.05
    print(f"The value {0.85} is significantly greater than the mean of f_model2.")
else:
    print(f"The value {0.85} is not significantly greater than the mean of f_model2.")

## Extra

### Comparing Walls Vs No Walls

In [ ]:
folds = 10
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, folds) 

In [ ]:
train_dataset = train_sets[0].copy()
test_dataset = o_test_dataset.copy()
neurons = o_neurons
metrics = [['mx', 'pred_cosX'],['my', 'pred_cosY']]
metrics = [['vel_X', 'pred_vel_X'], ['vel_Y', 'pred_vel_Y']]

#Gets the test set with direct and indirect separation
trials = trials_present(test_dataset)
trials_all_direct = np.array(standardDS.trial_info[standardDS.trial_info['num_barriers'] == 0]['trial_id'])
trials_test_direct = trials[np.isin(trials, trials_all_direct)]

direct_test_dataset = test_dataset[test_dataset['trial_id'].isin(trials_all_direct)].copy()
indirect_test_dataset = test_dataset[~test_dataset['trial_id'].isin(trials_all_direct)].copy()

In [ ]:
#Build the decoder
decoder = angle_decoder(train_dataset, 31, neurons)

#Updates the neurons
observed_matrix, predicted_matrix,_ = tune_neurons(train_dataset, decoder, 345, neurons)
decoder, neurons = remove_neurons(predicted_matrix, observed_matrix, direct_test_dataset, neurons, 11.4, metric='difference')

#Predicts every angle according to a decoder
angle_predictor(direct_test_dataset, decoder, 31, neurons, pred_pos=False)

#Gets correlation values
data = []
entries = []
for metric in metrics:
    mean, std, r2_mean, r2_std, entry = get_pearson(direct_test_dataset, metric, extra_metrics=True)
    data.append([mean, std, r2_mean, r2_std])
    entries.append(entry)

data = np.array(data)
entries = np.array(entries)
print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

In [ ]:
#Build the decoder
decoder = angle_decoder(train_dataset, 31, neurons)

#Updates the neurons
observed_matrix, predicted_matrix,_ = tune_neurons(train_dataset, decoder, 345, neurons)
decoder, neurons = remove_neurons(predicted_matrix, observed_matrix, indirect_test_dataset, neurons, 11.4, metric='difference')

#Predicts every angle according to a decoder
angle_predictor(indirect_test_dataset, decoder, 31, neurons, pred_pos=False)

#Gets correlation values
data = []
entries = []
for metric in metrics:
    mean, std, r2_mean, r2_std, entry = get_pearson(indirect_test_dataset, metric, extra_metrics=True)
    data.append([mean, std, r2_mean, r2_std])
    entries.append(entry)

data = np.array(data)
entries = np.array(entries)
print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

In [ ]:
#Converting sets to firing rate -> Otimization Needed!!!
firing_rate(train_dataset, 35, panda=True) 
firing_rate(direct_test_dataset, 35, panda=True) 
print('Dataset is now in rates and not spike count')

#Fitting the Gaussian Tuning Curve
mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
print('All neurons have been fitted')


#Decoding Moment
decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
pred_vel, max_prob = SNB_apply(direct_test_dataset, decoder)
print('Decoder Applied')

#Column Management
apply_2D_data(direct_test_dataset, pred_vel)
apply_2D_data(direct_test_dataset, np.array(direct_test_dataset['hand_vel']), col_name='vel')
print('Column Management Done')

#Gets correlation values
data = []
entries = []
for metric in metrics:
    mean, std, r2_mean, r2_std, entry = get_pearson(direct_test_dataset, metric, extra_metrics=True)
    data.append([mean, std, r2_mean, r2_std])
    entries.append(entry)

data = np.array(data)
entries = np.array(entries)
print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

In [ ]:
#Converting sets to firing rate -> Otimization Needed!!!
firing_rate(train_dataset, 35, panda=True) 
firing_rate(indirect_test_dataset, 35, panda=True) 
print('Dataset is now in rates and not spike count')

#Fitting the Gaussian Tuning Curve
mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
print('All neurons have been fitted')


#Decoding Moment
decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
pred_vel, max_prob = SNB_apply(indirect_test_dataset, decoder)
print('Decoder Applied')

#Column Management
apply_2D_data(indirect_test_dataset, pred_vel)
apply_2D_data(indirect_test_dataset, np.array(indirect_test_dataset['hand_vel']), col_name='vel')
print('Column Management Done')

#Gets correlation values
data = []
entries = []
for metric in metrics:
    mean, std, r2_mean, r2_std, entry = get_pearson(indirect_test_dataset, metric, extra_metrics=True)
    data.append([mean, std, r2_mean, r2_std])
    entries.append(entry)

data = np.array(data)
entries = np.array(entries)
print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

### Official Comparison

In [ ]:
'''
Run 10 fold comparison model of No Walls Vs Walls Population Vector
'''

#Separates the dataset into 3 sets
folds = 10
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, folds) 
o_neurons = standard_neurons

#Gets the test set with direct and indirect separation
trials = trials_present(o_test_dataset)
trials_all_direct = np.array(standardDS.trial_info[standardDS.trial_info['num_barriers'] == 0]['trial_id'])
trials_test_direct = trials[np.isin(trials, trials_all_direct)]

#Gets the separated test set
o_direct_dataset = o_test_dataset[o_test_dataset['trial_id'].isin(trials_all_direct)].copy()
o_indirect_dataset = o_test_dataset[~o_test_dataset['trial_id'].isin(trials_all_direct)].copy()

print('Dataset is now divided')

#Prepare Arrays
parameters = []
all_entries = []

#Runs the 10 fold for population vector
metrics = [['mx', 'pred_cosX'],['my', 'pred_cosY']]

print()
print('--------------Direct Test---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_direct_dataset.copy()
    neurons = o_neurons

    #Build the decoder
    decoder = angle_decoder(train_dataset, 31, neurons)
   
    #Updates the neurons
    observed_matrix, predicted_matrix,_ = tune_neurons(train_dataset, decoder, 345, neurons)
    decoder, neurons = remove_neurons(predicted_matrix, observed_matrix, test_dataset, neurons, 11.4, metric='difference')
    
    #Predicts every angle according to a decoder
    angle_predictor(test_dataset, decoder, 31, neurons, pred_pos=False)
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Indirect Test---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_indirect_dataset.copy()
    neurons = o_neurons

    #Build the decoder
    decoder = angle_decoder(train_dataset, 31, neurons)
   
    #Updates the neurons
    observed_matrix, predicted_matrix,_ = tune_neurons(train_dataset, decoder, 345, neurons)
    decoder, neurons = remove_neurons(predicted_matrix, observed_matrix, test_dataset, neurons, 11.4, metric='difference')
    
    #Predicts every angle according to a decoder
    angle_predictor(test_dataset, decoder, 31, neurons, pred_pos=False)
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')
parameters = np.array(parameters)

In [ ]:
parameters
all_entries
model_names = ['Simple Movement (No Walls)', 'Complex Movement (Walls)']
nr_folds = 10
eval='R2'
title_name='Population Vector - Wall Presence Analysis'
label_name='Models'
save=True
save_name='ppv_walls_compare_r2.png'

'''
Compare final models
'''

# Data extraction --> Models
model_1_parameters = parameters[0:10]
model_2_parameters = parameters[10:]

nr_trials = len(parameters)
if(eval == 'Pearson'):
    x_std = np.array([np.std(model_1_parameters[:,0,1]), np.std(model_2_parameters[:,0,1])])
    y_std = np.array([np.std(model_1_parameters[:,1,1]), np.std(model_2_parameters[:,1,1])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,0]), np.mean(model_2_parameters[:,0,0])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,0]), np.mean(model_2_parameters[:,1,0])])
    f_model1 = np.mean((model_1_parameters[:,0,0], model_1_parameters[:,1,0]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,0], model_2_parameters[:,1,0]), axis=0)

elif(eval == 'R2'):
    x_std = np.array([np.std(model_1_parameters[:,0,3]), np.std(model_2_parameters[:,0,3])])
    y_std = np.array([np.std(model_1_parameters[:,1,3]), np.std(model_2_parameters[:,1,3])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,2]), np.mean(model_2_parameters[:,0,2])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,2]), np.mean(model_2_parameters[:,1,2])])
    f_model1 = np.mean((model_1_parameters[:,0,2], model_1_parameters[:,1,2]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,2], model_2_parameters[:,1,2]), axis=0)

m_mean = np.mean((x_mean, y_mean), axis=0)
m_std = np.mean((x_std, y_std), axis=0)

# Extra Variables
error = m_std / np.sqrt(nr_trials)
error = m_std / np.sqrt(nr_trials)

#Begin Plotting
plt.figure(figsize=(6, 6), layout='constrained')
colors = ['tomato', 'dodgerblue', 'orange', 'dodgerblue', 'black']

#Scatter plotting
for i in range(nr_folds):
    plt.scatter(0, f_model1[i], color=colors[0], marker='x', alpha=0.5)
    plt.scatter(1, f_model2[i], color=colors[1], marker='x', alpha=0.5)

#Error bae plotting
plt.errorbar(0, m_mean[0], error[0], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[0], 
             label=f'Mean {eval} for Simple Movements: {round(m_mean[0],2)}')
plt.errorbar(1, m_mean[1], error[1], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[1],
             label=f'Mean {eval} for Complex Movements: {round(m_mean[1],2)}')
'''
#plt.errorbar(2, 0.85, error[1], ls='none', elinewidth=1, capthick=1, capsize=10, color=colors[2], label=f'Gold Standard {eval}: 0.85')
plt.scatter(2, 0.85, color=colors[2], alpha=1, marker='x', linewidth=10, label=f'Gold Standard {eval}: 0.85')
'''

# Add labels, title, and custom x-axis tick labels for the left y-axis
plt.ylabel(eval)
plt.title(title_name)
#ax1.set_xlabel(label_name)
plt.xticks(np.arange(len(model_names)), labels=np.array(model_names))
plt.ylim(m_mean[0]-0.01, m_mean[1]+0.01)
#plt.ylim(0, 1)
plt.xlim(-0.5, 1.5)
plt.legend(loc='upper left')

# Customize the grid
plt.grid(True)
'''
plt.grid(which='major', color='k', linestyle='-', linewidth=0.5)
plt.grid(which='minor', color='k', linestyle='-', linewidth=0.5, alpha=0.25)

# Enable minor ticks
plt.gca().minorticks_on()

# Customize the number of minor ticks
plt.gca().xaxis.set_minor_locator(AutoMinorLocator(10))  # 4 minor ticks between major ticks on x-axis
plt.gca().yaxis.set_minor_locator(AutoMinorLocator(4))  # 5 minor ticks between major ticks on y-axis
'''
# Save figure if required
if save: plt.savefig(save_name)

# Show plot
plt.show()

In [ ]:
# Perform two-sample t-test
t_statistic, p_value = stats.ttest_ind(f_model2, f_model1, alternative='greater')

# Print the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret the result
if p_value < 0.05:  # assuming a significance level of 0.05
    print("The mean of f_model2 is significantly greater than the mean of f_model1.")
else:
    print("The mean of f_model2 is not significantly greater than the mean of f_model1.")

In [ ]:
'''
Run 10 fold comparison model of No Walls Vs Walls Naïve Bayes
'''

#Separates the dataset into 3 sets
folds = 10
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, folds) 
o_neurons = standard_neurons

#Gets the test set with direct and indirect separation
trials = trials_present(o_test_dataset)
trials_all_direct = np.array(standardDS.trial_info[standardDS.trial_info['num_barriers'] == 0]['trial_id'])
trials_test_direct = trials[np.isin(trials, trials_all_direct)]

#Gets the separated test set
firing_rate(o_test_dataset, 35, panda=True)
o_direct_dataset = o_test_dataset[o_test_dataset['trial_id'].isin(trials_all_direct)].copy()
o_indirect_dataset = o_test_dataset[~o_test_dataset['trial_id'].isin(trials_all_direct)].copy()

print('Dataset is now divided')

#Prepare Arrays
parameters2 = []
all_entries = []

#Runs the 10 fold for Naive Bayes
metrics = [['vel_X', 'pred_vel_X'], ['vel_Y', 'pred_vel_Y']]

print()
print('--------------Direct Test---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_direct_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, 35, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
    pred_vel, max_prob = SNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters2.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Indirect Test---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_indirect_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, 35, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
    pred_vel, max_prob = SNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters2.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')
parameters2 = np.array(parameters2)
all_entries = np.array(all_entries)

In [ ]:
parameters
all_entries
model_names = ['Simple Movement (No Walls)', 'Complex Movement (Walls)']
nr_folds = 10
eval='R2'
title_name='Naïve Bayes - Wall Presence Analysis'
label_name='Models'
save=False
save_name='nb_walls_compare_r2.png'

'''
Compare final models
'''

# Data extraction --> Models
model_1_parameters = parameters[0:10]
model_2_parameters = parameters[10:]

nr_trials = len(parameters)
if(eval == 'Pearson'):
    x_std = np.array([np.std(model_1_parameters[:,0,1]), np.std(model_2_parameters[:,0,1])])
    y_std = np.array([np.std(model_1_parameters[:,1,1]), np.std(model_2_parameters[:,1,1])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,0]), np.mean(model_2_parameters[:,0,0])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,0]), np.mean(model_2_parameters[:,1,0])])
    f_model1 = np.mean((model_1_parameters[:,0,0], model_1_parameters[:,1,0]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,0], model_2_parameters[:,1,0]), axis=0)

elif(eval == 'R2'):
    x_std = np.array([np.std(model_1_parameters[:,0,3]), np.std(model_2_parameters[:,0,3])])
    y_std = np.array([np.std(model_1_parameters[:,1,3]), np.std(model_2_parameters[:,1,3])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,2]), np.mean(model_2_parameters[:,0,2])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,2]), np.mean(model_2_parameters[:,1,2])])
    f_model1 = np.mean((model_1_parameters[:,0,2], model_1_parameters[:,1,2]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,2], model_2_parameters[:,1,2]), axis=0)

m_mean = np.mean((x_mean, y_mean), axis=0)
m_std = np.mean((x_std, y_std), axis=0)

# Extra Variables
error = m_std / np.sqrt(nr_trials)
error = m_std / np.sqrt(nr_trials)

#Begin Plotting
plt.figure(figsize=(6, 6), layout='constrained')
colors = ['tomato', 'dodgerblue', 'orange', 'dodgerblue', 'black']

#Scatter plotting
for i in range(nr_folds):
    plt.scatter(0, f_model1[i], color=colors[0], marker='x', alpha=0.5)
    plt.scatter(1, f_model2[i], color=colors[1], marker='x', alpha=0.5)

#Error bae plotting
plt.errorbar(0, m_mean[0], error[0], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[0], 
             label=f'Mean {eval} for Simple Movements: {round(m_mean[0],2)}')
plt.errorbar(1, m_mean[1], error[1], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[1],
             label=f'Mean {eval} for Complex Movements: {round(m_mean[1],2)}')
'''
#plt.errorbar(2, 0.85, error[1], ls='none', elinewidth=1, capthick=1, capsize=10, color=colors[2], label=f'Gold Standard {eval}: 0.85')
plt.scatter(2, 0.85, color=colors[2], alpha=1, marker='x', linewidth=10, label=f'Gold Standard {eval}: 0.85')
'''

# Add labels, title, and custom x-axis tick labels for the left y-axis
plt.ylabel(eval)
plt.title(title_name)
#ax1.set_xlabel(label_name)
plt.xticks(np.arange(len(model_names)), labels=np.array(model_names))
plt.ylim(m_mean[0]-0.015, m_mean[1]+0.015)
#plt.ylim(0, 1)
plt.xlim(-0.5, 1.5)
plt.legend(loc='upper left')

# Customize the grid
plt.grid(True)
'''
plt.grid(which='major', color='k', linestyle='-', linewidth=0.5)
plt.grid(which='minor', color='k', linestyle='-', linewidth=0.5, alpha=0.25)

# Enable minor ticks
plt.gca().minorticks_on()

# Customize the number of minor ticks
plt.gca().xaxis.set_minor_locator(AutoMinorLocator(10))  # 4 minor ticks between major ticks on x-axis
plt.gca().yaxis.set_minor_locator(AutoMinorLocator(4))  # 5 minor ticks between major ticks on y-axis
'''
# Save figure if required
if save: plt.savefig(save_name)

# Show plot
plt.show()

In [ ]:
# Perform two-sample t-test
t_statistic, p_value = stats.ttest_ind(f_model2, f_model1, alternative='greater')

# Print the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret the result
if p_value < 0.05:  # assuming a significance level of 0.05
    print("The mean of f_model2 is significantly greater than the mean of f_model1.")
else:
    print("The mean of f_model2 is not significantly greater than the mean of f_model1.")

In [ ]:
'''
Run 10 fold comparison model of No Walls Vs Walls Naïve Bayes
'''

#Separates the dataset into 3 sets
folds = 10
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, folds) 
o_neurons = standard_neurons

#Gets the test set with direct and indirect separation
trials = trials_present(o_test_dataset)
trials_all_direct = np.array(standardDS.trial_info[standardDS.trial_info['num_barriers'] == 0]['trial_id'])
trials_test_direct = trials[np.isin(trials, trials_all_direct)]

#Gets the separated test set
firing_rate(o_test_dataset, 35, panda=True)
o_direct_dataset = o_test_dataset[o_test_dataset['trial_id'].isin(trials_all_direct)].copy()
o_indirect_dataset = o_test_dataset[~o_test_dataset['trial_id'].isin(trials_all_direct)].copy()

print('Dataset is now divided')

#Prepare Arrays
parameters3 = []
all_entries = []

#Runs the 10 fold for Naive Bayes
metrics = [['vel_X', 'pred_vel_X'], ['vel_Y', 'pred_vel_Y']]

print()
print('--------------Direct Test---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_direct_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, 35, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
    pred_vel, max_prob = SNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters3.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Direct Test (With out P(s))---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_direct_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, 35, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
    pred_vel, max_prob = RNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters3.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Indirect Test---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_indirect_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, 35, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
    pred_vel, max_prob = SNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters3.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Indirect Test (With out P(s))---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_indirect_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, 35, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
    pred_vel, max_prob = RNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters3.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')
parameters3 = np.array(parameters3)

In [ ]:
parameters
all_entries
model_names = ['Simple Movement (No Walls)', 'Simple Movement (No Walls) Modified', 'Complex Movement (Walls)', 'Complex Movement (Walls) Modified']
nr_folds = 10
eval='R2'
title_name='Naïve Bayes - Wall Presence Analysis'
label_name='Models'
save=True
save_name='nb_walls_compare_r2_prior.png'

'''
Compare final models
'''

# Data extraction --> Models
model_1_parameters = parameters3[0:10]
model_2_parameters = parameters3[10:20]
model_3_parameters = parameters3[20:30]
model_4_parameters = parameters3[30:]

nr_trials = len(parameters)
if(eval == 'Pearson'):
    x_std = np.array([np.std(model_1_parameters[:,0,1]), np.std(model_2_parameters[:,0,1])])
    y_std = np.array([np.std(model_1_parameters[:,1,1]), np.std(model_2_parameters[:,1,1])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,0]), np.mean(model_2_parameters[:,0,0])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,0]), np.mean(model_2_parameters[:,1,0])])
    f_model1 = np.mean((model_1_parameters[:,0,0], model_1_parameters[:,1,0]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,0], model_2_parameters[:,1,0]), axis=0)

elif(eval == 'R2'):
    x_std = np.array([np.std(model_1_parameters[:,0,3]), np.std(model_2_parameters[:,0,3]), np.std(model_3_parameters[:,0,3]), np.std(model_4_parameters[:,0,3])])
    y_std = np.array([np.std(model_1_parameters[:,1,3]), np.std(model_2_parameters[:,1,3]), np.std(model_3_parameters[:,1,3]), np.std(model_4_parameters[:,1,3])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,2]), np.mean(model_2_parameters[:,0,2]), np.mean(model_3_parameters[:,0,2]), np.mean(model_4_parameters[:,0,2])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,2]), np.mean(model_2_parameters[:,1,2]), np.mean(model_3_parameters[:,1,2]), np.mean(model_4_parameters[:,1,2])])
    f_model1 = np.mean((model_1_parameters[:,0,2], model_1_parameters[:,1,2]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,2], model_2_parameters[:,1,2]), axis=0)
    f_model3 = np.mean((model_3_parameters[:,0,2], model_3_parameters[:,1,2]), axis=0)
    f_model4 = np.mean((model_4_parameters[:,0,2], model_4_parameters[:,1,2]), axis=0)

m_mean = np.mean((x_mean, y_mean), axis=0)
m_std = np.mean((x_std, y_std), axis=0)

# Extra Variables
error = m_std / np.sqrt(nr_trials)

#Begin Plotting
plt.figure(figsize=(10, 10), layout='constrained')
colors = ['tomato', 'orange', 'dodgerblue', 'darkturquoise', 'black']

#Scatter plotting
for i in range(nr_folds):
    plt.scatter(0, f_model1[i], color=colors[0], marker='x', alpha=0.5)
    plt.scatter(1, f_model2[i], color=colors[1], marker='x', alpha=0.5)
    plt.scatter(2, f_model3[i], color=colors[2], marker='x', alpha=0.5)
    plt.scatter(3, f_model4[i], color=colors[3], marker='x', alpha=0.5)

#Error bae plotting
plt.errorbar(0, m_mean[0], error[0], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[0], 
             label=f'Mean {eval} for Simple Movements: {round(m_mean[0],2)}')
plt.errorbar(1, m_mean[1], error[1], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[1],
             label=f'Mean {eval} for Simple Movements Modified: {round(m_mean[1],2)}')
plt.errorbar(2, m_mean[2], error[2], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[2],
             label=f'Mean {eval} for Complex Movements: {round(m_mean[2],2)}')
plt.errorbar(3, m_mean[3], error[3], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[3],
             label=f'Mean {eval} for Complex Movements Modified: {round(m_mean[3],2)}')
'''
#plt.errorbar(2, 0.85, error[1], ls='none', elinewidth=1, capthick=1, capsize=10, color=colors[2], label=f'Gold Standard {eval}: 0.85')
plt.scatter(2, 0.85, color=colors[2], alpha=1, marker='x', linewidth=10, label=f'Gold Standard {eval}: 0.85')
'''

# Add labels, title, and custom x-axis tick labels for the left y-axis
plt.ylabel(eval)
plt.title(title_name)
#ax1.set_xlabel(label_name)
plt.xticks(np.arange(len(model_names)), labels=np.array(model_names), fontsize=8)
plt.ylim(m_mean[1]-0.015, m_mean[2]+0.015)
#plt.ylim(0, 1)
plt.xlim(-0.5, 3.5)
plt.legend(loc='upper left')

# Customize the grid
plt.grid(True)
'''
plt.grid(which='major', color='k', linestyle='-', linewidth=0.5)
plt.grid(which='minor', color='k', linestyle='-', linewidth=0.5, alpha=0.25)

# Enable minor ticks
plt.gca().minorticks_on()

# Customize the number of minor ticks
plt.gca().xaxis.set_minor_locator(AutoMinorLocator(10))  # 4 minor ticks between major ticks on x-axis
plt.gca().yaxis.set_minor_locator(AutoMinorLocator(4))  # 5 minor ticks between major ticks on y-axis
'''
# Save figure if required
if save: plt.savefig(save_name)

# Show plot
plt.show()

In [ ]:
'''
Run 10 fold comparison model of No Walls Vs Walls Naïve Bayes
'''

#Separates the dataset into 3 sets
folds = 10
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, folds) 
o_neurons = standard_neurons
window_size = 10

#Gets the test set with direct and indirect separation
trials = trials_present(o_test_dataset)
trials_all_direct = np.array(standardDS.trial_info[standardDS.trial_info['num_barriers'] == 0]['trial_id'])
trials_test_direct = trials[np.isin(trials, trials_all_direct)]

#Gets the separated test set
firing_rate(o_test_dataset, window_size, panda=True)
o_direct_dataset = o_test_dataset[o_test_dataset['trial_id'].isin(trials_all_direct)].copy()
o_indirect_dataset = o_test_dataset[~o_test_dataset['trial_id'].isin(trials_all_direct)].copy()

print('Dataset is now divided')

#Prepare Arrays
parameters4 = []
all_entries = []

#Runs the 10 fold for Naive Bayes
metrics = [['vel_X', 'pred_vel_X'], ['vel_Y', 'pred_vel_Y']]

print()
print('--------------Direct Test---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_direct_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, window_size, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted)
    pred_vel, max_prob = SNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters4.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Direct Test (With out P(s))---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_direct_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, window_size, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted)
    pred_vel, max_prob = RNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters4.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Indirect Test---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_indirect_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, window_size, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted)
    pred_vel, max_prob = SNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters4.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Indirect Test (With out P(s))---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_indirect_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, window_size, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted)
    pred_vel, max_prob = RNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters4.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')
parameters4 = np.array(parameters4)

In [ ]:
parameters
all_entries
model_names = ['Simple Movement (No Walls)', 'Simple Movement (No Walls) Modified', 'Complex Movement (Walls)', 'Complex Movement (Walls) Modified']
nr_folds = 10
eval='R2'
title_name='Naïve Bayes - Wall Presence Analysis\nWindow Size of 10 bins'
label_name='Models'
save=False
save_name='nb_walls_compare_r2_window10.png'

'''
Compare final models
'''

# Data extraction --> Models
model_1_parameters = parameters4[0:10]
model_2_parameters = parameters4[10:20]
model_3_parameters = parameters4[20:30]
model_4_parameters = parameters4[30:]

nr_trials = len(parameters)
if(eval == 'Pearson'):
    x_std = np.array([np.std(model_1_parameters[:,0,1]), np.std(model_2_parameters[:,0,1])])
    y_std = np.array([np.std(model_1_parameters[:,1,1]), np.std(model_2_parameters[:,1,1])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,0]), np.mean(model_2_parameters[:,0,0])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,0]), np.mean(model_2_parameters[:,1,0])])
    f_model1 = np.mean((model_1_parameters[:,0,0], model_1_parameters[:,1,0]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,0], model_2_parameters[:,1,0]), axis=0)

elif(eval == 'R2'):
    x_std = np.array([np.std(model_1_parameters[:,0,3]), np.std(model_2_parameters[:,0,3]), np.std(model_3_parameters[:,0,3]), np.std(model_4_parameters[:,0,3])])
    y_std = np.array([np.std(model_1_parameters[:,1,3]), np.std(model_2_parameters[:,1,3]), np.std(model_3_parameters[:,1,3]), np.std(model_4_parameters[:,1,3])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,2]), np.mean(model_2_parameters[:,0,2]), np.mean(model_3_parameters[:,0,2]), np.mean(model_4_parameters[:,0,2])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,2]), np.mean(model_2_parameters[:,1,2]), np.mean(model_3_parameters[:,1,2]), np.mean(model_4_parameters[:,1,2])])
    f_model1 = np.mean((model_1_parameters[:,0,2], model_1_parameters[:,1,2]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,2], model_2_parameters[:,1,2]), axis=0)
    f_model3 = np.mean((model_3_parameters[:,0,2], model_3_parameters[:,1,2]), axis=0)
    f_model4 = np.mean((model_4_parameters[:,0,2], model_4_parameters[:,1,2]), axis=0)

m_mean = np.mean((x_mean, y_mean), axis=0)
m_std = np.mean((x_std, y_std), axis=0)

# Extra Variables
error = m_std / np.sqrt(nr_trials)

#Begin Plotting
plt.figure(figsize=(10, 10), layout='constrained')
colors = ['tomato', 'orange', 'dodgerblue', 'darkturquoise', 'black']

#Scatter plotting
for i in range(nr_folds):
    plt.scatter(0, f_model1[i], color=colors[0], marker='x', alpha=0.5)
    #plt.scatter(1, f_model2[i], color=colors[1], marker='x', alpha=0.5)
    plt.scatter(1, f_model3[i], color=colors[2], marker='x', alpha=0.5)
    #plt.scatter(3, f_model4[i], color=colors[3], marker='x', alpha=0.5)

#Error bae plotting
plt.errorbar(0, m_mean[0], error[0], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[0], 
             label=f'Mean {eval} for Simple Movements: {round(m_mean[0],2)}')
#plt.errorbar(1, m_mean[1], error[1], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[1],
#             label=f'Mean {eval} for Simple Movements Modified: {round(m_mean[1],2)}')
plt.errorbar(1, m_mean[2], error[2], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[2],
             label=f'Mean {eval} for Complex Movements: {round(m_mean[2],2)}')
#plt.errorbar(3, m_mean[3], error[3], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[3],
#             label=f'Mean {eval} for Complex Movements Modified: {round(m_mean[3],2)}')
'''
#plt.errorbar(2, 0.85, error[1], ls='none', elinewidth=1, capthick=1, capsize=10, color=colors[2], label=f'Gold Standard {eval}: 0.85')
plt.scatter(2, 0.85, color=colors[2], alpha=1, marker='x', linewidth=10, label=f'Gold Standard {eval}: 0.85')
'''

# Add labels, title, and custom x-axis tick labels for the left y-axis
plt.ylabel(eval)
plt.title(title_name)
#ax1.set_xlabel(label_name)
plt.xticks(np.arange(len(model_names)), labels=np.array(model_names), fontsize=8)
plt.ylim(m_mean[0]-0.05, m_mean[2]+0.05)
#plt.ylim(0, 1)
plt.xlim(-0.5, 1.5)
plt.legend(loc='upper left')

# Customize the grid
plt.grid(True)
'''
plt.grid(which='major', color='k', linestyle='-', linewidth=0.5)
plt.grid(which='minor', color='k', linestyle='-', linewidth=0.5, alpha=0.25)

# Enable minor ticks
plt.gca().minorticks_on()

# Customize the number of minor ticks
plt.gca().xaxis.set_minor_locator(AutoMinorLocator(10))  # 4 minor ticks between major ticks on x-axis
plt.gca().yaxis.set_minor_locator(AutoMinorLocator(4))  # 5 minor ticks between major ticks on y-axis
'''
# Save figure if required
if save: plt.savefig(save_name)

# Show plot
plt.show()

In [ ]:
# Perform two-sample t-test
t_statistic, p_value = stats.ttest_ind(f_model1, f_model2, alternative='greater')

# Print the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret the result
if p_value < 0.05:  # assuming a significance level of 0.05
    print("The mean of f_model1 is significantly greater than the mean of f_model2.")
else:
    print("The mean of f_model1 is not significantly greater than the mean of f_model2.")

In [ ]:
# Perform two-sample t-test
t_statistic, p_value = stats.ttest_ind(f_model3, f_model4, alternative='greater')

# Print the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret the result
if p_value < 0.05:  # assuming a significance level of 0.05
    print("The mean of f_model3 is significantly greater than the mean of f_model4.")
else:
    print("The mean of f_model3 is not significantly greater than the mean of f_model4.")

In [ ]:
# Perform two-sample t-test
t_statistic, p_value = stats.ttest_ind(f_model3, f_model1, alternative='greater')

# Print the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret the result
if p_value < 0.05:  # assuming a significance level of 0.05
    print("The mean of f_model3 is significantly greater than the mean of f_model1.")
else:
    print("The mean of f_model3 is not significantly greater than the mean of f_model1.")

In [ ]:
'''
Run 10 fold comparison model of No Walls Vs Walls Naïve Bayes
'''

#Separates the dataset into 3 sets
folds = 10
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, folds) 
o_neurons = standard_neurons
window_size = 3

#Gets the test set with direct and indirect separation
trials = trials_present(o_test_dataset)
trials_all_direct = np.array(standardDS.trial_info[standardDS.trial_info['num_barriers'] == 0]['trial_id'])
trials_test_direct = trials[np.isin(trials, trials_all_direct)]

#Gets the separated test set
firing_rate(o_test_dataset, window_size, panda=True)
o_direct_dataset = o_test_dataset[o_test_dataset['trial_id'].isin(trials_all_direct)].copy()
o_indirect_dataset = o_test_dataset[~o_test_dataset['trial_id'].isin(trials_all_direct)].copy()

print('Dataset is now divided')

#Prepare Arrays
parameters5 = []
all_entries = []

#Runs the 10 fold for Naive Bayes
metrics = [['vel_X', 'pred_vel_X'], ['vel_Y', 'pred_vel_Y']]

print()
print('--------------Direct Test---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_direct_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, window_size, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted)
    pred_vel, max_prob = SNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters5.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Direct Test (With out P(s))---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_direct_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, window_size, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted)
    pred_vel, max_prob = RNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters5.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Indirect Test---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_indirect_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, window_size, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted)
    pred_vel, max_prob = SNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters5.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Indirect Test (With out P(s))---------------')
for i in range(folds):
    print()
    print(f'Starting fold {i+1}')
    train_dataset = train_sets[i].copy()
    test_dataset = o_indirect_dataset.copy()
    neurons = o_neurons
    
    #Converting sets to firing rate -> Otimization Needed!!!
    firing_rate(train_dataset, window_size, panda=True) 
    print('Dataset is now in rates and not spike count')
    
    #Fitting the Gaussian Tuning Curve
    mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
    print('All neurons have been fitted')

    #Decoding Moment
    decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted)
    pred_vel, max_prob = RNB_apply(test_dataset, decoder)
    print('Decoder Applied')

    #Column Management
    apply_2D_data(test_dataset, pred_vel)
    apply_2D_data(test_dataset, np.array(test_dataset['hand_vel']), col_name='vel')
    print('Column Management Done')
    
    #Gets correlation values
    data = []
    entries = []
    for metric in metrics:
        mean, std, r2_mean, r2_std, entry = get_pearson(test_dataset, metric, extra_metrics=True)
        data.append([mean, std, r2_mean, r2_std])
        entries.append(entry)

    data = np.array(data)
    entries = np.array(entries)
    parameters5.append(data)
    all_entries.append(entries)
    print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
    print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')
parameters5 = np.array(parameters5)

In [ ]:
parameters
all_entries
model_names = ['Simple Movement (No Walls)', 'Simple Movement (No Walls) Modified', 'Complex Movement (Walls)', 'Complex Movement (Walls) Modified']
nr_folds = 10
eval='R2'
title_name='Naïve Bayes - Wall Presence Analysis\nWindow Size of 3 bins'
label_name='Models'
save=True
save_name='nb_walls_compare_r2_window3.png'

'''
Compare final models
'''

# Data extraction --> Models
model_1_parameters = parameters5[0:10]
model_2_parameters = parameters5[10:20]
model_3_parameters = parameters5[20:30]
model_4_parameters = parameters5[30:]

nr_trials = len(parameters)
if(eval == 'Pearson'):
    x_std = np.array([np.std(model_1_parameters[:,0,1]), np.std(model_2_parameters[:,0,1])])
    y_std = np.array([np.std(model_1_parameters[:,1,1]), np.std(model_2_parameters[:,1,1])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,0]), np.mean(model_2_parameters[:,0,0])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,0]), np.mean(model_2_parameters[:,1,0])])
    f_model1 = np.mean((model_1_parameters[:,0,0], model_1_parameters[:,1,0]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,0], model_2_parameters[:,1,0]), axis=0)

elif(eval == 'R2'):
    x_std = np.array([np.std(model_1_parameters[:,0,3]), np.std(model_2_parameters[:,0,3]), np.std(model_3_parameters[:,0,3]), np.std(model_4_parameters[:,0,3])])
    y_std = np.array([np.std(model_1_parameters[:,1,3]), np.std(model_2_parameters[:,1,3]), np.std(model_3_parameters[:,1,3]), np.std(model_4_parameters[:,1,3])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,2]), np.mean(model_2_parameters[:,0,2]), np.mean(model_3_parameters[:,0,2]), np.mean(model_4_parameters[:,0,2])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,2]), np.mean(model_2_parameters[:,1,2]), np.mean(model_3_parameters[:,1,2]), np.mean(model_4_parameters[:,1,2])])
    f_model1 = np.mean((model_1_parameters[:,0,2], model_1_parameters[:,1,2]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,2], model_2_parameters[:,1,2]), axis=0)
    f_model3 = np.mean((model_3_parameters[:,0,2], model_3_parameters[:,1,2]), axis=0)
    f_model4 = np.mean((model_4_parameters[:,0,2], model_4_parameters[:,1,2]), axis=0)

m_mean = np.mean((x_mean, y_mean), axis=0)
m_std = np.mean((x_std, y_std), axis=0)

# Extra Variables
error = m_std / np.sqrt(nr_trials)

#Begin Plotting
plt.figure(figsize=(10, 10), layout='constrained')
colors = ['tomato', 'orange', 'dodgerblue', 'darkturquoise', 'black']

#Scatter plotting
for i in range(nr_folds):
    plt.scatter(0, f_model1[i], color=colors[0], marker='x', alpha=0.5)
    #plt.scatter(1, f_model2[i], color=colors[1], marker='x', alpha=0.5)
    plt.scatter(1, f_model3[i], color=colors[2], marker='x', alpha=0.5)
    #plt.scatter(3, f_model4[i], color=colors[3], marker='x', alpha=0.5)

#Error bae plotting
plt.errorbar(0, m_mean[0], error[0], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[0], 
             label=f'Mean {eval} for Simple Movements: {round(m_mean[0],2)}')
#plt.errorbar(1, m_mean[1], error[1], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[1],
#             label=f'Mean {eval} for Simple Movements Modified: {round(m_mean[1],2)}')
plt.errorbar(1, m_mean[2], error[2], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[2],
             label=f'Mean {eval} for Complex Movements: {round(m_mean[2],2)}')
#plt.errorbar(3, m_mean[3], error[3], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[3],
#             label=f'Mean {eval} for Complex Movements Modified: {round(m_mean[3],2)}')
'''
#plt.errorbar(2, 0.85, error[1], ls='none', elinewidth=1, capthick=1, capsize=10, color=colors[2], label=f'Gold Standard {eval}: 0.85')
plt.scatter(2, 0.85, color=colors[2], alpha=1, marker='x', linewidth=10, label=f'Gold Standard {eval}: 0.85')
'''

# Add labels, title, and custom x-axis tick labels for the left y-axis
plt.ylabel(eval)
plt.title(title_name)
#ax1.set_xlabel(label_name)
plt.xticks(np.arange(len(model_names)), labels=np.array(model_names), fontsize=8)
plt.ylim(m_mean[0]-0.05, m_mean[2]+0.05)
#plt.ylim(0, 1)
plt.xlim(-0.5, 1.5)
plt.legend(loc='upper left')

# Customize the grid
plt.grid(True)
'''
plt.grid(which='major', color='k', linestyle='-', linewidth=0.5)
plt.grid(which='minor', color='k', linestyle='-', linewidth=0.5, alpha=0.25)

# Enable minor ticks
plt.gca().minorticks_on()

# Customize the number of minor ticks
plt.gca().xaxis.set_minor_locator(AutoMinorLocator(10))  # 4 minor ticks between major ticks on x-axis
plt.gca().yaxis.set_minor_locator(AutoMinorLocator(4))  # 5 minor ticks between major ticks on y-axis
'''
# Save figure if required
if save: plt.savefig(save_name)

# Show plot
plt.show()

In [ ]:
# Perform two-sample t-test
t_statistic, p_value = stats.ttest_ind(f_model3, f_model1, alternative='greater')

# Print the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret the result
if p_value < 0.05:  # assuming a significance level of 0.05
    print("The mean of f_model3 is significantly greater than the mean of f_model1.")
else:
    print("The mean of f_model3 is not significantly greater than the mean of f_model1.")

In [ ]:
parameters
all_entries
model_names = ['PPV (No Walls)', 'PPV (Walls)', 'NB (No Walls)', 'NB (Walls)']
nr_folds = 10
eval='R2'
title_name='Wall Presence Analysis\nBoth Models'
label_name='Models'
save=False
save_name='both_models_walls_compare_r2.png'

'''
Compare final models
'''

# Data extraction --> Models
model_1_parameters = parameters[0:10]
model_2_parameters = parameters[10:]
model_3_parameters = parameters3[0:10]
model_4_parameters = parameters3[20:30]

nr_trials = len(parameters)
if(eval == 'Pearson'):
    x_std = np.array([np.std(model_1_parameters[:,0,1]), np.std(model_2_parameters[:,0,1])])
    y_std = np.array([np.std(model_1_parameters[:,1,1]), np.std(model_2_parameters[:,1,1])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,0]), np.mean(model_2_parameters[:,0,0])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,0]), np.mean(model_2_parameters[:,1,0])])
    f_model1 = np.mean((model_1_parameters[:,0,0], model_1_parameters[:,1,0]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,0], model_2_parameters[:,1,0]), axis=0)

elif(eval == 'R2'):
    x_std = np.array([np.std(model_1_parameters[:,0,3]), np.std(model_2_parameters[:,0,3]), np.std(model_3_parameters[:,0,3]), np.std(model_4_parameters[:,0,3])])
    y_std = np.array([np.std(model_1_parameters[:,1,3]), np.std(model_2_parameters[:,1,3]), np.std(model_3_parameters[:,1,3]), np.std(model_4_parameters[:,1,3])])
    x_mean = np.array([np.mean(model_1_parameters[:,0,2]), np.mean(model_2_parameters[:,0,2]), np.mean(model_3_parameters[:,0,2]), np.mean(model_4_parameters[:,0,2])])
    y_mean = np.array([np.mean(model_1_parameters[:,1,2]), np.mean(model_2_parameters[:,1,2]), np.mean(model_3_parameters[:,1,2]), np.mean(model_4_parameters[:,1,2])])
    f_model1 = np.mean((model_1_parameters[:,0,2], model_1_parameters[:,1,2]), axis=0)
    f_model2 = np.mean((model_2_parameters[:,0,2], model_2_parameters[:,1,2]), axis=0)
    f_model3 = np.mean((model_3_parameters[:,0,2], model_3_parameters[:,1,2]), axis=0)
    f_model4 = np.mean((model_4_parameters[:,0,2], model_4_parameters[:,1,2]), axis=0)

m_mean = np.mean((x_mean, y_mean), axis=0)
m_std = np.mean((x_std, y_std), axis=0)

# Extra Variables
error = m_std / np.sqrt(nr_trials)

#Begin Plotting
plt.figure(figsize=(6, 6), layout='constrained')
colors = ['tomato', 'orange', 'dodgerblue', 'darkturquoise', 'black']

#Scatter plotting
for i in range(nr_folds):
    plt.scatter(0, f_model1[i], color=colors[0], marker='x', alpha=0.5)
    plt.scatter(1, f_model2[i], color=colors[1], marker='x', alpha=0.5)
    plt.scatter(2, f_model3[i], color=colors[2], marker='x', alpha=0.5)
    plt.scatter(3, f_model4[i], color=colors[3], marker='x', alpha=0.5)

#Error bae plotting
plt.errorbar(0, m_mean[0], error[0], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[0], 
             label=f'Mean {eval} for Population Vector (Direct): {round(m_mean[0],2)}')
plt.errorbar(1, m_mean[1], error[1], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[1],
             label=f'Mean {eval} for Population Vector (Indirect): {round(m_mean[1],2)}')
plt.errorbar(2, m_mean[2], error[2], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[2],
             label=f'Mean {eval} for Naïve Bayes (Direct): {round(m_mean[2],2)}')
plt.errorbar(3, m_mean[3], error[3], ls='none', elinewidth=3.5, capthick=3.5, capsize=10, color=colors[3],
             label=f'Mean {eval} for Naïve Bayes (Indirect): {round(m_mean[3],2)}')
'''
#plt.errorbar(2, 0.85, error[1], ls='none', elinewidth=1, capthick=1, capsize=10, color=colors[2], label=f'Gold Standard {eval}: 0.85')
plt.scatter(2, 0.85, color=colors[2], alpha=1, marker='x', linewidth=10, label=f'Gold Standard {eval}: 0.85')
'''

# Add labels, title, and custom x-axis tick labels for the left y-axis
plt.ylabel(eval)
plt.title(title_name)
#ax1.set_xlabel(label_name)
plt.xticks(np.arange(len(model_names)), labels=np.array(model_names))
plt.ylim(m_mean[0]-0.05, m_mean[3]+0.05)
#plt.ylim(0, 1)
plt.xlim(-0.5, 3.5)
plt.legend(loc='upper left')

# Customize the grid
#plt.grid(True)
plt.grid(which='major', color='k', linestyle='-', linewidth=0.5)
plt.grid(which='minor', color='k', linestyle='-', linewidth=0.5, alpha=0.25)

# Enable minor ticks
plt.gca().minorticks_on()

# Customize the number of minor ticks
plt.gca().xaxis.set_minor_locator(AutoMinorLocator(4))  # 4 minor ticks between major ticks on x-axis
plt.gca().yaxis.set_minor_locator(AutoMinorLocator(2))  # 5 minor ticks between major ticks on y-axis

# Save figure if required
if save: plt.savefig(save_name)

# Show plot
plt.show()

## Predicetd Trajectories

In [ ]:
'''
Plots all trajectories separate (with trial duration in ms)
'''
def plot_allTrajectoriesS(dataset, original_dataset, nr_cols=4, save=False, save_name='plot.png'):
    #Gets the hand positions (trua and pred)
    cursor_pos = np.asarray(dataset['cursor_pos'], dtype='float64')
    true_pos = np.asarray(dataset['hand_pos'], dtype='float64')
    predicted_x = np.asarray(dataset['pred_X'], dtype='float64')
    predicted_y = np.asarray(dataset['pred_Y'], dtype='float64')
    true_x = true_pos[:,0]
    true_y = true_pos[:,1]
    cursor_x = cursor_pos[:,0]
    cursor_y = cursor_pos[:,1]
    
    #Gets the trials and time
    trial_ids = np.asarray(dataset['trial_id'], dtype='int64')
    mask = np.concatenate(([True], trial_ids[1:] != trial_ids[:-1]))
    split_indices = np.where(mask)[0]
    align_time = np.asarray(dataset['align_time'])
    
    #Gets the target positions per trial
    trials = np.unique(trial_ids)
    target_pos = np.asarray(original_dataset.trial_info[['active_pos_x', 'active_pos_y']], dtype='int64')[trials]
    barrier_pos = np.asarray(original_dataset.trial_info['barrier_pos'])[trials]
    barrier_lengths = np.array([len(inner_array) for inner_array in barrier_pos])
    
    
    #Splits all arrays acording to the trials
    predicted_x = np.split(predicted_x, split_indices)[1:]
    predicted_y = np.split(predicted_y, split_indices)[1:]
    align_time = np.split(align_time, split_indices)[1:]
    true_x = np.split(true_x, split_indices)[1:]
    true_y = np.split(true_y, split_indices)[1:]
    cursor_x = np.split(cursor_x, split_indices)[1:]
    cursor_y = np.split(cursor_y, split_indices)[1:]
    
    nr_trials = len(trials)
    #The ploting starts
    if(nr_trials%nr_cols == 0): nr_rows = nr_trials//nr_cols
    else: nr_rows = nr_trials//nr_cols + 1
    
    fig, axs = plt.subplots(nrows=nr_rows, ncols=nr_cols, figsize=(nr_cols*4, nr_rows*4.5))
    axs = axs.flatten()
    
    #Plotting
    for i in range(nr_trials):
        ax = axs[i]
        x_coords = true_x[i]
        y_coords = true_y[i]
        x_pred = predicted_x[i]
        y_pred = predicted_y[i]
        x_cursor = cursor_x[i]
        y_cursor = cursor_y[i]
        target_x = target_pos[i,0]
        target_y = target_pos[i,1]
        duration = int(align_time[i][-1])/1000000000
        distance = round(np.sqrt(np.power(x_pred[-1]-x_coords[-1],2) + np.power(y_pred[-1]-y_coords[-1],2)),3)
        barriers = barrier_pos[i]
    
        #Gets the limit
        x_lim = np.max([np.max(x_coords), np.max(x_pred), np.max(x_cursor), np.abs(np.min(x_coords)), np.abs(np.min(x_pred)), 
                        np.abs(np.min(x_cursor))])
        y_lim = np.max([np.max(y_coords), np.max(y_pred), np.max(y_cursor), np.abs(np.min(y_coords)), np.abs(np.min(y_pred)), 
                        np.abs(np.min(y_cursor))])
        lim_value = np.max([x_lim,y_lim])+15
        
        #Plots the barriers
        for j in range(barrier_lengths[i]):
            x, y, half_width, half_height = barrier_pos[i][j]
            left = x - half_width
            bottom = y - half_height
            width = 2 * half_width
            height = 2 * half_height
            ax.add_patch(plt.Rectangle((left, bottom), width, height, facecolor='gray'))
    
        ax.plot(x_cursor, y_cursor, label='Cursor Path')
        ax.plot(x_pred, y_pred, label='Predicted Path')
        #ax.plot(x_coords, y_coords, label='True Path')
        ax.scatter(target_x, target_y, label='Target Position', color='red')
        ax.scatter(x_pred[0], y_pred[0], label='Start Point', color='black')
        ax.set_title(f'Trial {trials[i]}\nLasted {duration} seconds\nDistance Final: {distance}')
        ax.set_xlim(-lim_value, lim_value)
        ax.set_ylim(-lim_value, lim_value)
        ax.legend()
        ax.grid(True)
    
    #plt.suptitle(f'Trial Trajectories', y=1, fontsize=20)
    plt.tight_layout()
    if(save): plt.savefig(save_name)
    plt.show()

In [ ]:
'''
Run 10 fold comparison model of No Walls Vs Walls Naïve Bayes
'''

#Separates the dataset into 3 sets
folds = 1
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, folds) 
o_neurons = standard_neurons

#Gets the test set with direct and indirect separation
trials = trials_present(o_test_dataset)
trials_all_direct = np.array(standardDS.trial_info[standardDS.trial_info['num_barriers'] == 0]['trial_id'])
trials_test_direct = trials[np.isin(trials, trials_all_direct)]

#Gets the separated test set
firing_rate(o_test_dataset, 35, panda=True)
o_direct_dataset = o_test_dataset[o_test_dataset['trial_id'].isin(trials_all_direct)].copy()
o_indirect_dataset = o_test_dataset[~o_test_dataset['trial_id'].isin(trials_all_direct)].copy()

print('Dataset is now divided')

#Runs the 10 fold for Naive Bayes
metrics = [['vel_X', 'pred_vel_X'], ['vel_Y', 'pred_vel_Y']]

print()
print('--------------Direct Test---------------')
print()
train_dataset = train_sets[0].copy()
direct_dataset = o_direct_dataset.copy()
neurons = o_neurons

#Converting sets to firing rate -> Otimization Needed!!!
firing_rate(train_dataset, 35, panda=True) 
print('Dataset is now in rates and not spike count')

#Fitting the Gaussian Tuning Curve
mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
print('All neurons have been fitted')

#Decoding Moment
decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
pred_vel, max_prob = SNB_apply(direct_dataset, decoder)
print('Decoder Applied')

#Column Management
apply_2D_data(direct_dataset, pred_vel)
apply_2D_data(direct_dataset, np.array(direct_dataset['hand_vel']), col_name='vel')
print('Column Management Done')

#Gets correlation values
data = []
entries = []
for metric in metrics:
    mean, std, r2_mean, r2_std, entry = get_pearson(direct_dataset, metric, extra_metrics=True)
    data.append([mean, std, r2_mean, r2_std])
    entries.append(entry)

data = np.array(data)
entries = np.array(entries)

print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Indirect Test---------------')

print()
train_dataset = train_sets[0].copy()
indirect_dataset = o_indirect_dataset.copy()
neurons = o_neurons

#Converting sets to firing rate -> Otimization Needed!!!
firing_rate(train_dataset, 35, panda=True) 
print('Dataset is now in rates and not spike count')

#Fitting the Gaussian Tuning Curve
mean_matrix, found_pairs, velocity_vector, grids, observed, predicted = tune_neurons3D(train_dataset, 'Gaussian', 15)
print('All neurons have been fitted')

#Decoding Moment
decoder = SNB_decoder(train_dataset, found_pairs, velocity_vector, predicted, spike_range=28)
pred_vel, max_prob = SNB_apply(indirect_dataset, decoder)
print('Decoder Applied')

#Column Management
apply_2D_data(indirect_dataset, pred_vel)
apply_2D_data(indirect_dataset, np.array(indirect_dataset['hand_vel']), col_name='vel')
print('Column Management Done')

#Gets correlation values
data = []
entries = []
for metric in metrics:
    mean, std, r2_mean, r2_std, entry = get_pearson(indirect_dataset, metric, extra_metrics=True)
    data.append([mean, std, r2_mean, r2_std])
    entries.append(entry)

data = np.array(data)
entries = np.array(entries)

print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

In [ ]:
convert_vel2pos(direct_dataset)
convert_vel2pos(indirect_dataset)

In [ ]:
plot_allTrajectoriesS(direct_dataset, standardDS, nr_cols=5, save=False, save_name='nb_direct_traj_all.pdf')

In [ ]:
trials = trials_present(direct_dataset)
desired_trials = [227, 302, 560, 344, 486]

desired_direct_dataset = direct_dataset[direct_dataset['trial_id'].isin(desired_trials)].copy()

In [ ]:
plot_allTrajectoriesS(desired_direct_dataset, standardDS, nr_cols=5, save=False, save_name='direct_nb_traj.png')

In [ ]:
plot_allTrajectoriesS(indirect_dataset, standardDS, nr_cols=5, save=False, save_name='nb_indirect_traj_all.pdf')

In [ ]:
trials = trials_present(indirect_dataset)
desired_trials = [233, 423, 381, 294, 159]

desired_indirect_dataset = indirect_dataset[indirect_dataset['trial_id'].isin(desired_trials)].copy()

In [ ]:
plot_allTrajectoriesS(desired_indirect_dataset, standardDS, nr_cols=5, save=False, save_name='indirect_nb_traj.png')

In [ ]:
'''
Run 10 fold comparison model of No Walls Vs Walls Population Vector
'''

#Separates the dataset into 3 sets
folds = 1
train_sets, val_sets, o_test_dataset = generate_sets(standardDS, 0.1, folds) 
o_neurons = standard_neurons

#Gets the test set with direct and indirect separation
trials = trials_present(o_test_dataset)
trials_all_direct = np.array(standardDS.trial_info[standardDS.trial_info['num_barriers'] == 0]['trial_id'])
trials_test_direct = trials[np.isin(trials, trials_all_direct)]

#Gets the separated test set
o_direct_dataset = o_test_dataset[o_test_dataset['trial_id'].isin(trials_all_direct)].copy()
o_indirect_dataset = o_test_dataset[~o_test_dataset['trial_id'].isin(trials_all_direct)].copy()

print('Dataset is now divided')

#Runs the 10 fold for population vector
metrics = [['mx', 'pred_cosX'],['my', 'pred_cosY']]

print()
print('--------------Direct Test---------------')
print()
print(f'Starting fold {1}')
train_dataset = train_sets[0].copy()
direct_ppv_dataset = o_direct_dataset.copy()
neurons = o_neurons

#Build the decoder
decoder = angle_decoder(train_dataset, 31, neurons)

#Updates the neurons
observed_matrix, predicted_matrix,_ = tune_neurons(train_dataset, decoder, 345, neurons)
decoder, neurons = remove_neurons(predicted_matrix, observed_matrix, direct_ppv_dataset, neurons, 11.4, metric='difference')

#Predicts every angle according to a decoder
angle_predictor(direct_ppv_dataset, decoder, 31, neurons, pred_pos=True)

#Gets correlation values
data = []
entries = []
for metric in metrics:
    mean, std, r2_mean, r2_std, entry = get_pearson(direct_ppv_dataset, metric, extra_metrics=True)
    data.append([mean, std, r2_mean, r2_std])
    entries.append(entry)

data = np.array(data)
entries = np.array(entries)
print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

print()
print('--------------Indirect Test---------------')
print()
print(f'Starting fold {1}')
train_dataset = train_sets[0].copy()
indirect_ppv_dataset = o_indirect_dataset.copy()
neurons = o_neurons

#Build the decoder
decoder = angle_decoder(train_dataset, 31, neurons)

#Updates the neurons
observed_matrix, predicted_matrix,_ = tune_neurons(train_dataset, decoder, 345, neurons)
decoder, neurons = remove_neurons(predicted_matrix, observed_matrix, indirect_ppv_dataset, neurons, 11.4, metric='difference')

#Predicts every angle according to a decoder
angle_predictor(indirect_ppv_dataset, decoder, 31, neurons, pred_pos=True)

#Gets correlation values
data = []
entries = []
for metric in metrics:
    mean, std, r2_mean, r2_std, entry = get_pearson(indirect_ppv_dataset, metric, extra_metrics=True)
    data.append([mean, std, r2_mean, r2_std])
    entries.append(entry)

data = np.array(data)
entries = np.array(entries)
print(f'X_Corr: {data[0,0]} ± {data[0,1]}\nY_Corr: {data[1,0]} ± {data[1,1]}')
print(f'X_R2: {data[0,2]} ± {data[0,3]}\nY_R2: {data[1,2]} ± {data[1,3]}')

os.system('afplay /System/Library/Sounds/Glass.aiff')

In [ ]:
plot_allTrajectoriesS(direct_ppv_dataset, standardDS, nr_cols=5, save=False, save_name='ppv_direct_traj_all.pdf')

In [ ]:
trials = trials_present(direct_ppv_dataset)
desired_trials = [4, 33, 72, 178, 459]

desired_direct_dataset_ppv = direct_ppv_dataset[direct_ppv_dataset['trial_id'].isin(desired_trials)].copy()

In [ ]:
plot_allTrajectoriesS(desired_direct_dataset_ppv, standardDS, nr_cols=5, save=True, save_name='direct_ppv_traj.png')

In [ ]:
plot_allTrajectoriesS(indirect_ppv_dataset, standardDS, nr_cols=5, save=False, save_name='ppv_indirect_traj_all.pdf')

In [ ]:
trials = trials_present(indirect_ppv_dataset)
desired_trials = [1912, 1947, 2118, 2197, 2244]

desired_indirect_dataset_ppv = indirect_ppv_dataset[indirect_ppv_dataset['trial_id'].isin(desired_trials)].copy()

In [ ]:
plot_allTrajectoriesS(desired_indirect_dataset_ppv, standardDS, nr_cols=5, save=True, save_name='indirect_ppv_traj.png')